In [1]:
import pandas as pd
import numpy as np
import random
import re
from requests import get
import urllib.request, json
from vectorhub.encoders.text.sentence_transformers import SentenceTransformer2Vec
from sentence_transformers import SentenceTransformer

np.random.seed(777)
random.seed(777)
random_state = 777

In [3]:
#Préparation du dataset à partir des données brutes

raw_data = pd.read_csv("/home/maver196/code/datagochi_mus_mov/data.csv")

#On elèves les rangées où le sondé n'a pas précisé son intention de vote, a indiqué qu'il ne voulait pas voter ou voter pour un autre parti
#Note: cela représente 23.25% des sondés
data_clean_intent = raw_data[(raw_data.op_intent_Other != 1) &
                             (raw_data.op_intent_no_vote != 1) &
                             raw_data.op_intent_CAQ.notnull()]

#On retire les variables inutiles ou inutilisables:
variables_to_drop = ['Unnamed: 0', 'id', 'postal_code', 'riding', 'time',
                      'op_intent_Other', 'op_intent_no_vote',
                       'op_intent', 'voting_probability']
regex_to_drop = 'macro|^party_id|^op_potential|^vote_pred|^people_pred|^health'
data_clean_columns = data_clean_intent.drop(variables_to_drop, axis=1)
data_clean_columns = data_clean_columns.drop(data_clean_columns.filter(regex=regex_to_drop).columns, axis=1) 
data_clean = data_clean_columns.dropna()

In [4]:
print(data_clean.shape)
data_clean.head()

(26227, 193)


,act_VisitsMuseumsGaleries,act_Fishing,act_Hunting,act_MotorizedOutdoorActivities,act_Volunteering,film_title,artist_name,title_album,pays_qc,immigrant,...,langFr,langEn,ses_languageOther,app_noTattoo,animal_farm,animal_domestic,animal_cat,animal_catNdog,animal_dog,animal_noPet
1,0.25,0.25,0.0,0.0,0.00,l'arrivée,imagine dragons,night visions,0,0,...,1,0,0,0,0,0,1,0,0,0
4,0.75,0.00,0.0,0.0,0.50,un prince à new york,joni mitchell,song to a seagull,0,0,...,1,0,0,1,0,0,1,0,0,0
5,0.50,0.00,0.0,0.0,0.25,avatar,patrick watson,waterproof9,0,0,...,1,0,0,0,0,0,1,0,0,0
7,0.25,0.00,0.0,0.5,0.50,le commando des bâtards,dua lipa,dua lipa,0,0,...,1,0,0,0,0,0,0,0,1,0
8,0.50,0.00,0.0,0.0,0.25,the crow,black veil brides,we stitch these wounds,0,0,...,1,0,0,0,0,0,0,0,1,0


In [ ]:
#correction titres de films
#On corrige les erreurs et on réduit le nombre de titres de films différents en n'utilisant que le titre original

data_clean.loc[data_clean['film_title']=="pulpe fiction", 'film_title'] = "pulp fiction"
data_clean.loc[data_clean['film_title']=="10 choses que je déteste de toi", 'film_title'] = "10 things i hate about you"
data_clean.loc[data_clean['film_title']=="(500) days of summer", 'film_title'] = "500 days of summer"
data_clean.loc[data_clean['film_title']=="@in the mood for love", 'film_title'] = "in the mood for love"
data_clean.loc[data_clean['film_title']=="007 - for our eyes only", 'film_title'] = "for our eyes only"
data_clean.loc[data_clean['film_title']=="'elf", 'film_title'] = "elf"
data_clean.loc[data_clean['film_title']=="007 quantum", 'film_title'] = "quantum of solace"
data_clean.loc[data_clean['film_title']=="007 skyfall", 'film_title'] = "skyfall"
data_clean.loc[data_clean['film_title']=="007 spectre", 'film_title'] = "spectre"
data_clean.loc[data_clean['film_title']=="007: licence to restore", 'film_title'] = "licence to kill"
data_clean.loc[data_clean['film_title']=="12 singes", 'film_title'] = "12 monkeys"
data_clean.loc[data_clean['film_title']=="120 battements par minute", 'film_title'] = "120 bpm"
data_clean.loc[data_clean['film_title']=="127 heures", 'film_title'] = "127 hours"
data_clean.loc[data_clean['film_title']=="13 ans, bientôt 30", 'film_title'] = "13 going on 30"
data_clean.loc[data_clean['film_title']=="13 days", 'film_title'] = "thirteen days"
data_clean.loc[data_clean['film_title']=="13 heures: le secret des soldats de benghazi", 'film_title'] = "13 hours"
data_clean.loc[data_clean['film_title']=="13 days", 'film_title'] = "thirteen days"
data_clean.loc[data_clean['film_title']=="17 ans, sérieusement ?", 'film_title'] = "the edge of seventeen"
data_clean.loc[data_clean['film_title']=="2001 : l'odyssée de l'espace", 'film_title'] = "2001: a space odyssey"
data_clean.loc[data_clean['film_title']=="17 ans, sérieusement ?", 'film_title'] = "the edge of seventeen"
data_clean.loc[data_clean['film_title']=="27 robes", 'film_title'] = "27 dresses"
data_clean.loc[data_clean['film_title']=="28 jours plus tard", 'film_title'] = "28 days later"
data_clean.loc[data_clean['film_title']=="3:10 pour yuma", 'film_title'] = "3:10 to yuma"
data_clean.loc[data_clean['film_title']=="365 jours", 'film_title'] = "365 days"
data_clean.loc[data_clean['film_title']=="365 jours : au lendemain", 'film_title'] = "365 days : this day"
data_clean.loc[data_clean['film_title']=="40 ans et encore puceau", 'film_title'] = "The 40-Year-Old Virgin"
data_clean.loc[data_clean['film_title']=="42 : l'histoire de jackie robinson", 'film_title'] = "42"
data_clean.loc[data_clean['film_title']=="45 ans", 'film_title'] = "45 years"
data_clean.loc[data_clean['film_title']=="58 minutes pour vivre", 'film_title'] = "die hard 2"
data_clean.loc[data_clean['film_title']=="7h58 ce samedi là", 'film_title'] = "before the devil knows you're dead"
data_clean.loc[data_clean['film_title']=="a brave heart", 'film_title'] = "braveheart"
data_clean.loc[data_clean['film_title']=="à cinq pieds de toi", 'film_title'] = "five feet appart"
data_clean.loc[data_clean['film_title']=="à couteaux tirés", 'film_title'] = "knives out"
data_clean.loc[data_clean['film_title']=="à l'épreuve de la mort", 'film_title'] = "grindhouse"
data_clean.loc[data_clean['film_title']=="à l'ombre de shawshank", 'film_title'] = "the shawshank redemption"
data_clean.loc[data_clean['film_title']=="à la conquête d'amy", 'film_title'] = "chasing amy"
data_clean.loc[data_clean['film_title']=="à la manière de carlito", 'film_title'] = "carlito's way"
data_clean.loc[data_clean['film_title']=="a la poursuite d'octobre rouge", 'film_title'] = "the hunt for red october"
data_clean.loc[data_clean['film_title']=="à la poursuite de mavericks", 'film_title'] = "chasing mavericks"
data_clean.loc[data_clean['film_title']=="à la poursuite du bonheur tour", 'film_title'] = "the pursuit of happyness"
data_clean.loc[data_clean['film_title']=="à la poursuite du diamant vert", 'film_title'] = "romancing the stone"
data_clean.loc[data_clean['film_title']=="à la rencontre de forrester", 'film_title'] = "finding forrester"
data_clean.loc[data_clean['film_title']=="à tous les garçons : p.s. je t'aime toujours", 'film_title'] = "to all the boys: p.s. i still love you"
data_clean.loc[data_clean['film_title']=="à tous les garçons que j'ai aimés", 'film_title'] = "to all the boys i've loved before"
data_clean.loc[data_clean['film_title']=="a tout jamais", 'film_title'] = "ever after"
data_clean.loc[data_clean['film_title']=="à travers l'univers", 'film_title'] = "across the universe"
data_clean.loc[data_clean['film_title']=="à travers le temps", 'film_title'] = "about time"
data_clean.loc[data_clean['film_title']=="a.i. intelligence artificielle", 'film_title'] = "a.i. artificial intelligence"
data_clean.loc[data_clean['film_title']=="académie de police", 'film_title'] = "police academy"
data_clean.loc[data_clean['film_title']=="académie de police 5: affectation miami beach", 'film_title'] = "police academy"
data_clean.loc[data_clean['film_title']=="ace ventura mène l'enquête", 'film_title'] = "ace ventura: pet detective"
data_clean.loc[data_clean['film_title']=="ace ventura: l'appel de la nature", 'film_title'] = "ace ventura: when nature calls"
data_clean.loc[data_clean['film_title']=="activité paranormale", 'film_title'] = "paranormal activity"
data_clean.loc[data_clean['film_title']=="adam à travers le temps", 'film_title'] = "the adam project"
data_clean.loc[data_clean['film_title']=="adieu las vegas", 'film_title'] = "leaving las vegas"
data_clean.loc[data_clean['film_title']=="after : la chute", 'film_title'] = "after we fell"
data_clean.loc[data_clean['film_title']=="after : la collision", 'film_title'] = "after we collided"
data_clean.loc[data_clean['film_title']=="after : la rencontre", 'film_title'] = "after"
data_clean.loc[data_clean['film_title']=="agence de renseignement", 'film_title'] = "central intelligence"
data_clean.loc[data_clean['film_title']=="agents troubles", 'film_title'] = "the departed"
data_clean.loc[data_clean['film_title']=="air bagnards", 'film_title'] = "con air"
data_clean.loc[data_clean['film_title']=="aladin", 'film_title'] = "aladdin"
data_clean.loc[data_clean['film_title']=="alerte à malibu", 'film_title'] = "baywatch"
data_clean.loc[data_clean['film_title']=="alerte rouge", 'film_title'] = "turning red"
data_clean.loc[data_clean['film_title']=="alice au pays des merveilles", 'film_title'] = "alice in wonderland"
data_clean.loc[data_clean['film_title']=="alien : covenant", 'film_title'] = "alien: covenant"
data_clean.loc[data_clean['film_title']=="alien : le 8e passager", 'film_title'] = "alien"
data_clean.loc[data_clean['film_title']=="aliens : le retour", 'film_title'] = "aliens"
data_clean.loc[data_clean['film_title']=="alita : l'ange conquérant", 'film_title'] = "alita: battle angel"
data_clean.loc[data_clean['film_title']=="almost everything you ever wanted to know about everything everywhere all at once", 'film_title'] = "everything everywhere all at once"
data_clean.loc[data_clean['film_title']=="amadeus director's cut", 'film_title'] = "amadeus"
data_clean.loc[data_clean['film_title']=="âme", 'film_title'] = "soul"
data_clean.loc[data_clean['film_title']=="american beauty: look closer...", 'film_title'] = "american beauty"
data_clean.loc[data_clean['film_title']=="american history", 'film_title'] = "american history x"
data_clean.loc[data_clean['film_title']=="amis modernes", 'film_title'] = "friends with benefits"
data_clean.loc[data_clean['film_title']=="amityville: la maison du diable", 'film_title'] = "the amityville horror"
data_clean.loc[data_clean['film_title']=="amour et magie", 'film_title'] = "practical magic"
data_clean.loc[data_clean['film_title']=="an old fashioned love story: making 'the bridges of madison county'", 'film_title'] = "the bridges of madison county"
data_clean.loc[data_clean['film_title']=="andre rieu - la vie est belle (life is beautiful)", 'film_title'] = "life is beautiful"
data_clean.loc[data_clean['film_title']=="anges & démons", 'film_title'] = "angels & demons"
data_clean.loc[data_clean['film_title']=="angry birds le film 2", 'film_title'] = "the angry birds movie 2"
data_clean.loc[data_clean['film_title']=="animaux nocturnes", 'film_title'] = "nocturnal animals"
data_clean.loc[data_clean['film_title']=="annabelle 3 : retour à la maison", 'film_title'] = "annabelle comes home"
data_clean.loc[data_clean['film_title']=="apocalypse now now", 'film_title'] = "apocalypse now"
data_clean.loc[data_clean['film_title']=="appelle-moi par ton nom", 'film_title'] = "call me by your name"
data_clean.loc[data_clean['film_title']=="armaguedon", 'film_title'] = "armagedon"
data_clean.loc[data_clean['film_title']=="armes aux poings", 'film_title'] = "to sir, with love"
data_clean.loc[data_clean['film_title']=="arrête-moi si tu peux", 'film_title'] = "catch me if you can"
data_clean.loc[data_clean['film_title']=="arrivals", 'film_title'] = "arrival"
data_clean.loc[data_clean['film_title']=="arthur, un amour de milliardaire", 'film_title'] = "arthur"
data_clean.loc[data_clean['film_title']=="as you wish: the story of 'the princess bride'", 'film_title'] = "the princess bride"
data_clean.loc[data_clean['film_title']=="assaut sur la maison-blanche", 'film_title'] = "olympus has fallen"
data_clean.loc[data_clean['film_title']=="assaut sur londres", 'film_title'] = "london has fallen"
data_clean.loc[data_clean['film_title']=="au-delà de nos rêves", 'film_title'] = "what dreams may come"
data_clean.loc[data_clean['film_title']=="au-delà des pins", 'film_title'] = "the place beyond the pines"
data_clean.loc[data_clean['film_title']=="aucun détour", 'film_title'] = "vehicle 19"
data_clean.loc[data_clean['film_title']=="austin powers - agent secret 00 sexe", 'film_title'] = "austin powers: the spy who shagged me"
data_clean.loc[data_clean['film_title']=="austin powers contre l'homme au membre d'or", 'film_title'] = "austin powers in goldmember"
data_clean.loc[data_clean['film_title']=="austin powers", 'film_title'] = "austin powers: international man of mystery"
data_clean.loc[data_clean['film_title']=="autant en emporte le vent", 'film_title'] = "gone with the wind"
data_clean.loc[data_clean['film_title']=="aux portes de l'enfer", 'film_title'] = "angel heart"
data_clean.loc[data_clean['film_title']=="avant minuit tout est possible", 'film_title'] = "before midnight"
data_clean.loc[data_clean['film_title']=="avant toi", 'film_title'] = "me before you"
data_clean.loc[data_clean['film_title']=="avatar 3d", 'film_title'] = "avatar"
data_clean.loc[data_clean['film_title']=="avatar: creating the world of pandora", 'film_title'] = "avatar"
data_clean.loc[data_clean['film_title']=="avatar: scene deconstruction", 'film_title'] = "avatar"
data_clean.loc[data_clean['film_title']=="avec amour, simon", 'film_title'] = "Love, Simon"
data_clean.loc[data_clean['film_title']=="avenger", 'film_title'] = "the avengers"
data_clean.loc[data_clean['film_title']=="avengers : la guerre de l'infini", 'film_title'] = "avengers: infinity war"
data_clean.loc[data_clean['film_title']=="avengers : phase finale", 'film_title'] = "avengers: endgame"
data_clean.loc[data_clean['film_title']=="babe - un cochon dans la ville", 'film_title'] = "babe: pig in the city"
data_clean.loc[data_clean['film_title']=="baby le chauffeur", 'film_title'] = "baby driver"
data_clean.loc[data_clean['film_title']=="back in time", 'film_title'] = "back to the future"
data_clean.loc[data_clean['film_title']=="back to the future part ii - back to the future night", 'film_title'] = "back to the future part ii"
data_clean.loc[data_clean['film_title']=="ballon chasseur : une vraie histoire de sous-estimés", 'film_title'] = "dodgeball: a true underdog story"
data_clean.loc[data_clean['film_title']=="barbie : dans la peau d'une princesse", 'film_title'] = "barbie princess adventure"
data_clean.loc[data_clean['film_title']=="basket spatial", 'film_title'] = "space jam"
data_clean.loc[data_clean['film_title']=="basket spatial : une nouvelle ère", 'film_title'] = "space jam: a new legacy"
data_clean.loc[data_clean['film_title']=="bataille navale", 'film_title'] = "battleship"
data_clean.loc[data_clean['film_title']=="batman : le commencement", 'film_title'] = "batman begins"
data_clean.loc[data_clean['film_title']=="batman : the long halloween 1ère partie", 'film_title'] = "batman: the long halloween, part one"
data_clean.loc[data_clean['film_title']=="batman vs superman : l'aube de la justice", 'film_title'] = "batman v superman: dawn of justice"
data_clean.loc[data_clean['film_title']=="beauté américaine", 'film_title'] = "american beauty"
data_clean.loc[data_clean['film_title']=="beauté cachée", 'film_title'] = "collateral beauty"
data_clean.loc[data_clean['film_title']=="ben-hur: a tale of the christ", 'film_title'] = "ben-hur"
data_clean.loc[data_clean['film_title']=="bételgeuse", 'film_title'] = "beetlejuice"
data_clean.loc[data_clean['film_title']=="bienvenue à gattaca", 'film_title'] = "gattaca"
data_clean.loc[data_clean['film_title']=="bienvenue à pleasantville", 'film_title'] = "pleasantville"
data_clean.loc[data_clean['film_title']=="big movie, big panic: junya sato on the bullet train", 'film_title'] = "bullet train"
data_clean.loc[data_clean['film_title']=="big fish: la légende du gros poisson", 'film_title'] = "big fish"
data_clean.loc[data_clean['film_title']=="birdman ou (les vertus insoupçonnées de l'ignorance)", 'film_title'] = "birdman or (the unexpected virtue of ignorance)"
data_clean.loc[data_clean['film_title']=="birds of prey et la fantabuleuse histoire de harley quinn", 'film_title'] = "birds of prey and the fantabulous emancipation of one harley quinn"
data_clean.loc[data_clean['film_title']=="bitter taste of love or frau schindler's list", 'film_title'] = "schindler's list"
data_clean.loc[data_clean['film_title']=="blanche-neige et les sept nains", 'film_title'] = "Snow White and the Seven Dwarfs"
data_clean.loc[data_clean['film_title']=="bleu d'enfer", 'film_title'] = "into the blue"
data_clean.loc[data_clean['film_title']=="blonde et légale", 'film_title'] = "legally blonde"
data_clean.loc[data_clean['film_title']=="bnp: les temps modernes", 'film_title'] = "modern times"
data_clean.loc[data_clean['film_title']=="bonjour l'amour", 'film_title'] = "the sweetest thing"
data_clean.loc[data_clean['film_title']=="bonjour, voisin", 'film_title'] = "a beautiful day in the neighborhood"
data_clean.loc[data_clean['film_title']=="bonjour l'amour", 'film_title'] = "the sweetest thing"
data_clean.loc[data_clean['film_title']=="bonne chance slevin", 'film_title'] = "lucky number slevin"
data_clean.loc[data_clean['film_title']=="bons baisers de russie", 'film_title'] = "from russia with love"
data_clean.loc[data_clean['film_title']=="bons garçons", 'film_title'] = "good boys"
data_clean.loc[data_clean['film_title']=="borat 2", 'film_title'] = "borat subsequent moviefilm"
data_clean.loc[data_clean['film_title']=="brillantine", 'film_title'] = "grease"
data_clean.loc[data_clean['film_title']=="brillantine 2", 'film_title'] = "grease 2"
data_clean.loc[data_clean['film_title']=="brisby et le secret de n.i.m.h.", 'film_title'] = "the secret of nimh"
data_clean.loc[data_clean['film_title']=="brooklin", 'film_title'] = "brooklyn"
data_clean.loc[data_clean['film_title']=="brûlé : un chef sous pression", 'film_title'] = "burnt"
data_clean.loc[data_clean['film_title']=="c'est ici que l'on se quitte", 'film_title'] = "this is where i leave you"
data_clean.loc[data_clean['film_title']=="c'est la fin", 'film_title'] = "this is the end"
data_clean.loc[data_clean['film_title']=="c'est la guerre", 'film_title'] = "this means war"
data_clean.loc[data_clean['film_title']=="c'est l'apocalypse'", 'film_title'] = "apocalypse now"
data_clean.loc[data_clean['film_title']=="c'est la vie", 'film_title'] = "that's life!"
data_clean.loc[data_clean['film_title']=="ça", 'film_title'] = "it"
data_clean.loc[data_clean['film_title']=="ça, c'est mon gars", 'film_title'] = "that's my boy"
data_clean.loc[data_clean['film_title']=="cache-cache", 'film_title'] = "hide and seek"
data_clean.loc[data_clean['film_title']=="capharnaüm", 'film_title'] = "capernaum"
data_clean.loc[data_clean['film_title']=="capitaine america : la guerre civile", 'film_title'] = "captain america: civil war"
data_clean.loc[data_clean['film_title']=="capitaine america : le premier vengeur", 'film_title'] = "captain america: the first avenger"
data_clean.loc[data_clean['film_title']=="capitaine america : le soldat de l'hiver", 'film_title'] = "captain america: the winter soldier"
data_clean.loc[data_clean['film_title']=="capitaine crochet", 'film_title'] = "hook"
data_clean.loc[data_clean['film_title']=="capitaine marvel", 'film_title'] = "captain marvel"
data_clean.loc[data_clean['film_title']=="capitaine phillips", 'film_title'] = "captain phillips"
data_clean.loc[data_clean['film_title']=="captain america", 'film_title'] = "captain america: the first avenger"
data_clean.loc[data_clean['film_title']=="captain america: the first avenger - the transformation", 'film_title'] = "captain america: the first avenger"
data_clean.loc[data_clean['film_title']=="cartographie des nuages", 'film_title'] = "cloud atlas"
data_clean.loc[data_clean['film_title']=="cashback: la beauté du temps", 'film_title'] = "cashback"
data_clean.loc[data_clean['film_title']=="ce qui se passe à vegas", 'film_title'] = "what happens in vegas"
data_clean.loc[data_clean['film_title']=="cendrillon", 'film_title'] = "Cinderella"
data_clean.loc[data_clean['film_title']=="chacun son combat", 'film_title'] = "never back down"
data_clean.loc[data_clean['film_title']=="chantez! 2", 'film_title'] = "sing 2"
data_clean.loc[data_clean['film_title']=="chantons sous la pluie", 'film_title'] = "singin' in the rain"
data_clean.loc[data_clean['film_title']=="charlie countryman doit mourir", 'film_title'] = "the necessary death of charlie countryman"
data_clean.loc[data_clean['film_title']=="charlie et la chocolaterie", 'film_title'] = "charlie and the chocolate factory"
data_clean.loc[data_clean['film_title']=="charlie et ses drôles de dames", 'film_title'] = "charlie's angels"
data_clean.loc[data_clean['film_title']=="chauffeur de taxi", 'film_title'] = "taxi driver"
data_clean.loc[data_clean['film_title']=="chaussure à son pied", 'film_title'] = "in her shoes"
data_clean.loc[data_clean['film_title']=="cherche homme parfait", 'film_title'] = "because i said so"
data_clean.loc[data_clean['film_title']=="cheval de guerre", 'film_title'] = "war horse"
data_clean.loc[data_clean['film_title']=="chevalier", 'film_title'] = "a knight's tale"
data_clean.loc[data_clean['film_title']=="chien", 'film_title'] = "dog"
data_clean.loc[data_clean['film_title']=="chien de flic", 'film_title'] = "k-9"
data_clean.loc[data_clean['film_title']=="chip on my shoulder: the cautionary tale of slapshot", 'film_title'] = "slap shot"
data_clean.loc[data_clean['film_title']=="cimetière vivant", 'film_title'] = "pet sematary"
data_clean.loc[data_clean['film_title']=="cinquante nuances de grey", 'film_title'] = "fifty shades of grey"
data_clean.loc[data_clean['film_title']=="cinquante nuances plus claires", 'film_title'] = "fifty shades freed"
data_clean.loc[data_clean['film_title']=="cinquante nuances plus sombres", 'film_title'] = "fifty shades darker"
data_clean.loc[data_clean['film_title']=="cité obscure", 'film_title'] = "dark city"
data_clean.loc[data_clean['film_title']=="clanches!", 'film_title'] = "speed"
data_clean.loc[data_clean['film_title']=="clochette et la fée pirate", 'film_title'] = "the pirate fairy"
data_clean.loc[data_clean['film_title']=="cochon", 'film_title'] = "pig"
data_clean.loc[data_clean['film_title']=="cœur vaillant", 'film_title'] = "braveheart"
data_clean.loc[data_clean['film_title']=="collatéral", 'film_title'] = "collateral"
data_clean.loc[data_clean['film_title']=="comédie", 'film_title'] = "comédie!"
data_clean.loc[data_clean['film_title']=="comme cendrillon", 'film_title'] = "a cinderella story"
data_clean.loc[data_clean['film_title']=="comme des bêtes", 'film_title'] = "the secret life of pets"
data_clean.loc[data_clean['film_title']=="comme des bêtes 2", 'film_title'] = "the secret life of pets 2"
data_clean.loc[data_clean['film_title']=="comment perdre son mec en 10 jours", 'film_title'] = "how to lose a guy in 10 days"
data_clean.loc[data_clean['film_title']=="commis en folie", 'film_title'] = "clerks"
data_clean.loc[data_clean['film_title']=="complètement dingo", 'film_title'] = "a goofy movie"
data_clean.loc[data_clean['film_title']=="compte sur moi", 'film_title'] = "stand by me"
data_clean.loc[data_clean['film_title']=="conan le barbare", 'film_title'] = "conan the barbarian"
data_clean.loc[data_clean['film_title']=="concours eurovision de la chanson : l'histoire de fire saga", 'film_title'] = "eurovision song contest: the story of fire saga"
data_clean.loc[data_clean['film_title']=="conduis mon char", 'film_title'] = "drive my car"
data_clean.loc[data_clean['film_title']=="coup musclé", 'film_title'] = "pain & gain"
data_clean.loc[data_clean['film_title']=="couple et couplets", 'film_title'] = "music and lyrics"
data_clean.loc[data_clean['film_title']=="cours après-moi,shérif", 'film_title'] = "smokey and the bandit"
data_clean.loc[data_clean['film_title']=="cowboys et aliens", 'film_title'] = "cowboys & aliens"
data_clean.loc[data_clean['film_title']=="crazy", 'film_title'] = "c.r.a.z.y."
data_clean.loc[data_clean['film_title']=="crazy rich à singapour", 'film_title'] = "crazy rich à asians"
data_clean.loc[data_clean['film_title']=="crier_au_loup", 'film_title'] = "cry wolf"
data_clean.loc[data_clean['film_title']=="croisière dans la jungle", 'film_title'] = "jungle cruise"
data_clean.loc[data_clean['film_title']=="cuirassé en péril", 'film_title'] = "under siege"
data_clean.loc[data_clean['film_title']=="dangereux 7", 'film_title'] = "furious 7"
data_clean.loc[data_clean['film_title']=="dans la peau de john malkovich", 'film_title'] = "being john malkovich"
data_clean.loc[data_clean['film_title']=="dans le sillage du titanic", 'film_title'] = "titanic"
data_clean.loc[data_clean['film_title']=="dans ses yeux", 'film_title'] = "the secret in their eyes"
data_clean.loc[data_clean['film_title']=="danse lascive", 'film_title'] = "dirty dancing"
data_clean.loc[data_clean['film_title']=="danse lascive 2: les nuits de la havane", 'film_title'] = "dirty dancing: havana nights"
data_clean.loc[data_clean['film_title']=="dancer dans le noir", 'film_title'] = "dancer in the dark"
data_clean.loc[data_clean['film_title']=="dansez dans les rues", 'film_title'] = "step up 2: the streets"
data_clean.loc[data_clean['film_title']=="dansez dans les rues 4", 'film_title'] = "step up revolution"
data_clean.loc[data_clean['film_title']=="décadence", 'film_title'] = "saw"
data_clean.loc[data_clean['film_title']=="décadence ii", 'film_title'] = "saw ii"
data_clean.loc[data_clean['film_title']=="décadence iii", 'film_title'] = "saw iii"
data_clean.loc[data_clean['film_title']=="décadence v", 'film_title'] = "saw v"
data_clean.loc[data_clean['film_title']=="défi bleu", 'film_title'] = "blue crush"
data_clean.loc[data_clean['film_title']=="déjà vu", 'film_title'] = "deja vu"
data_clean.loc[data_clean['film_title']=="demi-frères", 'film_title'] = "step brothers"
data_clean.loc[data_clean['film_title']=="demoiselles d'honneur", 'film_title'] = "bridesmaids"
data_clean.loc[data_clean['film_title']=="démolition", 'film_title'] = "demolition"
data_clean.loc[data_clean['film_title']=="denis la petite peste", 'film_title'] = "dennis the menace"
data_clean.loc[data_clean['film_title']=="dernier tour de table", 'film_title'] = "rounders"
data_clean.loc[data_clean['film_title']=="dernier train pour busan", 'film_title'] = "train to busan"
data_clean.loc[data_clean['film_title']=="des garçons épatants", 'film_title'] = "wonder boys"
data_clean.loc[data_clean['film_title']=="des gens comme les autres", 'film_title'] = "ordinary people"
data_clean.loc[data_clean['film_title']=="des pilotes en l'air", 'film_title'] = "hot shots!"
data_clean.loc[data_clean['film_title']=="des souris et des hommes", 'film_title'] = "of mice and men"
data_clean.loc[data_clean['film_title']=="destination ultime", 'film_title'] = "final destination"
data_clean.loc[data_clean['film_title']=="détestable moi", 'film_title'] = "despicable me"
data_clean.loc[data_clean['film_title']=="détestable moi 2", 'film_title'] = "despicable me 2"
data_clean.loc[data_clean['film_title']=="détour mortel", 'film_title'] = "wrong turn"
data_clean.loc[data_clean['film_title']=="détroit, ville du rock", 'film_title'] = "detroit rock city"
data_clean.loc[data_clean['film_title']=="deux épais en cavale", 'film_title'] = "dude, where's my car?"
data_clean.loc[data_clean['film_title']=="deux flics à miami", 'film_title'] = "miami vice"
data_clean.loc[data_clean['film_title']=="diaboliquement vôtre", 'film_title'] = "bedazzled"
data_clean.loc[data_clean['film_title']=="diamonds are forever: the don black songbook", 'film_title'] = "diamonds are forever"
data_clean.loc[data_clean['film_title']=="die fantastischen vier - captain fantastic tour - live in st. wendel", 'film_title'] = "captain fantastic"
data_clean.loc[data_clean['film_title']=="dieu n'est pas mort", 'film_title'] = "god's not dead"
data_clean.loc[data_clean['film_title']=="dinosaure", 'film_title'] = "dinosaur"
data_clean.loc[data_clean['film_title']=="divisé", 'film_title'] = "split"
data_clean.loc[data_clean['film_title']=="django déchaîné", 'film_title'] = "django unchained"
data_clean.loc[data_clean['film_title']=="docteur strange", 'film_title'] = "doctor strange"
data_clean.loc[data_clean['film_title']=="docteur strange dans le multivers de la folie", 'film_title'] = "doctor strange in the multiverse of madness"
data_clean.loc[data_clean['film_title']=="docteur strange in the multiverse of madness", 'film_title'] = "doctor strange in the multiverse of madness"
data_clean.loc[data_clean['film_title']=="dogme", 'film_title'] = "dogma"
data_clean.loc[data_clean['film_title']=="don't look up : déni cosmique", 'film_title'] = "don't look up"
data_clean.loc[data_clean['film_title']=="dora et la cité d'or perdue", 'film_title'] = "dora and the lost city of gold"
data_clean.loc[data_clean['film_title']=="double identité", 'film_title'] = "face/off"
data_clean.loc[data_clean['film_title']=="douleur et gloire", 'film_title'] = "pain and glory"
data_clean.loc[data_clean['film_title']=="douze hommes en colère", 'film_title'] = "12 angry men"
data_clean.loc[data_clean['film_title']=="downton abbey : une nouvelle ère", 'film_title'] = "downton abbey : a new era"
data_clean.loc[data_clean['film_title']=="dr. strange", 'film_title'] = "doctor strange"
data_clean.loc[data_clean['film_title']=="draco: la légende du dernier dragon", 'film_title'] = "dragonheart"
data_clean.loc[data_clean['film_title']=="dracula d'après l'oeuvre de bram stoker", 'film_title'] = "bram stocker's dracula"
data_clean.loc[data_clean['film_title']=="dragons", 'film_title'] = "how to train your dragon"
data_clean.loc[data_clean['film_title']=="dragons : le monde caché", 'film_title'] = "how to train your dragon: the hidden world"
data_clean.loc[data_clean['film_title']=="dragons 2", 'film_title'] = "how to train your dragon 2"
data_clean.loc[data_clean['film_title']=="drôle d'abeille", 'film_title'] = "bee movie"
data_clean.loc[data_clean['film_title']=="drôle de père", 'film_title'] = "big daddy"
data_clean.loc[data_clean['film_title']=="drôles de blondes", 'film_title'] = "white chicks"
data_clean.loc[data_clean['film_title']=="drôles de jeux", 'film_title'] = "funny games"
data_clean.loc[data_clean['film_title']=="druk", 'film_title'] = "drunk"
data_clean.loc[data_clean['film_title']=="du soleil plein la tête", 'film_title'] = "eternal sunshine of the spotless mind"
data_clean.loc[data_clean['film_title']=="dűne", 'film_title'] = "dune"
data_clean.loc[data_clean['film_title']=="dune: part two", 'film_title'] = "dune"
data_clean.loc[data_clean['film_title']=="dunkerque", 'film_title'] = "dunkirk"
data_clean.loc[data_clean['film_title']=="dutch angle: chas gerretsen & apocalypse now", 'film_title'] = "apocalypse now"
data_clean.loc[data_clean['film_title']=="e.t.: l'extra-terrestre", 'film_title'] = "e.t. the extra-terrestrial"
data_clean.loc[data_clean['film_title']=="éclair de lune", 'film_title'] = "moonstruck"
data_clean.loc[data_clean['film_title']=="edward au main dargent", 'film_title'] = "edward scissorhands"
data_clean.loc[data_clean['film_title']=="edward aux mains d'argent", 'film_title'] = "edward scissorhands"
data_clean.loc[data_clean['film_title']=="el camino : un film \"breaking bad\"", 'film_title'] = "el camino: a breaking bad movie"
data_clean.loc[data_clean['film_title']=="ella l'ensorcelée", 'film_title'] = "ella enchanted"
data_clean.loc[data_clean['film_title']=="elle", 'film_title'] = "her"
data_clean.loc[data_clean['film_title']=="elle a tout pour elle", 'film_title'] = "she's all that"
data_clean.loc[data_clean['film_title']=="elton john becoming rocketman", 'film_title'] = "rocketman"
data_clean.loc[data_clean['film_title']=="emmanuelle 4", 'film_title'] = "emmanuelle iv"
data_clean.loc[data_clean['film_title']=="en avant", 'film_title'] = "onward"
data_clean.loc[data_clean['film_title']=="en quête d'une galaxie", 'film_title'] = "galaxy quest"
data_clean.loc[data_clean['film_title']=="en souvenir des titans", 'film_title'] = "remember the titans"
data_clean.loc[data_clean['film_title']=="en temps", 'film_title'] = "in time"
data_clean.loc[data_clean['film_title']=="en-corps", 'film_title'] = "en corps"
data_clean.loc[data_clean['film_title']=="encanto, la fantastique famille madrigal", 'film_title'] = "encanto"
data_clean.loc[data_clean['film_title']=="encore 17 ans", 'film_title'] = "17 again"
data_clean.loc[data_clean['film_title']=="ennemi de l'état", 'film_title'] = "enemy of the state"
data_clean.loc[data_clean['film_title']=="ensemble, c'est tout", 'film_title'] = "beaches"
data_clean.loc[data_clean['film_title']=="entretien avec un vampire - les chroniques des vampires", 'film_title'] = "interview with the vampire: the vampire chronicles"
data_clean.loc[data_clean['film_title']=="escapade d'un soir", 'film_title'] = "adventures in babysitting"
data_clean.loc[data_clean['film_title']=="esclave pendant douze ans", 'film_title'] = "12 years a slave"
data_clean.loc[data_clean['film_title']=="escouade américaine: police du monde", 'film_title'] = "team america: world police"
data_clean.loc[data_clean['film_title']=="espionne", 'film_title'] = "spy"
data_clean.loc[data_clean['film_title']=="espions en herbe 2: l'île des rêves envolés", 'film_title'] = "spy kids 2: island of lost dreams"
data_clean.loc[data_clean['film_title']=="et si c'était vrai...", 'film_title'] = "just like heaven"
data_clean.loc[data_clean['film_title']=="été '95", 'film_title'] = "mid90s"
data_clean.loc[data_clean['film_title']=="eternal sunshine", 'film_title'] = "eternal sunshine of the spotless mind"
data_clean.loc[data_clean['film_title']=="éternelle adaline", 'film_title'] = "age of adaline"
data_clean.loc[data_clean['film_title']=="everafter", 'film_title'] = "ever after"
data_clean.loc[data_clean['film_title']=="explorez le festival du roi lion & de la jungle", 'film_title'] = "the lion king"
data_clean.loc[data_clean['film_title']=="extrême limite", 'film_title'] = "point break"
data_clean.loc[data_clean['film_title']=="f2 ferrovipathes", 'film_title'] = "t2 trainspotting"
data_clean.loc[data_clean['film_title']=="fabriqué en amérique", 'film_title'] = "american made"
data_clean.loc[data_clean['film_title']=="famille recomposée", 'film_title'] = "blended"
data_clean.loc[data_clean['film_title']=="fan fan", 'film_title'] = "fanfan"
data_clean.loc[data_clean['film_title']=="fantasme iii - le seigneur de la mort", 'film_title'] = "phantasm iii: lord of the dead"
data_clean.loc[data_clean['film_title']=="fantastique maître renard", 'film_title'] = "fantastic mr. fox"
data_clean.loc[data_clean['film_title']=="fast and furious - la saga no limit", 'film_title'] = "f9: the fast saga"
data_clean.loc[data_clean['film_title']=="fast and furry-ous", 'film_title'] = "fast and furious"
data_clean.loc[data_clean['film_title']=="faux départ", 'film_title'] = "failure to launch"
data_clean.loc[data_clean['film_title']=="ferrovipathes", 'film_title'] = "trainspotting"
data_clean.loc[data_clean['film_title']=="fiction pulpeuse", 'film_title'] = "pulp fiction"
data_clean.loc[data_clean['film_title']=="fight club: members only", 'film_title'] = "fight club"
data_clean.loc[data_clean['film_title']=="film de peur", 'film_title'] = "scary movie"
data_clean.loc[data_clean['film_title']=="film de peur 3", 'film_title'] = "scary movie 3"
data_clean.loc[data_clean['film_title']=="film de super-héros", 'film_title'] = "superhero movie"
data_clean.loc[data_clean['film_title']=="film for invisible ink, case no. 323. once upon a time in the west", 'film_title'] = "once upon a time in the west"
data_clean.loc[data_clean['film_title']=="fip the usual suspects", 'film_title'] = "the usual suspects"
data_clean.loc[data_clean['film_title']=="folies de graduation", 'film_title'] = "american pie"
data_clean.loc[data_clean['film_title']=="folies de graduation : la confrérie beta", 'film_title'] = "american pie presents: beta house"
data_clean.loc[data_clean['film_title']=="folies de graduation : la réunion", 'film_title'] = "american reunion"
data_clean.loc[data_clean['film_title']=="folies de graduation 2", 'film_title'] = "american pie 2"
data_clean.loc[data_clean['film_title']=="forbidden history revealed - ep 1: the egyptian presence in the americas and the pacific rim", 'film_title'] = "pacific rim"
data_clean.loc[data_clean['film_title']=="force majeure", 'film_title'] = "turist"
data_clean.loc[data_clean['film_title']=="ford contre ferrari", 'film_title'] = "ford v ferrari"
data_clean.loc[data_clean['film_title']=="fouquet, l'homme au masque de fer", 'film_title'] = "the man in the iron mask"
data_clean.loc[data_clean['film_title']=="frères d'arme", 'film_title'] = "band of brothers"
data_clean.loc[data_clean['film_title']=="friends (with benefits)", 'film_title'] = "friends with benefits"
data_clean.loc[data_clean['film_title']=="frisson", 'film_title'] = "scream"
data_clean.loc[data_clean['film_title']=="frissons", 'film_title'] = "scream"
data_clean.loc[data_clean['film_title']=="froid aux yeux", 'film_title'] = "bird box"
data_clean.loc[data_clean['film_title']=="furie", 'film_title'] = "the fury"
data_clean.loc[data_clean['film_title']=="game of thrones: the iron anniversary", 'film_title'] = "game of thrones"
data_clean.loc[data_clean['film_title']=="gangster américain", 'film_title'] = "american gangster"
data_clean.loc[data_clean['film_title']=="gants d'acier", 'film_title'] = "real steel"
data_clean.loc[data_clean['film_title']=="garçons sans honneur", 'film_title'] = "wedding crashers"
data_clean.loc[data_clean['film_title']=="garderie en folie", 'film_title'] = "daddy day care"
data_clean.loc[data_clean['film_title']=="gatsby le magnifique", 'film_title'] = "the great gatsby"
data_clean.loc[data_clean['film_title']=="geisha", 'film_title'] = "memoirs of a geisha"
data_clean.loc[data_clean['film_title']=="génération perdue", 'film_title'] = "the lost boys"
data_clean.loc[data_clean['film_title']=="generation x-treme", 'film_title'] = "american history x"
data_clean.loc[data_clean['film_title']=="george de la jungle", 'film_title'] = "george of the jungle"
data_clean.loc[data_clean['film_title']=="ghost buster", 'film_title'] = "ghostbusters"
data_clean.loc[data_clean['film_title']=="ghost in the shell : le film", 'film_title'] = "ghost in the shell"
data_clean.loc[data_clean['film_title']=="gladiateur", 'film_title'] = "gladiator"
data_clean.loc[data_clean['film_title']=="gladiateur, glaive et fantasmes", 'film_title'] = "gladiator"
data_clean.loc[data_clean['film_title']=="gladiateurs", 'film_title'] = "gladiator"
data_clean.loc[data_clean['film_title']=="gladiator eroticvs: the lesbian warriors", 'film_title'] = "gladiator"
data_clean.loc[data_clean['film_title']=="godzilla : roi des monstres", 'film_title'] = "godzilla: king of the monsters"
data_clean.loc[data_clean['film_title']=="goon : dur à cuire", 'film_title'] = "goon"
data_clean.loc[data_clean['film_title']=="graffiti américains", 'film_title'] = "american graffiti"
data_clean.loc[data_clean['film_title']=="grandes personnes", 'film_title'] = "grown ups"
data_clean.loc[data_clean['film_title']=="grandes personnes 2", 'film_title'] = "grown ups 2"
data_clean.loc[data_clean['film_title']=="gravité", 'film_title'] = "gravity"
data_clean.loc[data_clean['film_title']=="greased", 'film_title'] = "grease"
data_clean.loc[data_clean['film_title']=="grimsby - agent trop spécial", 'film_title'] = "grimsby"
data_clean.loc[data_clean['film_title']=="grrr", 'film_title'] = "rrrrrrr!!"
data_clean.loc[data_clean['film_title']=="guerrier", 'film_title'] = "warrior"
data_clean.loc[data_clean['film_title']=="halloween : la nuit des masques", 'film_title'] = "halloween"
data_clean.loc[data_clean['film_title']=="halloween h20 : 20 ans plus tard", 'film_title'] = "halloween h20: 20 years later"
data_clean.loc[data_clean['film_title']=="halloween tue", 'film_title'] = "halloween kills"
data_clean.loc[data_clean['film_title']=="hannibal lecter : les origines du mal", 'film_title'] = "hannibal rising"
data_clean.loc[data_clean['film_title']=="happy anniversary 007: 25 years of james bond", 'film_title'] = "dr. no"
data_clean.loc[data_clean['film_title']=="harcelé", 'film_title'] = "i.t."
data_clean.loc[data_clean['film_title']=="harold et kumar chassent le burger", 'film_title'] = "harold & kumar go to white castle"
data_clean.loc[data_clean['film_title']=="harold et kumar s'évadent de guantanamo", 'film_title'] = "harold & kumar escape from guantanamo bay"
data_clean.loc[data_clean['film_title']=="harry potter 20e anniversaire : retour à poudlard", 'film_title'] = "harry potter and the sorcerer's stone"
data_clean.loc[data_clean['film_title']=="harry potter and the philosopher's stone", 'film_title'] = "harry potter and the sorcerer's stone"
data_clean.loc[data_clean['film_title']=="harry potter à l'école des sorciers", 'film_title'] = "harry potter and the sorcerer's stone"
data_clean.loc[data_clean['film_title']=="harry potter et l'ordre du phénix", 'film_title'] = "harry potter and the order of the phoenix"
data_clean.loc[data_clean['film_title']=="harry potter et la chambre des secrets", 'film_title'] = "harry potter and the chamber of secrets"
data_clean.loc[data_clean['film_title']=="harry potter et la coupe de feu", 'film_title'] = "harry potter and the chamber of secrets"
data_clean.loc[data_clean['film_title']=="harry potter et le prince de sang-mêlé", 'film_title'] = "harry potter and the half-blood prince"
data_clean.loc[data_clean['film_title']=="harry potter et le prisonnier d'azkaban", 'film_title'] = "harry potter and the prisoner of azkaban"
data_clean.loc[data_clean['film_title']=="harry potter et les reliques de la mort : 1 ère partie", 'film_title'] = "harry potter and the deathly hallows: part 1"
data_clean.loc[data_clean['film_title']=="harry potter et les reliques de la mort : 2e partie", 'film_title'] = "harry potter and the deathly hallows: part 2"
data_clean.loc[data_clean['film_title']=="haut dans les airs", 'film_title'] = "up in the air"
data_clean.loc[data_clean['film_title']=="héréditaire", 'film_title'] = "hereditary"
data_clean.loc[data_clean['film_title']=="hérédité", 'film_title'] = "hereditary"
data_clean.loc[data_clean['film_title']=="heure limite", 'film_title'] = "rush hour"
data_clean.loc[data_clean['film_title']=="heure limite 2", 'film_title'] = "rush hour 2"
data_clean.loc[data_clean['film_title']=="heure limite 3", 'film_title'] = "rush hour 3"
data_clean.loc[data_clean['film_title']=="high school musical 3 : la dernière année", 'film_title'] = "high school musical: senior year"
data_clean.loc[data_clean['film_title']=="histoire de jouets", 'film_title'] = "toy story"
data_clean.loc[data_clean['film_title']=="histoire de jouets 2", 'film_title'] = "toy story 2"
data_clean.loc[data_clean['film_title']=="histoire de jouets 3", 'film_title'] = "toy story 3"
data_clean.loc[data_clean['film_title']=="histoire de jouets 4", 'film_title'] = "toy story 4"
data_clean.loc[data_clean['film_title']=="hommes en noir", 'film_title'] = "men in black"
data_clean.loc[data_clean['film_title']=="hommes en noir ii", 'film_title'] = "men in black ii"
data_clean.loc[data_clean['film_title']=="hommes en noir iii", 'film_title'] = "men in black 3"
data_clean.loc[data_clean['film_title']=="hope springs eternal: a look back at the shawshank redemption", 'film_title'] = "the shawshank redemption"
data_clean.loc[data_clean['film_title']=="hôtel mumbai", 'film_title'] = "hotel mumbai"
data_clean.loc[data_clean['film_title']=="how's it going to end - the making of 'the truman show'", 'film_title'] = "the truman show"
data_clean.loc[data_clean['film_title']=="how to train your dragon: snoggletog log", 'film_title'] = "how to train your dragon"
data_clean.loc[data_clean['film_title']=="dr. seuss' how the grinch stole christmas! and horton hears a who!", 'film_title'] = "how the grinch stole christmas!"
data_clean.loc[data_clean['film_title']=="howard, une nouvelle race de héros", 'film_title'] = "howard the duck"
data_clean.loc[data_clean['film_title']=="hudson hawk, gentleman et cambrioleur", 'film_title'] = "hudson hawk"
data_clean.loc[data_clean['film_title']=="huit en dessous", 'film_title'] = "eight below"
data_clean.loc[data_clean['film_title']=="hunger games : l'embrasement", 'film_title'] = "the hunger games: catching fire"
data_clean.loc[data_clean['film_title']=="hunger games : la révolte - dernière partie", 'film_title'] = "the hunger games: mockingjay - part 2"
data_clean.loc[data_clean['film_title']=="hunger games : le film", 'film_title'] = "the hunger games"
data_clean.loc[data_clean['film_title']=="hunger games : la révolte - partie i", 'film_title'] = "the hunger games: mockingjay - part 1"
data_clean.loc[data_clean['film_title']=="i, robot?", 'film_title'] = "i, robot"
data_clean.loc[data_clean['film_title']=="il danse avec les loups", 'film_title'] = "dances with wolves"
data_clean.loc[data_clean['film_title']=="il était une fois... à hollywood", 'film_title'] = "once upon a time in hollywood"
data_clean.loc[data_clean['film_title']=="il était une fois... lawrence d'arabie", 'film_title'] = "lawrence of arabia"
data_clean.loc[data_clean['film_title']=="il était une fois.... lost in translation", 'film_title'] = "lost in translation"
data_clean.loc[data_clean['film_title']=="il faut sauver le soldat ryan", 'film_title'] = "saving private ryan"
data_clean.loc[data_clean['film_title']=="il grande torino", 'film_title'] = "gran torino"
data_clean.loc[data_clean['film_title']=="il pleut des hamburgers", 'film_title'] = "cloudy with a chance of meatballs"
data_clean.loc[data_clean['film_title']=="impardonnable", 'film_title'] = "unforgiven"
data_clean.loc[data_clean['film_title']=="inception: jump right into the action", 'film_title'] = "inception"
data_clean.loc[data_clean['film_title']=="inception: music from the motion picture", 'film_title'] = "inception"
data_clean.loc[data_clean['film_title']=="inception: the cobol job", 'film_title'] = "inception"
data_clean.loc[data_clean['film_title']=="independence day : résurgence", 'film_title'] = "independence day: resurgence"
data_clean.loc[data_clean['film_title']=="independents' day", 'film_title'] = "independence day"
data_clean.loc[data_clean['film_title']=="indiana jones : à la recherche de l'âge d'or perdu", 'film_title'] = "indiana jones and the raiders of the lost ark"
data_clean.loc[data_clean['film_title']=="indiana jones et la dernière croisade", 'film_title'] = "indiana jones and the last crusade"
data_clean.loc[data_clean['film_title']=="indiana jones et le royaume du crâne de cristal", 'film_title'] = "indiana jones and the kingdom of the crystal skull"
data_clean.loc[data_clean['film_title']=="indiana jones et le temple maudit", 'film_title'] = "indiana jones and the temple of doom"
data_clean.loc[data_clean['film_title']=="inglourious basterds", 'film_title'] = "inglourious basterds"
data_clean.loc[data_clean['film_title']=="insaisissable", 'film_title'] = "now you see me"
data_clean.loc[data_clean['film_title']=="insaisissable 2", 'film_title'] = "now you see me 2"
data_clean.loc[data_clean['film_title']=="inside story: ferris bueller's day off", 'film_title'] = "ferris bueller's day off"
data_clean.loc[data_clean['film_title']=="insidieux", 'film_title'] = "insidious"
data_clean.loc[data_clean['film_title']=="insidieux : chapitre 2", 'film_title'] = "insidious: chapter 2"
data_clean.loc[data_clean['film_title']=="insidieux : la dernière clé", 'film_title'] = "insidious: the last key"
data_clean.loc[data_clean['film_title']=="instinct de survie", 'film_title'] = "47 meters down"
data_clean.loc[data_clean['film_title']=="interstellaire", 'film_title'] = "interstellar"
data_clean.loc[data_clean['film_title']=="interstellar wars", 'film_title'] = "interstellar"
data_clean.loc[data_clean['film_title']=="interstellar: nolan's odyssey", 'film_title'] = "interstellar"
data_clean.loc[data_clean['film_title']=="intraçable", 'film_title'] = "pay the ghost"
data_clean.loc[data_clean['film_title']=="invincible", 'film_title'] = "unbroken"
data_clean.loc[data_clean['film_title']=="ironman", 'film_title'] = "iron man"
data_clean.loc[data_clean['film_title']=="j't'aime mon homme", 'film_title'] = "i love you, man"
data_clean.loc[data_clean['film_title']=="jack et le haricot magique", 'film_title'] = "beanstalk"
data_clean.loc[data_clean['film_title']=="jackass toujours", 'film_title'] = "jackass forever"
data_clean.loc[data_clean['film_title']=="jamais entre amis", 'film_title'] = "sleeping with other people"
data_clean.loc[data_clean['film_title']=="james bond - licensed to die", 'film_title'] = "licence to kill"
data_clean.loc[data_clean['film_title']=="james bond 007 contre docteur no", 'film_title'] = "dr. no"
data_clean.loc[data_clean['film_title']=="james bond: the first 21 years", 'film_title'] = "dr. no"
data_clean.loc[data_clean['film_title']=="james corden's top gun training with tom cruise", 'film_title'] = "top gun"
data_clean.loc[data_clean['film_title']=="jay et bob contre-attaquent", 'film_title'] = "jay and bob strike back"
data_clean.loc[data_clean['film_title']=="je suis sam", 'film_title'] = "i am sam"
data_clean.loc[data_clean['film_title']=="je suis une légende", 'film_title'] = "i am legend"
data_clean.loc[data_clean['film_title']=="je t'aime, rosie", 'film_title'] = "love, rosie"
data_clean.loc[data_clean['film_title']=="invincible", 'film_title'] = "unbroken"
data_clean.loc[data_clean['film_title']=="je veux juste en finir", 'film_title'] = "i'm thinking of ending things"
data_clean.loc[data_clean['film_title']=="jeu d'enfant", 'film_title'] = "child's play"
data_clean.loc[data_clean['film_title']=="jeu de puissance", 'film_title'] = "the mighty ducks"
data_clean.loc[data_clean['film_title']=="jeu de puissance 2", 'film_title'] = "the mighty ducks"
data_clean.loc[data_clean['film_title']=="jeu de puissance 3", 'film_title'] = "the mighty ducks"
data_clean.loc[data_clean['film_title']=="jeu de survie", 'film_title'] = "survive the game"
data_clean.loc[data_clean['film_title']=="jeune fille interrompue", 'film_title'] = "girl, interrupted"
data_clean.loc[data_clean['film_title']=="jeunesse", 'film_title'] = "boyhood"
data_clean.loc[data_clean['film_title']=="jeux d'enfants", 'film_title'] = "child's play"
data_clean.loc[data_clean['film_title']=="jeux d'espionnage", 'film_title'] = "spy game"
data_clean.loc[data_clean['film_title']=="jeux de guerre", 'film_title'] = "war games"
data_clean.loc[data_clean['film_title']=="joe la crasse", 'film_title'] = "joe dirt"
data_clean.loc[data_clean['film_title']=="john wick : chapitre 2", 'film_title'] = "john wick: chapter 2"
data_clean.loc[data_clean['film_title']=="john wick : chapitre 3 - parabellum", 'film_title'] = "john wick : chapter 3 - parabellum"
data_clean.loc[data_clean['film_title']=="johnny english frappe à nouveau", 'film_title'] = "johnny english strikes again"
data_clean.loc[data_clean['film_title']=="johnny english renaît", 'film_title'] = "johnny english reborn"
data_clean.loc[data_clean['film_title']=="johnny mnémonique", 'film_title'] = "johnny mnemonic"
data_clean.loc[data_clean['film_title']=="jouer avec la mort", 'film_title'] = "the game"
data_clean.loc[data_clean['film_title']=="jour de formation", 'film_title'] = "training day"
data_clean.loc[data_clean['film_title']=="jours de tonnerre", 'film_title'] = "days of thunder"
data_clean.loc[data_clean['film_title']=="juge dredd", 'film_title'] = "judge dredd"
data_clean.loc[data_clean['film_title']=="jumanji : bienvenue dans la jungle", 'film_title'] = "jumanji: welcome to the jungle"
data_clean.loc[data_clean['film_title']=="jumanji : le prochain niveau", 'film_title'] = "jumanji: the next level"
data_clean.loc[data_clean['film_title']=="jurassic greatest moments: jurassic park to jurassic world", 'film_title'] = "jurassic park"
data_clean.loc[data_clean['film_title']=="k-pax - l'homme qui vient de loin", 'film_title'] = "k-pax"
data_clean.loc[data_clean['film_title']=="kill bill: the whole bloody affair", 'film_title'] = "kill bill: vol. 1"
data_clean.loc[data_clean['film_title']=="king richard : au-delà du jeu", 'film_title'] = "king richard"
data_clean.loc[data_clean['film_title']=="kingsman : le cercle d'or", 'film_title'] = "kingsman: the golden circle"
data_clean.loc[data_clean['film_title']=="kingsman : première mission", 'film_title'] = "the king's man"
data_clean.loc[data_clean['film_title']=="kingsman: services secrets", 'film_title'] = "kingsman: the secret service"
data_clean.loc[data_clean['film_title']=="kramer contre kramer", 'film_title'] = "kramer vs. kramer"
data_clean.loc[data_clean['film_title']=="kubo et l'épée magique", 'film_title'] = "kubo and the two strings"
data_clean.loc[data_clean['film_title']=="kubo", 'film_title'] = "kubo and the two strings"
data_clean.loc[data_clean['film_title']=="l'affaire pélican", 'film_title'] = "the pelican brief"
data_clean.loc[data_clean['film_title']=="l'affaire thomas crown", 'film_title'] = "the thomas crown affair"
data_clean.loc[data_clean['film_title']=="l'agence tous risques", 'film_title'] = "the a-team"
data_clean.loc[data_clean['film_title']=="l'agent fait la farce", 'film_title'] = "the naked gun: from the files of police squad!"
data_clean.loc[data_clean['film_title']=="l'agent fait la farce 2 1/2: l'odeur de la peur", 'film_title'] = "the naked gun 2½: the smell of fear"
data_clean.loc[data_clean['film_title']=="l'agent fait la farce 33 1/3: l'insulte finale", 'film_title'] = "naked gun 33 1/3: the final insult"
data_clean.loc[data_clean['film_title']=="l'aigle de la neuvième légion", 'film_title'] = "the eagle"
data_clean.loc[data_clean['film_title']=="l'amant", 'film_title'] = "the lover"
data_clean.loc[data_clean['film_title']=="l'antre de la folie", 'film_title'] = "in the mouth of madness"
data_clean.loc[data_clean['film_title']=="l'appel", 'film_title'] = "the call"
data_clean.loc[data_clean['film_title']=="l'apprenti sorcier", 'film_title'] = "the sorcerer's apprentice"
data_clean.loc[data_clean['film_title']=="l'apprentie sorcière", 'film_title'] = "bedknobs and broomsticks"
data_clean.loc[data_clean['film_title']=="l'arbre de la vie", 'film_title'] = "tree of life"
data_clean.loc[data_clean['film_title']=="l'arme fatale", 'film_title'] = "lethal weapon"
data_clean.loc[data_clean['film_title']=="l'arrivée", 'film_title'] = "arrival"
data_clean.loc[data_clean['film_title']=="l'ascension du chevalier noir", 'film_title'] = "the dark knight rises"
data_clean.loc[data_clean['film_title']=="l'attaque des clones", 'film_title'] = "star wars: episode ii - attack of the clones"
data_clean.loc[data_clean['film_title']=="l'attrape-parents", 'film_title'] = "the parent trap"
data_clean.loc[data_clean['film_title']=="l'aube de la planète des singes", 'film_title'] = "dawn of the planet of the apes"
data_clean.loc[data_clean['film_title']=="l'avènement", 'film_title'] = "the arrival"
data_clean.loc[data_clean['film_title']=="l'avocat du diable", 'film_title'] = "the devil's advocate"
data_clean.loc[data_clean['film_title']=="l'école du rock", 'film_title'] = "school of rock"
data_clean.loc[data_clean['film_title']=="l'écrivain fantôme", 'film_title'] = "the ghost writter"
data_clean.loc[data_clean['film_title']=="l'effroyable chose", 'film_title'] = "the thing"
data_clean.loc[data_clean['film_title']=="l'empire contre-attaque", 'film_title'] = "star wars: episode v - the empire strikes back"
data_clean.loc[data_clean['film_title']=="l'enfant lumière", 'film_title'] = "the shining"
data_clean.loc[data_clean['film_title']=="l'enfant sacré du tibet", 'film_title'] = "the golden child"
data_clean.loc[data_clean['film_title']=="l'enlèvement", 'film_title'] = "taken"
data_clean.loc[data_clean['film_title']=="l'enlèvement 2", 'film_title'] = "taken 2"
data_clean.loc[data_clean['film_title']=="l'épouse", 'film_title'] = "the wife"
data_clean.loc[data_clean['film_title']=="l'épreuve : le labyrinthe", 'film_title'] = "the maze runner"
data_clean.loc[data_clean['film_title']=="l'épreuve : le remède mortel", 'film_title'] = "maze runner: the death cure"
data_clean.loc[data_clean['film_title']=="l'ère de glace", 'film_title'] = "ice age"
data_clean.loc[data_clean['film_title']=="l'ère de glace: l'aube des dinosaures", 'film_title'] = "ice age: dawn of the dinosaurs"
data_clean.loc[data_clean['film_title']=="l'espion qui m'a dompée", 'film_title'] = "the spy who dumped me"
data_clean.loc[data_clean['film_title']=="l'été de mes 11 ans", 'film_title'] = "my girl"
data_clean.loc[data_clean['film_title']=="l'étoffe des héros", 'film_title'] = "the right stuff"
data_clean.loc[data_clean['film_title']=="l'étrange histoire de benjamin button", 'film_title'] = "the curious case of benjamin button"
data_clean.loc[data_clean['film_title']=="l'étrange noël de monsieur jack", 'film_title'] = "the nightmare before christmas"
data_clean.loc[data_clean['film_title']=="l'évadé d'alcatraz", 'film_title'] = "escape from alcatraz"
data_clean.loc[data_clean['film_title']=="l'éveil", 'film_title'] = "awakenings"
data_clean.loc[data_clean['film_title']=="l'éveil d'un champion", 'film_title'] = "the blind side"
data_clean.loc[data_clean['film_title']=="l'exorciste", 'film_title'] = "the exorcist"
data_clean.loc[data_clean['film_title']=="l'extraordinaire spider-man", 'film_title'] = "the amazing spider-man"
data_clean.loc[data_clean['film_title']=="l'heure la plus sombre", 'film_title'] = "darkest hour"
data_clean.loc[data_clean['film_title']=="l'histoire de pi", 'film_title'] = "life of pi"
data_clean.loc[data_clean['film_title']=="l'histoire sans fin", 'film_title'] = "the neverending story"
data_clean.loc[data_clean['film_title']=="l'homme au masque de fer", 'film_title'] = "the man in the iron mask"
data_clean.loc[data_clean['film_title']=="l'homme au pistolet d'or", 'film_title'] = "the man with the golden gun"
data_clean.loc[data_clean['film_title']=="l'homme bicentenaire", 'film_title'] = "bicentennial man"
data_clean.loc[data_clean['film_title']=="l'homme d'encino", 'film_title'] = "encino man"
data_clean.loc[data_clean['film_title']=="l'homme en feu", 'film_title'] = "man on fire"
data_clean.loc[data_clean['film_title']=="l'homme gris", 'film_title'] = "the grey man"
data_clean.loc[data_clean['film_title']=="l'homme invisible", 'film_title'] = "the invisible man"
data_clean.loc[data_clean['film_title']=="l'homme qui murmurait à l'oreille de chevaux", 'film_title'] = "the horse whisperer"
data_clean.loc[data_clean['film_title']=="l'homme, c'est elle", 'film_title'] = "she's the man"
data_clean.loc[data_clean['film_title']=="l'honneur à tout prix", 'film_title'] = "men of honnor"
data_clean.loc[data_clean['film_title']=="l'hôtel grand budapest", 'film_title'] = "the grand budapest hotel"
data_clean.loc[data_clean['film_title']=="l'île", 'film_title'] = "the island"
data_clean.loc[data_clean['film_title']=="l'île aux chiens", 'film_title'] = "isle of dogs"
data_clean.loc[data_clean['film_title']=="l'illusionniste", 'film_title'] = "the illusionist"
data_clean.loc[data_clean['film_title']=="l'imaginarium du docteur parnassus", 'film_title'] = "the imaginarium of doctor parnassus"
data_clean.loc[data_clean['film_title']=="l'inconnu de las vegas", 'film_title'] = "ocean's eleven"
data_clean.loc[data_clean['film_title']=="l'irlandais", 'film_title'] = "the guard"
data_clean.loc[data_clean['film_title']=="l'oeil de feu", 'film_title'] = "goldeneye"
data_clean.loc[data_clean['film_title']=="l'oeuvre de tim burton: la mariée cadavérique", 'film_title'] = "corpse bride"
data_clean.loc[data_clean['film_title']=="l'opéra de la terreur", 'film_title'] = "the evil dead"
data_clean.loc[data_clean['film_title']=="l'opéra de la terreur 2", 'film_title'] = "the evil dead 2"
data_clean.loc[data_clean['film_title']=="l'université des monstres", 'film_title'] = "monsters university"
data_clean.loc[data_clean['film_title']=="la 5ème vague", 'film_title'] = "the 5th wave"
data_clean.loc[data_clean['film_title']=="la bataille de la planète des singes", 'film_title'] = "battle for the planet of the apes"
data_clean.loc[data_clean['film_title']=="la belle et la bête", 'film_title'] = "beauty and the beast"
data_clean.loc[data_clean['film_title']=="la belle et le clochard", 'film_title'] = "lady and the tramp"
data_clean.loc[data_clean['film_title']=="la blonde de mon père", 'film_title'] = "stepmom"
data_clean.loc[data_clean['film_title']=="la bunny du campus", 'film_title'] = "the house bunny"
data_clean.loc[data_clean['film_title']=="la cabane dans les bois", 'film_title'] = "the cabin in the woods"
data_clean.loc[data_clean['film_title']=="la chose", 'film_title'] = "the thing"
data_clean.loc[data_clean['film_title']=="la chute du faucon noir", 'film_title'] = "black hawk down"
data_clean.loc[data_clean['film_title']=="la cité des anges", 'film_title'] = "city of angels"
data_clean.loc[data_clean['film_title']=="la cité des ténèbres: la coupe mortelle", 'film_title'] = "the mortal instruments: city of bones"
data_clean.loc[data_clean['film_title']=="la cloche et l'idiot", 'film_title'] = "dumb and dumber"
data_clean.loc[data_clean['film_title']=="la cloche et l'idiot 2", 'film_title'] = "dumb and dumber to"
data_clean.loc[data_clean['film_title']=="la coccinelle: tout équipée", 'film_title'] = "herbie fully loaded"
data_clean.loc[data_clean['film_title']=="la conjuration", 'film_title'] = "the conjuring"
data_clean.loc[data_clean['film_title']=="la conjuration 2", 'film_title'] = "the conjuring 2"
data_clean.loc[data_clean['film_title']=="la correction", 'film_title'] = "sleepers"
data_clean.loc[data_clean['film_title']=="la couleur des sentiments", 'film_title'] = "the help"
data_clean.loc[data_clean['film_title']=="la défense lincoln", 'film_title'] = "the lincoln lawyer"
data_clean.loc[data_clean['film_title']=="la dernière chanson", 'film_title'] = "the last song"
data_clean.loc[data_clean['film_title']=="la dernière maison sur la gauche", 'film_title'] = "the last house on the left"
data_clean.loc[data_clean['film_title']=="la faille", 'film_title'] = "fracture"
data_clean.loc[data_clean['film_title']=="la famille addams", 'film_title'] = "the addams family"
data_clean.loc[data_clean['film_title']=="la famille stone", 'film_title'] = "the family stone"
data_clean.loc[data_clean['film_title']=="la fille du train", 'film_title'] = "the girl on the train"
data_clean.loc[data_clean['film_title']=="la foire aux malheurs", 'film_title'] = "the money pit"
data_clean.loc[data_clean['film_title']=="la folle course vers sugarland", 'film_title'] = "the sugarland express"
data_clean.loc[data_clean['film_title']=="la folle histoire de l'espace", 'film_title'] = "spaceballs"
data_clean.loc[data_clean['film_title']=="la folle journée de ferris bueller", 'film_title'] = "ferris bueller's day off"
data_clean.loc[data_clean['film_title']=="la fontaine", 'film_title'] = "the fountain"
data_clean.loc[data_clean['film_title']=="la forme de l'eau", 'film_title'] = "the shape of water"
data_clean.loc[data_clean['film_title']=="la grande évasion", 'film_title'] = "the great escape"
data_clean.loc[data_clean['film_title']=="la guerre de demain", 'film_title'] = "the tomorrow war"
data_clean.loc[data_clean['film_title']=="la guerre de la planète des singes", 'film_title'] = "war for the planet of the apes"
data_clean.loc[data_clean['film_title']=="la guerre des étoiles", 'film_title'] = "star wars: episode iv - a new hope"
data_clean.loc[data_clean['film_title']=="la guerre des mariées", 'film_title'] = "bride wars"
data_clean.loc[data_clean['film_title']=="la guerre des tuques 3d", 'film_title'] = "la guerre des tuques"
data_clean.loc[data_clean['film_title']=="la haine qu'on donne", 'film_title'] = "the hate u give"
data_clean.loc[data_clean['film_title']=="la ligne verte", 'film_title'] = "the green mile"
data_clean.loc[data_clean['film_title']=="la liste de schindler", 'film_title'] = "schindler's list"
data_clean.loc[data_clean['film_title']=="la magie du destin", 'film_title'] = "sleepless in seattle"
data_clean.loc[data_clean['film_title']=="la main meurtrière", 'film_title'] = "idle hands"
data_clean.loc[data_clean['film_title']=="la maison près du lac", 'film_title'] = "the lake house"
data_clean.loc[data_clean['film_title']=="la maison que jack a construite", 'film_title'] = "the house that jack built"
data_clean.loc[data_clean['film_title']=="la maison sur la falaise", 'film_title'] = "life as a house"
data_clean.loc[data_clean['film_title']=="la mariée est en fuite!", 'film_title'] = "runaway bride"
data_clean.loc[data_clean['film_title']=="la matrice", 'film_title'] = "the matrix"
data_clean.loc[data_clean['film_title']=="la matrice rechargée", 'film_title'] = "the matrix reloaded"
data_clean.loc[data_clean['film_title']=="la matrice : résurrections", 'film_title'] = "the matrix resurrections"
data_clean.loc[data_clean['film_title']=="la mélodie du bonheur", 'film_title'] = "the sound of music"
data_clean.loc[data_clean['film_title']=="la mémoire dans la peau", 'film_title'] = "the bourne identity"
data_clean.loc[data_clean['film_title']=="la mission", 'film_title'] = "news of the world"
data_clean.loc[data_clean['film_title']=="la menace fantôme", 'film_title'] = "star wars: episode i - the phantom menace"
data_clean.loc[data_clean['film_title']=="la mince ligne rouge", 'film_title'] = "the thin red line"
data_clean.loc[data_clean['film_title']=="la momie", 'film_title'] = "the mummy"
data_clean.loc[data_clean['film_title']=="la mouche", 'film_title'] = "the fly"
data_clean.loc[data_clean['film_title']=="la mort dans la peau", 'film_title'] = "the bourne supremacy"
data_clean.loc[data_clean['film_title']=="la mort de staline", 'film_title'] = "the death of stalin"
data_clean.loc[data_clean['film_title']=="la neuvième porte", 'film_title'] = "the ninth gate"
data_clean.loc[data_clean['film_title']=="la nostalgie de l'ange", 'film_title'] = "the lovely bones"
data_clean.loc[data_clean['film_title']=="la note parfaite", 'film_title'] = "pitch perfect"
data_clean.loc[data_clean['film_title']=="la note parfaite 2", 'film_title'] = "pitch perfect 2"
data_clean.loc[data_clean['film_title']=="la note parfaite 3", 'film_title'] = "pitch perfect 3"
data_clean.loc[data_clean['film_title']=="la nuit des rois", 'film_title'] = "twelfth night or what you will"
data_clean.loc[data_clean['film_title']=="la nuit la plus longue", 'film_title'] = "from dusk till dawn"
data_clean.loc[data_clean['film_title']=="la part des anges", 'film_title'] = "the angels' share"
data_clean.loc[data_clean['film_title']=="la pat' patrouille : le film", 'film_title'] = "paw patrol: the movie"
data_clean.loc[data_clean['film_title']=="la petite sirène", 'film_title'] = "the little mermaid"
data_clean.loc[data_clean['film_title']=="la peur dans la peau: l'héritage de bourne", 'film_title'] = "the bourne legacy"
data_clean.loc[data_clean['film_title']=="la plage", 'film_title'] = "the beach"
data_clean.loc[data_clean['film_title']=="la planète des chimpanzés", 'film_title'] = "planet of the apes"
data_clean.loc[data_clean['film_title']=="la planète des singes", 'film_title'] = "planet of the apes"
data_clean.loc[data_clean['film_title']=="la plateforme", 'film_title'] = "the platform"
data_clean.loc[data_clean['film_title']=="la porte des étoiles", 'film_title'] = "stargate"
data_clean.loc[data_clean['film_title']=="la poursuite du bonheur", 'film_title'] = "the pursuit of happyness"
data_clean.loc[data_clean['film_title']=="la princesse bouton d'or", 'film_title'] = "the princess bride"
data_clean.loc[data_clean['film_title']=="la prophétie des ombres", 'film_title'] = "the mothman prophecies"
data_clean.loc[data_clean['film_title']=="la proposition", 'film_title'] = "the proposal"
data_clean.loc[data_clean['film_title']=="la purge", 'film_title'] = "the purge"
data_clean.loc[data_clean['film_title']=="la purge : anarchie", 'film_title'] = "the purge: anarchy"
data_clean.loc[data_clean['film_title']=="la reine des neiges", 'film_title'] = "frozen"
data_clean.loc[data_clean['film_title']=="la reine des neiges - bande annonce olaf", 'film_title'] = "frozen"
data_clean.loc[data_clean['film_title']=="la reine des neiges : une fête givrée", 'film_title'] = "frozen"
data_clean.loc[data_clean['film_title']=="la reine des neiges 2", 'film_title'] = "frozen ii"
data_clean.loc[data_clean['film_title']=="la religieuse", 'film_title'] = "the nun"
data_clean.loc[data_clean['film_title']=="la rivière du sixième jour", 'film_title'] = "a river runs through it"
data_clean.loc[data_clean['film_title']=="la route d'eldorado", 'film_title'] = "the road to el dorado"
data_clean.loc[data_clean['film_title']=="la saga twilight : hésitation", 'film_title'] = "the twilight saga: eclipse"
data_clean.loc[data_clean['film_title']=="la saga twilight : révélation - partie 1", 'film_title'] = "the twilight saga: breaking dawn - part 1"
data_clean.loc[data_clean['film_title']=="la saga twilight : tentation", 'film_title'] = "the twilight saga: new moon"
data_clean.loc[data_clean['film_title']=="la saga twilight : révélation - partie 2", 'film_title'] = "the twilight saga: breaking dawn - part 2"
data_clean.loc[data_clean['film_title']=="la sagesse de la pieuvre", 'film_title'] = "my octopus teacher"
data_clean.loc[data_clean['film_title']=="la saison miracle", 'film_title'] = "the miracle season"
data_clean.loc[data_clean['film_title']=="la série divergence : insurgés", 'film_title'] = "insurgent"
data_clean.loc[data_clean['film_title']=="la société des poètes disparus", 'film_title'] = "dead poets society"
data_clean.loc[data_clean['film_title']=="la sorcière", 'film_title'] = "the witch"
data_clean.loc[data_clean['film_title']=="la taupe", 'film_title'] = "tinker tailor soldier spy"
data_clean.loc[data_clean['film_title']=="la taupe 2", 'film_title'] = "tinker tailor soldier spy"
data_clean.loc[data_clean['film_title']=="la tempête", 'film_title'] = "the perfect storm"
data_clean.loc[data_clean['film_title']=="la tête dans les nuages", 'film_title'] = "dazed and confused"
data_clean.loc[data_clean['film_title']=="la théorie de l'univers", 'film_title'] = "the theory of everything"
data_clean.loc[data_clean['film_title']=="la tour infernale", 'film_title'] = "the towering inferno"
data_clean.loc[data_clean['film_title']=="la vengeance dans la peau", 'film_title'] = "the bourne ultimatum"
data_clean.loc[data_clean['film_title']=="la vie des autres", 'film_title'] = "the lives of others"
data_clean.loc[data_clean['film_title']=="la vie en soi", 'film_title'] = "life itself"
data_clean.loc[data_clean['film_title']=="la vie sauvage", 'film_title'] = "the wild"
data_clean.loc[data_clean['film_title']=="la vie, tout simplement", 'film_title'] = "life as we know it"
data_clean.loc[data_clean['film_title']=="la vie est belle", 'film_title'] = "life is beautiful"
data_clean.loc[data_clean['film_title']=="la vita è bella", 'film_title'] = "life is beautiful"
data_clean.loc[data_clean['film_title']=="la visite", 'film_title'] = "the visit"
data_clean.loc[data_clean['film_title']=="la voie de la justice", 'film_title'] = "just mercy"
data_clean.loc[data_clean['film_title']=="la voleuse de livres", 'film_title'] = "the book thief"
data_clean.loc[data_clean['film_title']=="labyrinthe", 'film_title'] = "labyrinth"
data_clean.loc[data_clean['film_title']=="laisse tomber, il te mérite pas!", 'film_title'] = "he's just not that into you"
data_clean.loc[data_clean['film_title']=="lancé-frappé 2", 'film_title'] = "slap shot 2: breaking the ice"
data_clean.loc[data_clean['film_title']=="lancer-frappé", 'film_title'] = "slap shot"
data_clean.loc[data_clean['film_title']=="lars et l'amour en boîte", 'film_title'] = "lars and the real girl"
data_clean.loc[data_clean['film_title']=="lawrence of arabia: the battle for the arab world", 'film_title'] = "lawrence of arabia"
data_clean.loc[data_clean['film_title']=="le 13ème guerrier", 'film_title'] = "the 13th warrior"
data_clean.loc[data_clean['film_title']=="le bal du monstre", 'film_title'] = "monster's ball"
data_clean.loc[data_clean['film_title']=="le balafré", 'film_title'] = "scarface"
data_clean.loc[data_clean['film_title']=="le batman", 'film_title'] = "the batman"
data_clean.loc[data_clean['film_title']=="le bébé de rosemary", 'film_title'] = "rosemary's baby"
data_clean.loc[data_clean['film_title']=="le bon côté des choses", 'film_title'] = "silver linings playbook"
data_clean.loc[data_clean['film_title']=="le bon, la brute et le truand", 'film_title'] = "the good, the bad and the ugly"
data_clean.loc[data_clean['film_title']=="le bras de fer", 'film_title'] = "over the top"
data_clean.loc[data_clean['film_title']=="le cadeau", 'film_title'] = "the gift"
data_clean.loc[data_clean['film_title']=="le carnet noir", 'film_title'] = "black book"
data_clean.loc[data_clean['film_title']=="le casse du siècle", 'film_title'] = "the big short"
data_clean.loc[data_clean['film_title']=="le cercle", 'film_title'] = "the ring"
data_clean.loc[data_clean['film_title']=="le champion", 'film_title'] = "the champ"
data_clean.loc[data_clean['film_title']=="le champs des rêves", 'film_title'] = "field of dreams"
data_clean.loc[data_clean['film_title']=="le chant de la mer", 'film_title'] = "song of the sea"
data_clean.loc[data_clean['film_title']=="le chanteur de noces", 'film_title'] = "the wedding singer"
data_clean.loc[data_clean['film_title']=="le chasseur de primes", 'film_title'] = "the bounty hunter"
data_clean.loc[data_clean['film_title']=="le château dans le ciel", 'film_title'] = "castle in the sky"
data_clean.loc[data_clean['film_title']=="le chevalier noir", 'film_title'] = "the dark knight"
data_clean.loc[data_clean['film_title']=="le choix", 'film_title'] = "the choice"
data_clean.loc[data_clean['film_title']=="le choix de sophie", 'film_title'] = "sophie's choice"
data_clean.loc[data_clean['film_title']=="le ciel peut attendre", 'film_title'] = "heaven can wait"
data_clean.loc[data_clean['film_title']=="le cinquième élément", 'film_title'] = "the fifth element"
data_clean.loc[data_clean['film_title']=="le code da vinci", 'film_title'] = "the da vinci code"
data_clean.loc[data_clean['film_title']=="le commando des bâtards", 'film_title'] = "inglourious basterds"
data_clean.loc[data_clean['film_title']=="le complot", 'film_title'] = "unlocked"
data_clean.loc[data_clean['film_title']=="le convoi", 'film_title'] = "precious cargo"
data_clean.loc[data_clean['film_title']=="le corps de jennifer", 'film_title'] = "jennifer's body"
data_clean.loc[data_clean['film_title']=="le crime de l'orient-express", 'film_title'] = "murder on the orient express"
data_clean.loc[data_clean['film_title']=="le défunt", 'film_title'] = "here comes mr. jordan"
data_clean.loc[data_clean['film_title']=="le démon de néon", 'film_title'] = "the neon demon"
data_clean.loc[data_clean['film_title']=="le dernier boy-scout", 'film_title'] = "the last boy scout"
data_clean.loc[data_clean['film_title']=="le dernier des mohicans", 'film_title'] = "the last of the mohicans"
data_clean.loc[data_clean['film_title']=="le dernier essai", 'film_title'] = "the longest yard"
data_clean.loc[data_clean['film_title']=="le dernier homme noir de san francisco", 'film_title'] = "the last black man in san francisco"
data_clean.loc[data_clean['film_title']=="le dernier pub avant la fin du monde", 'film_title'] = "the world's end"
data_clean.loc[data_clean['film_title']=="le dernier samouraï", 'film_title'] = "the last samurai"
data_clean.loc[data_clean['film_title']=="le dernier vice-roi des indes", 'film_title'] = "viceroy's house"
data_clean.loc[data_clean['film_title']=="le destin de will hunting", 'film_title'] = "good will hunting"
data_clean.loc[data_clean['film_title']=="le destructeur", 'film_title'] = "demolition man"
data_clean.loc[data_clean['film_title']=="le diable s'habille en prada", 'film_title'] = "the devil wears prada"
data_clean.loc[data_clean['film_title']=="le diamant de sang", 'film_title'] = "blood diamond"
data_clean.loc[data_clean['film_title']=="le dictateur", 'film_title'] = "the great dictator"
data_clean.loc[data_clean['film_title']=="le docteur jivago", 'film_title'] = "doctor zhivago"
data_clean.loc[data_clean['film_title']=="le facteur", 'film_title'] = "the postman"
data_clean.loc[data_clean['film_title']=="le fantôme du paradis", 'film_title'] = "phantom of the paradise"
data_clean.loc[data_clean['film_title']=="le faucon au beurre d'arachide", 'film_title'] = "the peanut butter falcon"
data_clean.loc[data_clean['film_title']=="le feu de la danse", 'film_title'] = "flashdance"
data_clean.loc[data_clean['film_title']=="le fil caché", 'film_title'] = "phantom thread"
data_clean.loc[data_clean['film_title']=="le film lego", 'film_title'] = "the lego movie"
data_clean.loc[data_clean['film_title']=="le flic de beverly hills", 'film_title'] = "beverly hills cop"
data_clean.loc[data_clean['film_title']=="le fondateur", 'film_title'] = "the founder"
data_clean.loc[data_clean['film_title']=="le fugitif", 'film_title'] = "the fugitive"
data_clean.loc[data_clean['film_title']=="le garde du corps", 'film_title'] = "the bodyguard"
data_clean.loc[data_clean['film_title']=="le gardien", 'film_title'] = "the guardian"
data_clean.loc[data_clean['film_title']=="le grand coup", 'film_title'] = "the score"
data_clean.loc[data_clean['film_title']=="le grincheux", 'film_title'] = "how the grinch stole christmas"
data_clean.loc[data_clean['film_title']=="le grincheux qui voulait gâcher noël!", 'film_title'] = "how the grinch stole christmas"
data_clean.loc[data_clean['film_title']=="le guide galactique", 'film_title'] = "the hitchhiker's guide to the galaxy"
data_clean.loc[data_clean['film_title']=="le hobbit : la bataille des cinq armées", 'film_title'] = "the Hobbit: the battle of the five armies"
data_clean.loc[data_clean['film_title']=="le hobbit : la désolation de smaug", 'film_title'] = "the hobbit: the desolation of smaug"
data_clean.loc[data_clean['film_title']=="le hobbit : un voyage inattendu", 'film_title'] = "the hobbit: an unexpected journey"
data_clean.loc[data_clean['film_title']=="le hobbit: le retour du roi du cantal", 'film_title'] = "the lord of the rings: the return of the king"
data_clean.loc[data_clean['film_title']=="le homard", 'film_title'] = "the lobster"
data_clean.loc[data_clean['film_title']=="le jardin secret", 'film_title'] = "the secret garden"
data_clean.loc[data_clean['film_title']=="le jeu de l'imitation", 'film_title'] = "the imitation game"
data_clean.loc[data_clean['film_title']=="le homard", 'film_title'] = "the lobster"
data_clean.loc[data_clean['film_title']=="le joker", 'film_title'] = "joker"
data_clean.loc[data_clean['film_title']=="le jour d'après", 'film_title'] = "the day after tomorrow"
data_clean.loc[data_clean['film_title']=="le jour de l'indépendance", 'film_title'] = "independance day"
data_clean.loc[data_clean['film_title']=="le jour de la marmotte", 'film_title'] = "groundhog day"
data_clean.loc[data_clean['film_title']=="le journal d'une princesse", 'film_title'] = "the princess diaries"
data_clean.loc[data_clean['film_title']=="le journal d'une princesse 2: les fiançailles royale", 'film_title'] = "the princess diaries 2: royal engagement"
data_clean.loc[data_clean['film_title']=="le journal de bridget jones", 'film_title'] = "bridget jones's diary"
data_clean.loc[data_clean['film_title']=="le justicier", 'film_title'] = "the equalizer"
data_clean.loc[data_clean['film_title']=="le karaté kid", 'film_title'] = "the karate kid"
data_clean.loc[data_clean['film_title']=="le labyrinthe", 'film_title'] = "labyrinth"
data_clean.loc[data_clean['film_title']=="le labyrinthe de pan", 'film_title'] = "pan's labyrinth"
data_clean.loc[data_clean['film_title']=="le lagon bleu", 'film_title'] = "the blue lagoon"
data_clean.loc[data_clean['film_title']=="le livre de green", 'film_title'] = "green book"
data_clean.loc[data_clean['film_title']=="le lorax", 'film_title'] = "the lorax"
data_clean.loc[data_clean['film_title']=="le loup de wall street", 'film_title'] = "the wolf of wall street"
data_clean.loc[data_clean['film_title']=="le lutin", 'film_title'] = "elf"
data_clean.loc[data_clean['film_title']=="le machiniste", 'film_title'] = "the machinist"
data_clean.loc[data_clean['film_title']=="le magicien d'oz", 'film_title'] = "the wizard of oz"
data_clean.loc[data_clean['film_title']=="le maître de la scène", 'film_title'] = "the greatest showman"
data_clean.loc[data_clean['film_title']=="le masque", 'film_title'] = "the mask"
data_clean.loc[data_clean['film_title']=="le mercenaire", 'film_title'] = "last man standing"
data_clean.loc[data_clean['film_title']=="le meurtre dans le sang", 'film_title'] = "natural born killers"
data_clean.loc[data_clean['film_title']=="le mississippi brûle", 'film_title'] = "mississippi burning"
data_clean.loc[data_clean['film_title']=="le moment de vérité", 'film_title'] = "the karate kid"
data_clean.loc[data_clean['film_title']=="le monde de nathan", 'film_title'] = "x+y"
data_clean.loc[data_clean['film_title']=="le monde perdu: jurassic park", 'film_title'] = "the lost world: jurassic park"
data_clean.loc[data_clean['film_title']=="le monde selon wayne", 'film_title'] = "wayne's world"
data_clean.loc[data_clean['film_title']=="le monstre des mers", 'film_title'] = "the sea beast"
data_clean.loc[data_clean['film_title']=="le moulin rouge: the cabaret of life", 'film_title'] = "moulin rouge!"
data_clean.loc[data_clean['film_title']=="le nom de la rose", 'film_title'] = "the name of the rose"
data_clean.loc[data_clean['film_title']=="le nom de la rose : le documentaire", 'film_title'] = "the name of the rose"
data_clean.loc[data_clean['film_title']=="le nouveau", 'film_title'] = "the new guy"
data_clean.loc[data_clean['film_title']=="le parc jurassique", 'film_title'] = "jurassic park"
data_clean.loc[data_clean['film_title']=="le parc jurassique iii", 'film_title'] = "jurassic park iii"
data_clean.loc[data_clean['film_title']=="le parrain", 'film_title'] = "the godfather"
data_clean.loc[data_clean['film_title']=="le parrain ii", 'film_title'] = "the godfather part ii"
data_clean.loc[data_clean['film_title']=="le parrain iii", 'film_title'] = "the godfather part iii"
data_clean.loc[data_clean['film_title']=="le passeur", 'film_title'] = "the giver"
data_clean.loc[data_clean['film_title']=="le patriote", 'film_title'] = "the patriot"
data_clean.loc[data_clean['film_title']=="le peintre des vanités", 'film_title'] = "tulip fever"
data_clean.loc[data_clean['film_title']=="le père", 'film_title'] = "the father"
data_clean.loc[data_clean['film_title']=="le père de la mariée", 'film_title'] = "father of the bride"
data_clean.loc[data_clean['film_title']=="le plus beau des chemins", 'film_title'] = "the longest ride"
data_clean.loc[data_clean['film_title']=="le porte-bonheur", 'film_title'] = "the lucky one"
data_clean.loc[data_clean['film_title']=="le porteur d'eau", 'film_title'] = "the waterboy"
data_clean.loc[data_clean['film_title']=="le pouilleux millionnaire", 'film_title'] = "slumdog millionaire"
data_clean.loc[data_clean['film_title']=="le prédateur", 'film_title'] = "predator"
data_clean.loc[data_clean['film_title']=="le premier homme", 'film_title'] = "the first man"
data_clean.loc[data_clean['film_title']=="le prince d'égypte", 'film_title'] = "the prince of egypt"
data_clean.loc[data_clean['film_title']=="le prince et moi", 'film_title'] = "the prince and me"
data_clean.loc[data_clean['film_title']=="le prix à payer", 'film_title'] = "the high cost of living"
data_clean.loc[data_clean['film_title']=="le projet blair", 'film_title'] = "the blair witch project"
data_clean.loc[data_clean['film_title']=="le réseau social", 'film_title'] = "the social network"
data_clean.loc[data_clean['film_title']=="le retour de batman", 'film_title'] = "batman returns"
data_clean.loc[data_clean['film_title']=="le retour de danny ocean", 'film_title'] = "ocean's twelve"
data_clean.loc[data_clean['film_title']=="le retour de papa", 'film_title'] = "daddy's home"
data_clean.loc[data_clean['film_title']=="le retour du jedi", 'film_title'] = "star wars: episode vi - return of the jedi"
data_clean.loc[data_clean['film_title']=="le réveil des gardiens", 'film_title'] = "rise of the guardians"
data_clean.loc[data_clean['film_title']=="le revenant", 'film_title'] = "the revenant"
data_clean.loc[data_clean['film_title']=="le rituel", 'film_title'] = "the ritual"
data_clean.loc[data_clean['film_title']=="le rocher", 'film_title'] = "the rock"
data_clean.loc[data_clean['film_title']=="le rôdeur", 'film_title'] = "nightcrawler"
data_clean.loc[data_clean['film_title']=="le roi", 'film_title'] = "the king"
data_clean.loc[data_clean['film_title']=="le roi arthur : la légende d'excalibur", 'film_title'] = "king arthur: legend of the sword"
data_clean.loc[data_clean['film_title']=="le roi lion", 'film_title'] = "the lion king"
data_clean.loc[data_clean['film_title']=="le roi lion 1½", 'film_title'] = "the lion king 1½"
data_clean.loc[data_clean['film_title']=="le roi pêcheur", 'film_title'] = "the fisher king"
data_clean.loc[data_clean['film_title']=="le royaume des cieux", 'film_title'] = "kingdom of heaven"
data_clean.loc[data_clean['film_title']=="le sapin a des boules", 'film_title'] = "national lampoon's christmas vacation"
data_clean.loc[data_clean['film_title']=="le secret de brokeback mountain", 'film_title'] = "brokeback mountain"
data_clean.loc[data_clean['film_title']=="le secret est dans la sauce", 'film_title'] = "fried green tomatoes"
data_clean.loc[data_clean['film_title']=="le seigneur des anneaux", 'film_title'] = "the lord of the rings: the fellowship of the ring"
data_clean.loc[data_clean['film_title']=="le seigneur des anneaux: la communauté de l'anneau", 'film_title'] = "the lord of the rings: the fellowship of the ring"
data_clean.loc[data_clean['film_title']=="le seigneur des anneaux: le retour du roi", 'film_title'] = "the lord of the rings: the return of the king"
data_clean.loc[data_clean['film_title']=="le seigneur des anneaux: les deux tours", 'film_title'] = "the lord of the rings: the two towers"
data_clean.loc[data_clean['film_title']=="le sel de la terre", 'film_title'] = "the salt of the earth"
data_clean.loc[data_clean['film_title']=="le show truman", 'film_title'] = "the truman show"
data_clean.loc[data_clean['film_title']=="le silence des agneaux", 'film_title'] = "the silence of the lambs"
data_clean.loc[data_clean['film_title']=="le sixième sens", 'film_title'] = "the sixth sense"
data_clean.loc[data_clean['film_title']=="le soleil de minuit", 'film_title'] = "midnight sun"
data_clean.loc[data_clean['film_title']=="le stagiaire", 'film_title'] = "the intern"
data_clean.loc[data_clean['film_title']=="le téléphone noir", 'film_title'] = "the black phone"
data_clean.loc[data_clean['film_title']=="le temps des gitans punk opera", 'film_title'] = "time of the gypsies"
data_clean.loc[data_clean['film_title']=="le tout pour le tout", 'film_title'] = "bring it on"
data_clean.loc[data_clean['film_title']=="le traqueur", 'film_title'] = "the rundown"
data_clean.loc[data_clean['film_title']=="le trône de fer : the last watch", 'film_title'] = "game of thrones"
data_clean.loc[data_clean['film_title']=="le tunnel", 'film_title'] = "the tunnel"
data_clean.loc[data_clean['film_title']=="le village", 'film_title'] = "the village"
data_clean.loc[data_clean['film_title']=="le vingtième siècle", 'film_title'] = "the 20th century"
data_clean.loc[data_clean['film_title']=="le wolverine", 'film_title'] = "the wolverine"
data_clean.loc[data_clean['film_title']=="le zodiaque", 'film_title'] = "zodiac"
data_clean.loc[data_clean['film_title']=="lego avengers infinity war: endgame", 'film_title'] = "avengers: infinity war"
data_clean.loc[data_clean['film_title']=="lendemain de veille", 'film_title'] = "the hangover"
data_clean.loc[data_clean['film_title']=="lendemain de veille 2", 'film_title'] = "the hangover part ii"
data_clean.loc[data_clean['film_title']=="lendemain de veille 3", 'film_title'] = "the hangover part iii"
data_clean.loc[data_clean['film_title']=="les 101 dalmatiens", 'film_title'] = "one hundred and one dalmatians"
data_clean.loc[data_clean['film_title']=="les 50 premiers rendez-vous", 'film_title'] = "50 first dates"
data_clean.loc[data_clean['film_title']=="les 8 enragés", 'film_title'] = "the hateful eight"
data_clean.loc[data_clean['film_title']=="les affranchis", 'film_title'] = "goodfellas"
data_clean.loc[data_clean['film_title']=="les âmes vagabondes", 'film_title'] = "the host (2013)"
data_clean.loc[data_clean['film_title']=="les animaux fantastiques", 'film_title'] = "fantastic beasts and where to find them"
data_clean.loc[data_clean['film_title']=="les animaux fantastiques  : les secrets de dumbledore", 'film_title'] = "fantastic beasts: the secrets of dumbledore"
data_clean.loc[data_clean['film_title']=="les apparences", 'film_title'] = "gone girl"
data_clean.loc[data_clean['film_title']=="les aristochats", 'film_title'] = "the aristocats"
data_clean.loc[data_clean['film_title']=="les avengers", 'film_title'] = "the avengers"
data_clean.loc[data_clean['film_title']=="les aventures de buckaroo banzaï à travers la huitième dimension", 'film_title'] = "the adventures of buckaroo banzai across the 8th dimension"
data_clean.loc[data_clean['film_title']=="les aventures de jack burton dans les griffes du mandarin", 'film_title'] = "big trouble in little china"
data_clean.loc[data_clean['film_title']=="les aventures du capitaine bobette : le film", 'film_title'] = "captain underpants: the first epic movie"
data_clean.loc[data_clean['film_title']=="les aventuriers de l'arche perdue", 'film_title'] = "indiana jones and the raiders of the lost ark"
data_clean.loc[data_clean['film_title']=="les aventuriers du timbre perdu", 'film_title'] = "tommy tricker and the stamp traveller"
data_clean.loc[data_clean['film_title']=="les bagnoles", 'film_title'] = "cars"
data_clean.loc[data_clean['film_title']=="les bagnoles 2", 'film_title'] = "cars 2"
data_clean.loc[data_clean['film_title']=="les bagnoles 3", 'film_title'] = "cars 3"
data_clean.loc[data_clean['film_title']=="les benchwarmers ça chauffe sur le banc", 'film_title'] = "the benchwarmers"
data_clean.loc[data_clean['film_title']=="les bêtes du sud sauvage", 'film_title'] = "beasts of the southern wild"
data_clean.loc[data_clean['film_title']=="les chariots de feu", 'film_title'] = "chariots of fire"
data_clean.loc[data_clean['film_title']=="les chroniques de narnia: l'armoire magique", 'film_title'] = "the chronicles of narnia: the lion, the witch and the wardrobe"
data_clean.loc[data_clean['film_title']=="les chroniques de narnia: le prince caspian", 'film_title'] = "the chronicles of narnia: prince caspian"
data_clean.loc[data_clean['film_title']=="les chroniques de noël", 'film_title'] = "the christmas chronicles"
data_clean.loc[data_clean['film_title']=="les chroniques de riddick", 'film_title'] = "the chronicles of riddick"
data_clean.loc[data_clean['film_title']=="les chroniques de spiderwick", 'film_title'] = "the spiderwick chronicles"
data_clean.loc[data_clean['film_title']=="les collégiennes de beverly hills", 'film_title'] = "clueless"
data_clean.loc[data_clean['film_title']=="les dents de la mer", 'film_title'] = "jaws"
data_clean.loc[data_clean['film_title']=="les dernières vacances", 'film_title'] = "last holiday"
data_clean.loc[data_clean['film_title']=="les désaxées", 'film_title'] = "the misfits"
data_clean.loc[data_clean['film_title']=="les deux font la paire", 'film_title'] = "they came together"
data_clean.loc[data_clean['film_title']=="les enfants du silence", 'film_title'] = "children of a lesser god"
data_clean.loc[data_clean['film_title']=="les enquêtes du département v : l'effet papillon", 'film_title'] = "the butterfly effect"
data_clean.loc[data_clean['film_title']=="les évadés", 'film_title'] = "the shawshank redemption"
data_clean.loc[data_clean['film_title']=="les femmes de stepford", 'film_title'] = "the stepford wives"
data_clean.loc[data_clean['film_title']=="les femmes du 20e siècle", 'film_title'] = "20th century women"
data_clean.loc[data_clean['film_title']=="les figures de l'ombre", 'film_title'] = "hidden figures"
data_clean.loc[data_clean['film_title']=="les fils de l'homme", 'film_title'] = "children of men"
data_clean.loc[data_clean['film_title']=="les frères blues", 'film_title'] = "the blues brothers"
data_clean.loc[data_clean['film_title']=="les frères blues 2000", 'film_title'] = "blues brothers 2000"
data_clean.loc[data_clean['film_title']=="les frères grimm", 'film_title'] = "the brothers grimm"
data_clean.loc[data_clean['film_title']=="les gangs de new york", 'film_title'] = "gangs of new york"
data_clean.loc[data_clean['film_title']=="les gardiens de la galaxie", 'film_title'] = "guardians of the galaxy"
data_clean.loc[data_clean['film_title']=="les gardiens de la galaxie vol. 2", 'film_title'] = "guardians of the galaxy vol. 2"
data_clean.loc[data_clean['film_title']=="les gentlemen", 'film_title'] = "the gentlemen"
data_clean.loc[data_clean['film_title']=="les goonies", 'film_title'] = "the goonies"
data_clean.loc[data_clean['film_title']=="les gremlins", 'film_title'] = "gremlins"
data_clean.loc[data_clean['film_title']=="les griffes de la nuit", 'film_title'] = "a nightmare on elm street"
data_clean.loc[data_clean['film_title']=="les griffes de la nuit 2 : la revanche de freddy", 'film_title'] = "a nightmare on elm street part 2: freddy's revenge"
data_clean.loc[data_clean['film_title']=="les heures", 'film_title'] = "the hours"
data_clean.loc[data_clean['film_title']=="les horizons perdus", 'film_title'] = "lost horizon"
data_clean.loc[data_clean['film_title']=="les inadaptés", 'film_title'] = "the outsiders"
data_clean.loc[data_clean['film_title']=="les inconnus : proies nocturnes", 'film_title'] = "the strangers: prey at night"
data_clean.loc[data_clean['film_title']=="les incroyables", 'film_title'] = "the incredibles"
data_clean.loc[data_clean['film_title']=="les insoumis", 'film_title'] = "the darkest minds"
data_clean.loc[data_clean['film_title']=="les invasions barbares", 'film_title'] = "the hours"
data_clean.loc[data_clean['film_title']=="les liaisons dangereuses", 'film_title'] = "dangerous liaisons"
data_clean.loc[data_clean['film_title']=="les mauvais garçons", 'film_title'] = "bad boys"
data_clean.loc[data_clean['film_title']=="les méchants", 'film_title'] = "the bad boys"
data_clean.loc[data_clean['film_title']=="les mères indignes se tapent noël", 'film_title'] = "a bad moms christmas"
data_clean.loc[data_clean['film_title']=="les merveilleuses aventures de winnie l'ourson", 'film_title'] = "the many adventures of winnie the pooh"
data_clean.loc[data_clean['film_title']=="les minions", 'film_title'] = "minions"
data_clean.loc[data_clean['film_title']=="les minions 2 : il était une fois gru", 'film_title'] = "minions: the rise of gru"
data_clean.loc[data_clean['film_title']=="les mitchell contre les machines", 'film_title'] = "the mitchells vs the machines"
data_clean.loc[data_clean['film_title']=="les mondes de ralph", 'film_title'] = "wreck-it ralph"
data_clean.loc[data_clean['film_title']=="les morts ne meurent pas", 'film_title'] = "the dead don't die"
data_clean.loc[data_clean['film_title']=="les neuf vies de m. boule-de-poil", 'film_title'] = "nine lives"
data_clean.loc[data_clean['film_title']=="les oiseaux", 'film_title'] = "the birds"
data_clean.loc[data_clean['film_title']=="les oiseaux se cachent pour mourir 1", 'film_title'] = "the thorn birds"
data_clean.loc[data_clean['film_title']=="les oiseaux se cachent pour mourir 3", 'film_title'] = "the thorn birds"
data_clean.loc[data_clean['film_title']=="les oiseaux se cachent pour mourir 5", 'film_title'] = "the thorn birds"
data_clean.loc[data_clean['film_title']=="les pages de notre amour", 'film_title'] = "the notebook"
data_clean.loc[data_clean['film_title']=="les patrouilleurs de l'espace", 'film_title'] = "starship troopers"
data_clean.loc[data_clean['film_title']=="les petits garnements", 'film_title'] = "the little rascals"
data_clean.loc[data_clean['film_title']=="les pierrafeu", 'film_title'] = "the flintstones"
data_clean.loc[data_clean['film_title']=="les princes de la gâchette", 'film_title'] = "young guns"
data_clean.loc[data_clean['film_title']=="les quatre filles du docteur march", 'film_title'] = "little women"
data_clean.loc[data_clean['film_title']=="les renforts", 'film_title'] = "the other guys"
data_clean.loc[data_clean['film_title']=="les sacrifiés", 'film_title'] = "the expendables"
data_clean.loc[data_clean['film_title']=="les sacrifiés 3", 'film_title'] = "the expendables 3"
data_clean.loc[data_clean['film_title']=="les sept mercenaires", 'film_title'] = "the magnificent seven"
data_clean.loc[data_clean['film_title']=="les simpson: le film", 'film_title'] = "the simpsons movie"
data_clean.loc[data_clean['film_title']=="les survivants", 'film_title'] = "alive"
data_clean.loc[data_clean['film_title']=="les temps modernes", 'film_title'] = "modern times"
data_clean.loc[data_clean['film_title']=="les tortues ninja", 'film_title'] = "teenage mutant ninja turtles"
data_clean.loc[data_clean['film_title']=="les vacances", 'film_title'] = "the holiday"
data_clean.loc[data_clean['film_title']=="les yeux grand fermés", 'film_title'] = "eyes wide shut"
data_clean.loc[data_clean['film_title']=="lettres d'iwo jima", 'film_title'] = "letters from iwo jima"
data_clean.loc[data_clean['film_title']=="lewis black: black to the future", 'film_title'] = "back to the future"
data_clean.loc[data_clean['film_title']=="libellule", 'film_title'] = "dragonfly"
data_clean.loc[data_clean['film_title']=="life gone with the wind", 'film_title'] = "gone with the wind"
data_clean.loc[data_clean['film_title']=="life of pie", 'film_title'] = "life of pi"
data_clean.loc[data_clean['film_title']=="ligue majeure", 'film_title'] = "major league"
data_clean.loc[data_clean['film_title']=="ligue majeure ii", 'film_title'] = "major league ii"
data_clean.loc[data_clean['film_title']=="lol", 'film_title'] = "lol (laughing out loud)"
data_clean.loc[data_clean['film_title']=="looney tunes: les revoilà !", 'film_title'] = "looney tunes: back in action"
data_clean.loc[data_clean['film_title']=="louis de funès et les gendarmes", 'film_title'] = "le gendarme de saint-tropez"
data_clean.loc[data_clean['film_title']=="luke la main froide", 'film_title'] = "cool hand luke"
data_clean.loc[data_clean['film_title']=="m. et mme smith", 'film_title'] = "mr. & mrs. smith"
data_clean.loc[data_clean['film_title']=="ma vie pour la tienne", 'film_title'] = "my sister's keeper"
data_clean.loc[data_clean['film_title']=="mad max : au-delà du dôme du tonnerre", 'film_title'] = "mad max beyond thunderdome"
data_clean.loc[data_clean['film_title']=="mad max : la route du chaos", 'film_title'] = "mad max: fury road"
data_clean.loc[data_clean['film_title']=="madagascar 3 : bons baisers d'europe", 'film_title'] = "madagascar 3: europe's most wanted"
data_clean.loc[data_clean['film_title']=="madame doubtfire", 'film_title'] = "mrs. doubtfire"
data_clean.loc[data_clean['film_title']=="maison hantée 2", 'film_title'] = "a haunted house 2"
data_clean.loc[data_clean['film_title']=="maître à bord: de l'autre côté du monde", 'film_title'] = "master and commander: the far side of the world"
data_clean.loc[data_clean['film_title']=="making 'snatch'", 'film_title'] = "snatch"
data_clean.loc[data_clean['film_title']=="malefique", 'film_title'] = "maleficent"
data_clean.loc[data_clean['film_title']=="maman, j'ai encore raté l'avion !", 'film_title'] = "home alone 2: lost in new york"
data_clean.loc[data_clean['film_title']=="maman, j'ai raté l'avion", 'film_title'] = "home alone"
data_clean.loc[data_clean['film_title']=="maman, j'ai raté l'avion !", 'film_title'] = "home alone"
data_clean.loc[data_clean['film_title']=="maman, je m'occupe des méchants!", 'film_title'] = "home alone 3"
data_clean.loc[data_clean['film_title']=="mamma mia! c'est reparti", 'film_title'] = "mamma mia! here we go again"
data_clean.loc[data_clean['film_title']=="mamma mia! le film", 'film_title'] = "mamma mia!"
data_clean.loc[data_clean['film_title']=="mammamia", 'film_title'] = "mamma mia"
data_clean.loc[data_clean['film_title']=="mange prie aime", 'film_title'] = "eat pray love"
data_clean.loc[data_clean['film_title']=="marche ou crève: vengeance définitive", 'film_title'] = "die hard with a vengeance"
data_clean.loc[data_clean['film_title']=="marge de manœuvre", 'film_title'] = "margin call"
data_clean.loc[data_clean['film_title']=="mariage, orgueil et préjugés", 'film_title'] = "marrying mr. darcy"
data_clean.loc[data_clean['film_title']=="marie a un je-ne-sais-quoi", 'film_title'] = "there's something about mary"
data_clean.loc[data_clean['film_title']=="marley et moi", 'film_title'] = "marley & me"
data_clean.loc[data_clean['film_title']=="mars attaque!", 'film_title'] = "mars attacks"
data_clean.loc[data_clean['film_title']=="martian", 'film_title'] = "the martian"
data_clean.loc[data_clean['film_title']=="marvel", 'film_title'] = "captain marvel"
data_clean.loc[data_clean['film_title']=="marvel studios: assembling a universe", 'film_title'] = "the martian"
data_clean.loc[data_clean['film_title']=="mary et max", 'film_title'] = "mary and max"
data_clean.loc[data_clean['film_title']=="massacre à la scie", 'film_title'] = "texas chainsaw massacre"
data_clean.loc[data_clean['film_title']=="massacre à la tronçonneuse", 'film_title'] = "the texas chain saw massacre"
data_clean.loc[data_clean['film_title']=="massacre à la tronçonneuse : le commencement", 'film_title'] = "the texas chainsaw massacre: the beginning"
data_clean.loc[data_clean['film_title']=="matrix", 'film_title'] = "the matrix"
data_clean.loc[data_clean['film_title']=="mauvais garçons", 'film_title'] = "bad boys"
data_clean.loc[data_clean['film_title']=="mauvais garçons ii", 'film_title'] = "bad boys ii"
data_clean.loc[data_clean['film_title']=="mauvais garçons pour la vie", 'film_title'] = "bad boys for life"
data_clean.loc[data_clean['film_title']=="max la menace", 'film_title'] = "get smart"
data_clean.loc[data_clean['film_title']=="méchant menteur", 'film_title'] = "just go with it"
data_clean.loc[data_clean['film_title']=="méchantes ados", 'film_title'] = "mean girls"
data_clean.loc[data_clean['film_title']=="méchantes ados 2", 'film_title'] = "mean girls 2"
data_clean.loc[data_clean['film_title']=="mel gibson's 'braveheart': a filmmaker's passion", 'film_title'] = "braveheart"
data_clean.loc[data_clean['film_title']=="merci de fumer", 'film_title'] = "thank you for smoking"
data_clean.loc[data_clean['film_title']=="mère !", 'film_title'] = "mother!"
data_clean.loc[data_clean['film_title']=="merveilleux", 'film_title'] = "wonder"
data_clean.loc[data_clean['film_title']=="mes vies de chien", 'film_title'] = "a dog's purpose"
data_clean.loc[data_clean['film_title']=="meurs un autre jour", 'film_title'] = "die another day"
data_clean.loc[data_clean['film_title']=="meurtre et mystère", 'film_title'] = "murder mystery"
data_clean.loc[data_clean['film_title']=="midsommar : solstice d'été", 'film_title'] = "midsommar"
data_clean.loc[data_clean['film_title']=="mille et une façons de mourir dans l'ouest", 'film_title'] = "a million ways to die in the west"
data_clean.loc[data_clean['film_title']=="mille mots", 'film_title'] = "a thousand words"
data_clean.loc[data_clean['film_title']=="millénium, le film", 'film_title'] = "the girl with the dragon tattoo"
data_clean.loc[data_clean['film_title']=="millenium", 'film_title'] = "the girl with the dragon tattoo (2011)"
data_clean.loc[data_clean['film_title']=="millennium", 'film_title'] = "the girl with the dragon tattoo (2011)"
data_clean.loc[data_clean['film_title']=="miracle de la 34e rue", 'film_title'] = "miracle on 34th street"
data_clean.loc[data_clean['film_title']=="miss personnalité", 'film_title'] = "miss congeniality"
data_clean.loc[data_clean['film_title']=="mission : impossible", 'film_title'] = "mission: impossible"
data_clean.loc[data_clean['film_title']=="mission : impossible - répercussions", 'film_title'] = "mission: impossible - fallout"
data_clean.loc[data_clean['film_title']=="mission : impossible 3", 'film_title'] = "mission: impossible iii"
data_clean.loc[data_clean['film_title']=="mission : impossible ii", 'film_title'] = "mission: impossible ii"
data_clean.loc[data_clean['film_title']=="mission sur mars", 'film_title'] = "mission to mars"
data_clean.loc[data_clean['film_title']=="mission: impossible 4 - protocole fantôme", 'film_title'] = "mission: impossible - ghost protocol"
data_clean.loc[data_clean['film_title']=="mission: impossible 5 - la nation rogue", 'film_title'] = "mission: impossible - rogue nation"
data_clean.loc[data_clean['film_title']=="mohana", 'film_title'] = "moana"
data_clean.loc[data_clean['film_title']=="moi, belle et jolie", 'film_title'] = "i feel pretty"
data_clean.loc[data_clean['film_title']=="moi, tonya", 'film_title'] = "i, tonya"
data_clean.loc[data_clean['film_title']=="moins cher la douzaine", 'film_title'] = "cheaper by the dozen"
data_clean.loc[data_clean['film_title']=="moins-que-rien", 'film_title'] = "nobody"
data_clean.loc[data_clean['film_title']=="mon ami willy", 'film_title'] = "free willy"
data_clean.loc[data_clean['film_title']=="mon amour à jamais", 'film_title'] = "forever my girl"
data_clean.loc[data_clean['film_title']=="mon fantôme d'amour", 'film_title'] = "ghost"
data_clean.loc[data_clean['film_title']=="mon frère l'ours", 'film_title'] = "brother bear"
data_clean.loc[data_clean['film_title']=="mon nom est personne", 'film_title'] = "my name is nobody"
data_clean.loc[data_clean['film_title']=="mon père", 'film_title'] = "about my father"
data_clean.loc[data_clean['film_title']=="monde infernal", 'film_title'] = "underworld"
data_clean.loc[data_clean['film_title']=="monde jurassique", 'film_title'] = "jurassic world"
data_clean.loc[data_clean['film_title']=="monde jurassique : le royaume déchu", 'film_title'] = "jurassic world: fallen kingdom"
data_clean.loc[data_clean['film_title']=="monde jurassique la domination", 'film_title'] = "jurassic world: dominion"
data_clean.loc[data_clean['film_title']=="money ball", 'film_title'] = "moneyball"
data_clean.loc[data_clean['film_title']=="moneyball : l'art de gagner", 'film_title'] = "moneyball"
data_clean.loc[data_clean['film_title']=="moneyball: playing the game", 'film_title'] = "moneyball"
data_clean.loc[data_clean['film_title']=="monstres, inc.", 'film_title'] = "monsters, inc."
data_clean.loc[data_clean['film_title']=="moonlight : l'histoire d'une vie", 'film_title'] = "moonlight"
data_clean.loc[data_clean['film_title']=="moulin rouge", 'film_title'] = "moulin rouge!"
data_clean.loc[data_clean['film_title']=="mourir peut attendre", 'film_title'] = "no time to die"
data_clean.loc[data_clean['film_title']=="mulholland dr.", 'film_title'] = "mulholland drive"
data_clean.loc[data_clean['film_title']=="my $on hunter", 'film_title'] = "my son hunter"
data_clean.loc[data_clean['film_title']=="naruto the last: le film", 'film_title'] = "the last: naruto the movie"
data_clean.loc[data_clean['film_title']=="ne t'inquiète pas chérie", 'film_title'] = "don't worry darling"
data_clean.loc[data_clean['film_title']=="née pour danser", 'film_title'] = "save the last dance"
data_clean.loc[data_clean['film_title']=="never just a dream: stanley kubrick and eyes wide shut", 'film_title'] = "eyes wide shut"
data_clean.loc[data_clean['film_title']=="no country for old men: josh brolin's unauthorized behind the scenes", 'film_title'] = "no country for old men"
data_clean.loc[data_clean['film_title']=="noël chez les muppets", 'film_title'] = "the muppet christmas carol"
data_clean.loc[data_clean['film_title']=="noël ou presque", 'film_title'] = "almost christmas"
data_clean.loc[data_clean['film_title']=="non, ce pays n'est pas pour le vieil homme", 'film_title'] = "no country for old man"
data_clean.loc[data_clean['film_title']=="normand du nord", 'film_title'] = "norm of the north"
data_clean.loc[data_clean['film_title']=="nos étoiles contraires", 'film_title'] = "the fault in our stars"
data_clean.loc[data_clean['film_title']=="nos plus belles années", 'film_title'] = "the way we were"
data_clean.loc[data_clean['film_title']=="notice rouge", 'film_title'] = "red notice"
data_clean.loc[data_clean['film_title']=="notre histoire", 'film_title'] = "the story of us"
data_clean.loc[data_clean['film_title']=="notre planète : les coulisses", 'film_title'] = "our planet"
data_clean.loc[data_clean['film_title']=="nous", 'film_title'] = "us"
data_clean.loc[data_clean['film_title']=="nous étions guerriers", 'film_title'] = "once were warriors"
data_clean.loc[data_clean['film_title']=="nous sommes les miller", 'film_title'] = "we're the millers"
data_clean.loc[data_clean['film_title']=="nouveau refrain", 'film_title'] = "begin again"
data_clean.loc[data_clean['film_title']=="nuit et jour", 'film_title'] = "knight and day"
data_clean.loc[data_clean['film_title']=="nwa lords of the ring", 'film_title'] = "the lord of the rings: the fellowship of the ring"
data_clean.loc[data_clean['film_title']=="nymphomaniac: directors cut", 'film_title'] = "nymphomaniac: vol. i"
data_clean.loc[data_clean['film_title']=="ocean's 11", 'film_title'] = "ocean's eleven"
data_clean.loc[data_clean['film_title']=="oh, brother", 'film_title'] = "o brother, where art thou?"
data_clean.loc[data_clean['film_title']=="oliver & compagnie", 'film_title'] = "oliver & company"
data_clean.loc[data_clean['film_title']=="on ne rigole pas avec le zohan", 'film_title'] = "you don't mess with the zohan"
data_clean.loc[data_clean['film_title']=="on ne vit que deux fois", 'film_title'] = "you only live twice"
data_clean.loc[data_clean['film_title']=="one piece!", 'film_title'] = "one piece film red"
data_clean.loc[data_clean['film_title']=="opération avant l'aube", 'film_title'] = "zero dark thirty"
data_clean.loc[data_clean['film_title']=="opération beurre de pinottes", 'film_title'] = "the peanut butter solution"
data_clean.loc[data_clean['film_title']=="opération dragon", 'film_title'] = "enter the dragon"
data_clean.loc[data_clean['film_title']=="orange mécanique", 'film_title'] = "a clockwork orange"
data_clean.loc[data_clean['film_title']=="orgueil & préjugés", 'film_title'] = "pride & prejudice"
data_clean.loc[data_clean['film_title']=="origine", 'film_title'] = "inception"
data_clean.loc[data_clean['film_title']=="oublie sarah marshall", 'film_title'] = "forgetting sarah marshall"
data_clean.loc[data_clean['film_title']=="ovni l'ultime rencontre", 'film_title'] = "fire in the sky"
data_clean.loc[data_clean['film_title']=="panthère noire", 'film_title'] = "black panther"
data_clean.loc[data_clean['film_title']=="parasit", 'film_title'] = "parasite"
data_clean.loc[data_clean['film_title']=="parfum de femme", 'film_title'] = "scent of a woman"
data_clean.loc[data_clean['film_title']=="partis en 60 secondes", 'film_title'] = "gone in sixty seconds"
data_clean.loc[data_clean['film_title']=="passagers", 'film_title'] = "passengers"
data_clean.loc[data_clean['film_title']=="pat'patrouille : secours supersoniques", 'film_title'] = "paw patrol: the movie"
data_clean.loc[data_clean['film_title']=="payez au suivant", 'film_title'] = "pay it forward"
data_clean.loc[data_clean['film_title']=="percy jackson : la mer des monstres", 'film_title'] = "percy jackson: sea of monsters"
data_clean.loc[data_clean['film_title']=="percy jackson : le voleur de foudre", 'film_title'] = "percy jackson & the olympians: the lightning thief"
data_clean.loc[data_clean['film_title']=="père de famille", 'film_title'] = "the family man"
data_clean.loc[data_clean['film_title']=="permis de tuer", 'film_title'] = "licence to kill"
data_clean.loc[data_clean['film_title']=="petit bonhomme", 'film_title'] = "big"
data_clean.loc[data_clean['film_title']=="petit-pied, le dinosaure", 'film_title'] = "a tale before time"
data_clean.loc[data_clean['film_title']=="petit-pied, le dinosaure 2 : l'aventure de la grande vallée", 'film_title'] = "the land before time ii: the great valley adventure"
data_clean.loc[data_clean['film_title']=="peur et dégoût à las vegas", 'film_title'] = "fear and loathing in las vegas"
data_clean.loc[data_clean['film_title']=="philadelphie", 'film_title'] = "philadelphia"
data_clean.loc[data_clean['film_title']=="piège de cristal", 'film_title'] = "die hard"
data_clean.loc[data_clean['film_title']=="pierre lapin", 'film_title'] = "peter rabbit"
data_clean.loc[data_clean['film_title']=="pirates des caraïbes : jusqu'au bout du monde", 'film_title'] = "pirates of the caribbean: at world's end"
data_clean.loc[data_clean['film_title']=="pirates des caraïbes : la fontaine de jouvence", 'film_title'] = "pirates of the caribbean: on stranger tides"
data_clean.loc[data_clean['film_title']=="pirates des caraïbes : la malédiction de la perle noire", 'film_title'] = "pirates of the caribbean: the curse of the black pearl"
data_clean.loc[data_clean['film_title']=="pirates des caraïbes : le coffre du mort", 'film_title'] = "pirates of the caribbean: dead man's chest"
data_clean.loc[data_clean['film_title']=="pirates des caraïbes : les morts ne racontent pas d'histoires", 'film_title'] = "pirates of the caribbean: dead men tell no tales"
data_clean.loc[data_clean['film_title']=="planète terreur", 'film_title'] = "planet terror"
data_clean.loc[data_clean['film_title']=="player one", 'film_title'] = "ready player one"
data_clean.loc[data_clean['film_title']=="pokémon : détective pikachu", 'film_title'] = "pokémon: detective pikachu"
data_clean.loc[data_clean['film_title']=="pokémon 2 : le pouvoir est en toi", 'film_title'] = "pokémon: the power of us"
data_clean.loc[data_clean['film_title']=="pokémon, le film - mewtwo contre mew", 'film_title'] = "ready player one"
data_clean.loc[data_clean['film_title']=="police académie", 'film_title'] = "police academy"
data_clean.loc[data_clean['film_title']=="poltergeist : la vengeance des fantômes", 'film_title'] = "poltergeist"
data_clean.loc[data_clean['film_title']=="pompiers en alerte", 'film_title'] = "backdraft"
data_clean.loc[data_clean['film_title']=="poulets en fuite", 'film_title'] = "chicken run"
data_clean.loc[data_clean['film_title']=="pour l'amour d'hollywood", 'film_title'] = "la la land"
data_clean.loc[data_clean['film_title']=="pour un instant de bonheur", 'film_title'] = "a little bit of heaven"
data_clean.loc[data_clean['film_title']=="poulets en fuite", 'film_title'] = "chicken run"
data_clean.loc[data_clean['film_title']=="pray.", 'film_title'] = "prey"
data_clean.loc[data_clean['film_title']=="premières de classe", 'film_title'] = "booksmart"
data_clean.loc[data_clean['film_title']=="présentateur vedette 2: la légende se poursuit", 'film_title'] = "anchorman 2: the legend continues"
data_clean.loc[data_clean['film_title']=="présentateur vedette: la légende de ron burgundy", 'film_title'] = "anchorman: the legend of ron burgundy"
data_clean.loc[data_clean['film_title']=="pretty woman and other stories, a film with alexander kluge", 'film_title'] = "pretty woman"
data_clean.loc[data_clean['film_title']=="prisonniers", 'film_title'] = "prisoners"
data_clean.loc[data_clean['film_title']=="prisonnier", 'film_title'] = "prisoners"
data_clean.loc[data_clean['film_title']=="psychose", 'film_title'] = "psycho"
data_clean.loc[data_clean['film_title']=="pulp fiction art", 'film_title'] = "pulp fiction"
data_clean.loc[data_clean['film_title']=="pulp fiction: the golden age of storytelling", 'film_title'] = "pulp fiction"
data_clean.loc[data_clean['film_title']=="pulpe fiction", 'film_title'] = "pulp fiction"
data_clean.loc[data_clean['film_title']=="pulpfiction", 'film_title'] = "pulp fiction"
data_clean.loc[data_clean['film_title']=="plump fiction", 'film_title'] = "pulp fiction"
data_clean.loc[data_clean['film_title']=="pursuit of happyness", 'film_title'] = "the pursuit of happyness"
data_clean.loc[data_clean['film_title']=="quand harry rencontre sally...", 'film_title'] = "when harry met sally..."
data_clean.loc[data_clean['film_title']=="quand les femmes s'en mêlent", 'film_title'] = "working girl"
data_clean.loc[data_clean['film_title']=="quelque chose d'inattendu", 'film_title'] = "something's gotta give "
data_clean.loc[data_clean['film_title']=="quelque part dans le temps", 'film_title'] = "somewhere in time"
data_clean.loc[data_clean['film_title']=="quelques minutes après minuit", 'film_title'] = "a monster calls"
data_clean.loc[data_clean['film_title']=="qui veut la peau de roger rabbit", 'film_title'] = "who framed roger rabbit"
data_clean.loc[data_clean['film_title']=="rage meurtrière", 'film_title'] = "the grudge"
data_clean.loc[data_clean['film_title']=="raiders of the lost ark: the adaptation", 'film_title'] = "indiana jones and the raiders of the lost ark"
data_clean.loc[data_clean['film_title']=="raiders of the lost shark", 'film_title'] = "indiana jones and the raiders of the lost ark"
data_clean.loc[data_clean['film_title']=="raiponce", 'film_title'] = "tangled"
data_clean.loc[data_clean['film_title']=="ralph brise l'internet", 'film_title'] = "ralph breaks the internet"
data_clean.loc[data_clean['film_title']=="rambo : la dernière mission", 'film_title'] = "rambo: last blood"
data_clean.loc[data_clean['film_title']=="rambo : le dévastateur", 'film_title'] = "first blood"
data_clean.loc[data_clean['film_title']=="rambo ii : la mission", 'film_title'] = "rambo: first blood part ii"
data_clean.loc[data_clean['film_title']=="rambo: the rescue", 'film_title'] = "first blood"
data_clean.loc[data_clean['film_title']=="rapides et dangereux", 'film_title'] = "the fast and the furious"
data_clean.loc[data_clean['film_title']=="rapides et dangereux : tokyo drift", 'film_title'] = "the fast and the furious: tokyo drift"
data_clean.loc[data_clean['film_title']=="rapides et dangereux 2", 'film_title'] = "2 fast 2 furious"
data_clean.loc[data_clean['film_title']=="rapides et dangereux 4", 'film_title'] = "fast & furious"
data_clean.loc[data_clean['film_title']=="rapides et dangereux 5", 'film_title'] = "fast five"
data_clean.loc[data_clean['film_title']=="rapides et dangereux 6", 'film_title'] = "fast & furious 6"
data_clean.loc[data_clean['film_title']=="rapides et dangereux 9 la saga", 'film_title'] = "f9: the fast saga"
data_clean.loc[data_clean['film_title']=="rapides et dangereux présentent: hobbs et shaw", 'film_title'] = "fast & furious presents: hobbs & shaw"
data_clean.loc[data_clean['film_title']=="rapport minoritaire", 'film_title'] = "minority report"
data_clean.loc[data_clean['film_title']=="ratatouille: the tiktok musical", 'film_title'] = "ratatouille"
data_clean.loc[data_clean['film_title']=="rebelle", 'film_title'] = "brave"
data_clean.loc[data_clean['film_title']=="réellement l'amour", 'film_title'] = "love actually"
data_clean.loc[data_clean['film_title']=="rencontre avec joe black", 'film_title'] = "meet joe black"
data_clean.loc[data_clean['film_title']=="rencontres du troisième type", 'film_title'] = "close encounters of the third kind"
data_clean.loc[data_clean['film_title']=="requiem for a scream", 'film_title'] = "requiem for a dream"
data_clean.loc[data_clean['film_title']=="resident evil : bienvenue à raccoon city", 'film_title'] = "resident evil: welcome to raccoon city"
data_clean.loc[data_clean['film_title']=="retour à brooklyn", 'film_title'] = "requiem for a dream"
data_clean.loc[data_clean['film_title']=="retour au bercail", 'film_title'] = "homeward bound: the incredible journey"
data_clean.loc[data_clean['film_title']=="retour au bercail: l'incroyable randonnée", 'film_title'] = "homeward bound: the incredible journey"
data_clean.loc[data_clean['film_title']=="retour vers le futur", 'film_title'] = "back to the future"
data_clean.loc[data_clean['film_title']=="retour vers le futur : 30 ans d'un film culte", 'film_title'] = "back to the future"
data_clean.loc[data_clean['film_title']=="retour vers le futur ii", 'film_title'] = "back to the future part ii"
data_clean.loc[data_clean['film_title']=="retour vers le futur iii", 'film_title'] = "back to the future part iii"
data_clean.loc[data_clean['film_title']=="rêver grand", 'film_title'] = "licorice pizza"
data_clean.loc[data_clean['film_title']=="richard et le secret des livres magiques", 'film_title'] = "the pagemaster"
data_clean.loc[data_clean['film_title']=="rives du pacifique", 'film_title'] = "pacific rim"
data_clean.loc[data_clean['film_title']=="robin des bois", 'film_title'] = "robin hood"
data_clean.loc[data_clean['film_title']=="robin des bois: héros en collants", 'film_title'] = "robin hood: men in tights"
data_clean.loc[data_clean['film_title']=="robin des bois: prince des voleurs", 'film_title'] = "robin hood: prince of thieves"
data_clean.loc[data_clean['film_title']=="robocop: prime directives", 'film_title'] = "robocop"
data_clean.loc[data_clean['film_title']=="rock'n nonne 2 : de retour au couvent", 'film_title'] = "sister act 2: back in the habit"
data_clean.loc[data_clean['film_title']=="rock'n nonne", 'film_title'] = "sister act"
data_clean.loc[data_clean['film_title']=="rocky iii: l'oeil du tigre", 'film_title'] = "rocky iii"
data_clean.loc[data_clean['film_title']=="rocky iv: rocky vs. drago the ultimate directors cut released", 'film_title'] = "rocky iv"
data_clean.loc[data_clean['film_title']=="rogue one : une histoire de star wars", 'film_title'] = "rogue one: a star wars story"
data_clean.loc[data_clean['film_title']=="roméo doit mourir", 'film_title'] = "romeo must die"
data_clean.loc[data_clean['film_title']=="roméo et juliette", 'film_title'] = "romeo + juliet"
data_clean.loc[data_clean['film_title']=="room: le monde de jack", 'film_title'] = "room"
data_clean.loc[data_clean['film_title']=="rox et rouky", 'film_title'] = "the fox and the hound"
data_clean.loc[data_clean['film_title']=="s.o.s. fantômes", 'film_title'] = "ghostbusters"
data_clean.loc[data_clean['film_title']=="sacrifice", 'film_title'] = "offret"
data_clean.loc[data_clean['film_title']=="sang-froid", 'film_title'] = "drive"
data_clean.loc[data_clean['film_title']=="sans arrêt", 'film_title'] = "non-stop"
data_clean.loc[data_clean['film_title']=="sans lendemain", 'film_title'] = "edge of tomorrow"
data_clean.loc[data_clean['film_title']=="sans limites", 'film_title'] = "limitless"
data_clean.loc[data_clean['film_title']=="schindler's list: 25 years later", 'film_title'] = "schindler's list"
data_clean.loc[data_clean['film_title']=="scooby-doo 2: monstres en liberté", 'film_title'] = "scooby-doo 2: monsters unleashed"
data_clean.loc[data_clean['film_title']=="scorsese's goodfellas", 'film_title'] = "goodfellas"
data_clean.loc[data_clean['film_title']=="scott pilgrim vs le monde", 'film_title'] = "scott pilgrim vs. the world"
data_clean.loc[data_clean['film_title']=="seigneur de guerre", 'film_title'] = "lord of war"
data_clean.loc[data_clean['film_title']=="sens dessus dessous", 'film_title'] = "inside out"
data_clean.loc[data_clean['film_title']=="sept", 'film_title'] = "se7en"
data_clean.loc[data_clean['film_title']=="sept vies", 'film_title'] = "seven pounds"
data_clean.loc[data_clean['film_title']=="serenity!", 'film_title'] = "serenity"
data_clean.loc[data_clean['film_title']=="seul au monde", 'film_title'] = "cast away"
data_clean.loc[data_clean['film_title']=="seul sur mars", 'film_title'] = "the martian"
data_clean.loc[data_clean['film_title']=="seuls les braves", 'film_title'] = "only the braves"
data_clean.loc[data_clean['film_title']=="seven", 'film_title'] = "se7en"
data_clean.loc[data_clean['film_title']=="seven samurai: origins and influences", 'film_title'] = "seven samourai"
data_clean.loc[data_clean['film_title']=="sexe à new york", 'film_title'] = "sex and the city"
data_clean.loc[data_clean['film_title']=="sexe à new york 2", 'film_title'] = "sex and the city 2"
data_clean.loc[data_clean['film_title']=="shang-chi et la légende des dix anneaux", 'film_title'] = "shang-chi and the legend of the ten rings"
data_clean.loc[data_clean['film_title']=="shaun et les zombies", 'film_title'] = "shaun of the dead"
data_clean.loc[data_clean['film_title']=="shérif, fais-moi peur", 'film_title'] = "the dukes of hazzard"
data_clean.loc[data_clean['film_title']=="sherlock holmes: le jeu des ombres", 'film_title'] = "sherlock holmes: a game of shadows"
data_clean.loc[data_clean['film_title']=="short cuts : la leçon de piano", 'film_title'] = "the piano"
data_clean.loc[data_clean['film_title']=="short cuts : cris et chuchotements", 'film_title'] = "cries & whispers"
data_clean.loc[data_clean['film_title']=="shreck", 'film_title'] = "shrek"
data_clean.loc[data_clean['film_title']=="shrek 4,  il était une fin", 'film_title'] = "shrek forever after"
data_clean.loc[data_clean['film_title']=="shrek le troisième", 'film_title'] = "shrek the third"
data_clean.loc[data_clean['film_title']=="silence de mort", 'film_title'] = "dead silence"
data_clean.loc[data_clean['film_title']=="sinistre", 'film_title'] = "sinister"
data_clean.loc[data_clean['film_title']=="sinistre 2", 'film_title'] = "sinister"
data_clean.loc[data_clean['film_title']=="sissi, l'impératrice de la solitude", 'film_title'] = "sissi"
data_clean.loc[data_clean['film_title']=="sissi, l'impératrice rebelle", 'film_title'] = "sissi"
data_clean.loc[data_clean['film_title']=="six jours,sept nuits", 'film_title'] = "six days seven nights"
data_clean.loc[data_clean['film_title']=="sky high: école des superhéros", 'film_title'] = "sky high"
data_clean.loc[data_clean['film_title']=="slc punk", 'film_title'] = "slc punk!"
data_clean.loc[data_clean['film_title']=="snapshots of italy: the making of call me by your name", 'film_title'] = "call me by your name"
data_clean.loc[data_clean['film_title']=="soleil de minuit", 'film_title'] = "midnight sun"
data_clean.loc[data_clean['film_title']=="soleil vert", 'film_title'] = "soylent green"
data_clean.loc[data_clean['film_title']=="sonic le hérisson", 'film_title'] = "sonic the hedgehog"
data_clean.loc[data_clean['film_title']=="sonic le hérisson 2", 'film_title'] = "sonic the hedgehog 2"
data_clean.loc[data_clean['film_title']=="sortilège", 'film_title'] = "beastly"
data_clean.loc[data_clean['film_title']=="sos fantômes", 'film_title'] = "ghostbusters"
data_clean.loc[data_clean['film_title']=="sos fantômes : l'au-delà", 'film_title'] = "ghostbusters: afterlife"
data_clean.loc[data_clean['film_title']=="south park: post covid", 'film_title'] = "south park: post covid special!"
data_clean.loc[data_clean['film_title']=="south park: the 25th anniversary concert", 'film_title'] = "south park: bigger, longer & uncut"
data_clean.loc[data_clean['film_title']=="souvenirs d'afrique", 'film_title'] = "out of africa"
data_clean.loc[data_clean['film_title']=="soyons flics", 'film_title'] = "let's be cops"
data_clean.loc[data_clean['film_title']=="spaceballs: in conversation - mel brooks and thomas meehan", 'film_title'] = "spaceballs"
data_clean.loc[data_clean['film_title']=="sphère", 'film_title'] = "sphere"
data_clean.loc[data_clean['film_title']=="spider-man : dans le spider-verse", 'film_title'] = "spider-man: into the spider-verse"
data_clean.loc[data_clean['film_title']=="spider-man : les retrouvailles", 'film_title'] = "spider-man: homecoming"
data_clean.loc[data_clean['film_title']=="spider-man : loin des siens", 'film_title'] = "spider-man: far from home"
data_clean.loc[data_clean['film_title']=="spider-man : sans retour", 'film_title'] = "spider-man: no way home"
data_clean.loc[data_clean['film_title']=="spirale : l'héritage de décadence", 'film_title'] = "spiral: from the book of saw"
data_clean.loc[data_clean['film_title']=="spirit : l'indomptable", 'film_title'] = "spirit untamed"
data_clean.loc[data_clean['film_title']=="spirit, l'étalon des plaines", 'film_title'] = "spirit: stallion of the cimarron"
data_clean.loc[data_clean['film_title']=="spotlight : édition spéciale", 'film_title'] = "spotlight"
data_clean.loc[data_clean['film_title']=="staircases to nowhere: making stanley kubrick's the shining", 'film_title'] = "the shining"
data_clean.loc[data_clean['film_title']=="star trek : générations", 'film_title'] = "star trek : generations"
data_clean.loc[data_clean['film_title']=="star trek : premier contact", 'film_title'] = "star trek: first contact"
data_clean.loc[data_clean['film_title']=="star trek au-delà", 'film_title'] = "star trek beyond"
data_clean.loc[data_clean['film_title']=="star trek i: le film", 'film_title'] = "star trek: the motion picture"
data_clean.loc[data_clean['film_title']=="star trek ii : la colère de khan", 'film_title'] = "star trek ii: the wrath of khan"
data_clean.loc[data_clean['film_title']=="star trek iv : retour sur terre", 'film_title'] = "star trek iv: the voyage home"
data_clean.loc[data_clean['film_title']=="star trek vi : terre inconnue", 'film_title'] = "star trek vi: the undiscovered country"
data_clean.loc[data_clean['film_title']=="star trek: némésis", 'film_title'] = "star trek: nemesis"
data_clean.loc[data_clean['film_title']=="star wars", 'film_title'] = "star wars: episode iv - a new hope"
data_clean.loc[data_clean['film_title']=="star wars, épisode iii - la revanche des sith", 'film_title'] = "star wars: episode iii - revenge of the sith"
data_clean.loc[data_clean['film_title']=="star wars, épisode ix : l'ascension de skywalker", 'film_title'] = "star wars: episode ix - the rise of skywalker"
data_clean.loc[data_clean['film_title']=="star wars, épisode vii : le réveil de la force", 'film_title'] = "star wars: episode vii - the force awakens"
data_clean.loc[data_clean['film_title']=="star wars, épisode viii : les derniers jedi", 'film_title'] = "star wars: episode viii - the last jedi"
data_clean.loc[data_clean['film_title']=="star wars: return of the jedi", 'film_title'] = "star wars: episode vi - return of the jedi"
data_clean.loc[data_clean['film_title']=="star wars: the force awakens", 'film_title'] = "star wars: episode vii - the force awakens"
data_clean.loc[data_clean['film_title']=="sueurs froides", 'film_title'] = "vertigo"
data_clean.loc[data_clean['film_title']=="super flic", 'film_title'] = "hot fuzz"
data_clean.loc[data_clean['film_title']=="supermalades", 'film_title'] = "superbad"
data_clean.loc[data_clean['film_title']=="superman - le film", 'film_title'] = "superman"
data_clean.loc[data_clean['film_title']=="sur la route de madison", 'film_title'] = "the bridges of madison county"
data_clean.loc[data_clean['film_title']=="surfeuse dans l'âme", 'film_title'] = "soul"
data_clean.loc[data_clean['film_title']=="suspects de convenance", 'film_title'] = "the usual suspects"
data_clean.loc[data_clean['film_title']=="suspense.", 'film_title'] = "suspense"
data_clean.loc[data_clean['film_title']=="swallowed souls: the making of evil dead 2", 'film_title'] = "evil dead 2"
data_clean.loc[data_clean['film_title']=="sympathie pour le diable", 'film_title'] = "sympathy for the devil"
data_clean.loc[data_clean['film_title']=="tarantino", 'film_title'] = "pulp fiction"
data_clean.loc[data_clean['film_title']=="tarantino's basterds", 'film_title'] = "inglourious basterds"
data_clean.loc[data_clean['film_title']=="tarkovsky's andrei rublev: a journey", 'film_title'] = "andrei rublev"
data_clean.loc[data_clean['film_title']=="teenage mutant ninja turtles: rise of the turtles", 'film_title'] = "rise of the teenage mutant ninja turtles"
data_clean.loc[data_clean['film_title']=="témoin sous surveillance", 'film_title'] = "witness"
data_clean.loc[data_clean['film_title']=="tension", 'film_title'] = "heat"
data_clean.loc[data_clean['film_title']=="terminator 2: le jugement dernier", 'film_title'] = "terminator 2: judgment day"
data_clean.loc[data_clean['film_title']=="terminator: rédemption", 'film_title'] = "terminator salvation"
data_clean.loc[data_clean['film_title']=="terminator", 'film_title'] = "the terminator"
data_clean.loc[data_clean['film_title']=="terre, la vie cachée d'une planète", 'film_title'] = "planet earth"
data_clean.loc[data_clean['film_title']=="the adventures of young indiana jones: hollywood follies", 'film_title'] = "indiana jones and the raiders of the lost ark"
data_clean.loc[data_clean['film_title']=="the art of 'spirited away'", 'film_title'] = "spirited away"
data_clean.loc[data_clean['film_title']=="the birth of \"princess mononoke\" part 1: a drama on paper", 'film_title'] = "mononoke hime"
data_clean.loc[data_clean['film_title']=="the black knight", 'film_title'] = "the dark knight"
data_clean.loc[data_clean['film_title']=="the fear of god: 25 years of the exorcist", 'film_title'] = "the exorcist"
data_clean.loc[data_clean['film_title']=="the girl with the dragon tattoo: on location - sweden and hollywood", 'film_title'] = "the girl with the dragon tattoo"
data_clean.loc[data_clean['film_title']=="the green book: guide to freedom", 'film_title'] = "green book"
data_clean.loc[data_clean['film_title']=="the green line", 'film_title'] = "the green mile"
data_clean.loc[data_clean['film_title']=="the hitch hikers guide to the galaxy", 'film_title'] = "the hitchhiker's guide to the galaxy"
data_clean.loc[data_clean['film_title']=="the joker", 'film_title'] = "joker"
data_clean.loc[data_clean['film_title']=="the james bond story", 'film_title'] = "dr. no"
data_clean.loc[data_clean['film_title']=="the kingdom of heaven", 'film_title'] = "kingdom of heaven"
data_clean.loc[data_clean['film_title']=="the making of requiem for a dream", 'film_title'] = "requiem for a dream"
data_clean.loc[data_clean['film_title']=="the nightmare before christmas: the original poem", 'film_title'] = "the nightmare before christmas"
data_clean.loc[data_clean['film_title']=="the real da vinci code", 'film_title'] = "the da vinci code"
data_clean.loc[data_clean['film_title']=="the real inglorious bastards", 'film_title'] = "the inglorious bastards"
data_clean.loc[data_clean['film_title']=="the return of the king", 'film_title'] = "the lord of the rings: the return of the king"
data_clean.loc[data_clean['film_title']=="the road to el camino: behind the scenes of el camino: a breaking bad movie", 'film_title'] = "el camino: a breaking bad movie"
data_clean.loc[data_clean['film_title']=="the sound of music live!", 'film_title'] = "the sound of music"
data_clean.loc[data_clean['film_title']=="the stars of star wars", 'film_title'] = "star wars: episode iv - a new hope"
data_clean.loc[data_clean['film_title']=="the the naked lunch and the naked the naked lunch", 'film_title'] = "the naked lunch"
data_clean.loc[data_clean['film_title']=="the way we were: looking back", 'film_title'] = "the way we were"
data_clean.loc[data_clean['film_title']=="the wes anderson collection: the grand budapest hotel", 'film_title'] = "the grand budapest hotel"
data_clean.loc[data_clean['film_title']=="the woods movie: the making of the blair witch project", 'film_title'] = "the blair witch project"
data_clean.loc[data_clean['film_title']=="the zodiac", 'film_title'] = "zodiac"
data_clean.loc[data_clean['film_title']=="thor : amour et tonnerre", 'film_title'] = "thor: love and thunder"
data_clean.loc[data_clean['film_title']=="thor: god of thunder", 'film_title'] = "thor"
data_clean.loc[data_clean['film_title']=="through the eyes of forrest gump", 'film_title'] = "forrest gump"
data_clean.loc[data_clean['film_title']=="tigre et dragon", 'film_title'] = "crouching tiger, hidden dragon"
data_clean.loc[data_clean['film_title']=="tireur d'élite", 'film_title'] = "shooter"
data_clean.loc[data_clean['film_title']=="tireur d'élite américain", 'film_title'] = "american sniper"
data_clean.loc[data_clean['film_title']=="titanic 2000", 'film_title'] = "titanic"
data_clean.loc[data_clean['film_title']=="titanic 666", 'film_title'] = "titanic"
data_clean.loc[data_clean['film_title']=="titanic ii", 'film_title'] = "titanic"
data_clean.loc[data_clean['film_title']=="titanic: 20 years later with james cameron", 'film_title'] = "titanic"
data_clean.loc[data_clean['film_title']=="titanic: the final word with james cameron", 'film_title'] = "titanic"
data_clean.loc[data_clean['film_title']=="tobby : le chien étoile", 'film_title'] = "air bud"
data_clean.loc[data_clean['film_title']=="tokyo new city orchestra - sword art online film orchestra concert 2021", 'film_title'] = "sword art online"
data_clean.loc[data_clean['film_title']=="tonnerre sous les tropiques", 'film_title'] = "tropic thunder"
data_clean.loc[data_clean['film_title']=="top gear: fast and furious 2", 'film_title'] = "2 fast 2 furious"
data_clean.loc[data_clean['film_title']=="top gum", 'film_title'] = "top gun"
data_clean.loc[data_clean['film_title']=="top gun : les coulisses du grand retour", 'film_title'] = "top gun"
data_clean.loc[data_clean['film_title']=="tornade", 'film_title'] = "twister"
data_clean.loc[data_clean['film_title']=="tortues ninja", 'film_title'] = "teenage mutant ninja turtles"
data_clean.loc[data_clean['film_title']=="tous les chiens vont au paradis", 'film_title'] = "all dogs go to heaven"
data_clean.loc[data_clean['film_title']=="tout pour un a", 'film_title'] = "easy girl"
data_clean.loc[data_clean['film_title']=="tout, partout, tout à la fois", 'film_title'] = "everything everywhere all at once"
data_clean.loc[data_clean['film_title']=="traduction infidèle", 'film_title'] = "lost in translation"
data_clean.loc[data_clean['film_title']=="train à grande vitesse", 'film_title'] = "bullet train"
data_clean.loc[data_clean['film_title']=="transformers : l'ère de l'extinction", 'film_title'] = "transformers: age of extinction"
data_clean.loc[data_clean['film_title']=="transformers : le dernier chevalier", 'film_title'] = "transformers: the last knight"
data_clean.loc[data_clean['film_title']=="transformers : le film", 'film_title'] = "transformers"
data_clean.loc[data_clean['film_title']=="transformers 3 : la face cachée de la lune", 'film_title'] = "transformers 3: dark of the moon"
data_clean.loc[data_clean['film_title']=="transformers: la revanche", 'film_title'] = "transformers: revenge of the fallen"
data_clean.loc[data_clean['film_title']=="trésor national", 'film_title'] = "national treasure"
data_clean.loc[data_clean['film_title']=="trésor national - le livre des secrets", 'film_title'] = "national treasure: book of secrets"
data_clean.loc[data_clean['film_title']=="trevor noah: lost in translation", 'film_title'] = "lost in translation"
data_clean.loc[data_clean['film_title']=="trois affiches tout près d'ebbing, missouri", 'film_title'] = "three billboards outside ebbing, missouri"
data_clean.loc[data_clean['film_title']=="trois femmes", 'film_title'] = "3 women"
data_clean.loc[data_clean['film_title']=="tron: l'héritage", 'film_title'] = "tron: legacy"
data_clean.loc[data_clean['film_title']=="trouver doris", 'film_title'] = "finding dory"
data_clean.loc[data_clean['film_title']=="trouver nemo", 'film_title'] = "finding nemo"
data_clean.loc[data_clean['film_title']=="tu fais pas le poids, shérif !", 'film_title'] = "smokey and the bandit ii"
data_clean.loc[data_clean['film_title']=="tu ne tueras point", 'film_title'] = "hacksaw ridge"
data_clean.loc[data_clean['film_title']=="tuer bill: volume 1", 'film_title'] = "kill bill: vol. i"
data_clean.loc[data_clean['film_title']=="tuer bill: volume 2", 'film_title'] = "kill bill: vol. ii"
data_clean.loc[data_clean['film_title']=="tunnels de guerre : la grande évasion", 'film_title'] = "the great escape"
data_clean.loc[data_clean['film_title']=="twilight : la fascination", 'film_title'] = "twilight"
data_clean.loc[data_clean['film_title']=="twin peaks : feu marche avec moi", 'film_title'] = "twin peaks"
data_clean.loc[data_clean['film_title']=="ultimate iron man: the making of iron man 2", 'film_title'] = "iron man 2"
data_clean.loc[data_clean['film_title']=="un ange dénommé wanda", 'film_title'] = "a fish called wanda"
data_clean.loc[data_clean['film_title']=="un boulot à l'italienne", 'film_title'] = "the italian job"
data_clean.loc[data_clean['film_title']=="un choix", 'film_title'] = "the choice"
data_clean.loc[data_clean['film_title']=="un ciel couleur vanille", 'film_title'] = "vanilla sky"
data_clean.loc[data_clean['film_title']=="un coin tranquille", 'film_title'] = "a quiet place"
data_clean.loc[data_clean['film_title']=="un coin tranquille : 2e partie", 'film_title'] = "a quiet place part ii"
data_clean.loc[data_clean['film_title']=="un duo d'enfer", 'film_title'] = "the heat"
data_clean.loc[data_clean['film_title']=="un empereur nouveau genre", 'film_title'] = "the emperor's new groove"
data_clean.loc[data_clean['film_title']=="un été en provence", 'film_title'] = "avis de mistral"
data_clean.loc[data_clean['film_title']=="un film d'horreur classique", 'film_title'] = "a classic horror story"
data_clean.loc[data_clean['film_title']=="un flic à la maternelle 2", 'film_title'] = "kindergarten cop"
data_clean.loc[data_clean['film_title']=="un génie, deux associés, une cloche", 'film_title'] = "un genio, due compari, un pollo"
data_clean.loc[data_clean['film_title']=="un homme au singulier", 'film_title'] = "a single man"
data_clean.loc[data_clean['film_title']=="un honnête citoyen", 'film_title'] = "a law abiding citizen"
data_clean.loc[data_clean['film_title']=="un jour sans lendemain", 'film_title'] = "edge of tomorrow"
data_clean.loc[data_clean['film_title']=="un monde à l'envers", 'film_title'] = "upside down"
data_clean.loc[data_clean['film_title']=="un officier et un gentleman", 'film_title'] = "an officer and a gentleman"
data_clean.loc[data_clean['film_title']=="un pari cruel", 'film_title'] = "cruel intentions"
data_clean.loc[data_clean['film_title']=="un prince à new york", 'film_title'] = "coming to america"
data_clean.loc[data_clean['film_title']=="une aventure de cendrillon", 'film_title'] = "a cinderella story"
data_clean.loc[data_clean['film_title']=="une ballade pour bobby long", 'film_title'] = "a love song for bobby long"
data_clean.loc[data_clean['film_title']=="une étoile est née", 'film_title'] = "a star is born"
data_clean.loc[data_clean['film_title']=="une famille immédiate", 'film_title'] = "instant family"
data_clean.loc[data_clean['film_title']=="une femme est une femme", 'film_title'] = "a woman is a woman"
data_clean.loc[data_clean['film_title']=="une ferme plus grande que nature", 'film_title'] = "the biggest little farm"
data_clean.loc[data_clean['film_title']=="une histoire de fantôme", 'film_title'] = "a ghost story"
data_clean.loc[data_clean['film_title']=="une histoire de sin city", 'film_title'] = "sin city"
data_clean.loc[data_clean['film_title']=="une jeune femme pleine de promesses", 'film_title'] = "promising young woman"
data_clean.loc[data_clean['film_title']=="une jolie femme", 'film_title'] = "pretty woman"
data_clean.loc[data_clean['film_title']=="une ligue en jupons", 'film_title'] = "a league of their own"
data_clean.loc[data_clean['film_title']=="une nuit au musée", 'film_title'] = "night at the museum"
data_clean.loc[data_clean['film_title']=="une promenade inoubliable", 'film_title'] = "a walk to remember"
data_clean.loc[data_clean['film_title']=="une seconde chance", 'film_title'] = "the best of me"
data_clean.loc[data_clean['film_title']=="une vie cachée", 'film_title'] = "a hidden life"
data_clean.loc[data_clean['film_title']=="une vie de bestiole", 'film_title'] = "a bug's life"
data_clean.loc[data_clean['film_title']=="une vie de mensonges", 'film_title'] = "body of lies"
data_clean.loc[data_clean['film_title']=="une vie fantastique", 'film_title'] = "captain fantastic"
data_clean.loc[data_clean['film_title']=="untouchable", 'film_title'] = "intouchables"
data_clean.loc[data_clean['film_title']=="v pour vendetta", 'film_title'] = "v for vendetta"
data_clean.loc[data_clean['film_title']=="vague de froid", 'film_title'] = "out cold"
data_clean.loc[data_clean['film_title']=="vendredi 13", 'film_title'] = "friday the 13th"
data_clean.loc[data_clean['film_title']=="vendredi 13 : chapitre vi - jason le mort-vivant", 'film_title'] = "jason lives: friday the 13th part vi"
data_clean.loc[data_clean['film_title']=="vers l'inconnu", 'film_title'] = "into the wild"
data_clean.loc[data_clean['film_title']=="vingt mille lieues sous les mers", 'film_title'] = "20,000 leagues under the sea"
data_clean.loc[data_clean['film_title']=="virtuosité", 'film_title'] = "virtuosity"
data_clean.loc[data_clean['film_title']=="vis libre ou crève", 'film_title'] = "live free or die hard"
data_clean.loc[data_clean['film_title']=="vivien leigh, autant en emporte le vent", 'film_title'] = "gone with the wind"
data_clean.loc[data_clean['film_title']=="vivre", 'film_title'] = "living"
data_clean.loc[data_clean['film_title']=="voici polly", 'film_title'] = "along came polly"
data_clean.loc[data_clean['film_title']=="vol au-dessus d'un nid de coucou", 'film_title'] = "one flew over the cuckoo's nest"
data_clean.loc[data_clean['film_title']=="vous avez un message", 'film_title'] = "you've got mail"
data_clean.loc[data_clean['film_title']=="voyage au bout de l'enfer", 'film_title'] = "the deer hunter"
data_clean.loc[data_clean['film_title']=="voyage au centre de la terre", 'film_title'] = "journey to the center of the earth"
data_clean.loc[data_clean['film_title']=="vrai mensonge", 'film_title'] = "true lies"
data_clean.loc[data_clean['film_title']=="watchmen : les gardiens", 'film_title'] = "watchmen"
data_clean.loc[data_clean['film_title']=="war of the worlds: annihilation", 'film_title'] = "war of the worlds"
data_clean.loc[data_clean['film_title']=="wes anderson, class, and aesthetic: the royal tenenbaums and fantastic mr. fox", 'film_title'] = "the royal tenenbaums"
data_clean.loc[data_clean['film_title']=="what we do in the shadows: interviews with some vampires", 'film_title'] = "interview with the vampire: the vampire chronicles"
data_clean.loc[data_clean['film_title']=="wolfgang amadeus mozart: lucio silla", 'film_title'] = "amadeus"
data_clean.loc[data_clean['film_title']=="wwa the inception", 'film_title'] = "inception"
data_clean.loc[data_clean['film_title']=="wwf: stone cold steve austin - lord of the ring", 'film_title'] = "the lord of the rings: the fellowship of the ring"
data_clean.loc[data_clean['film_title']=="x-man", 'film_title'] = "x-men"
data_clean.loc[data_clean['film_title']=="x-men, les origines: wolverine", 'film_title'] = "x-men origins: wolverine"
data_clean.loc[data_clean['film_title']=="y a-t-il un pilote dans l'avion?", 'film_title'] = "airplane!"
data_clean.loc[data_clean['film_title']=="yogi l'ours", 'film_title'] = "yogi bear"
data_clean.loc[data_clean['film_title']=="zoë bell: the woman behind the action of tarantino's 'once upon a time in hollywood'", 'film_title'] = "once upon a time in hollywood"
data_clean.loc[data_clean['film_title']=="zombieland : le doublé", 'film_title'] = "zombieland: double tap"
data_clean.loc[data_clean['film_title']=="the pursuit of happiness", 'film_title'] = "the pursuit of happyness"
data_clean.loc[data_clean['film_title']=="schinderl's list", 'film_title'] = "schindler's list"
data_clean.loc[data_clean['film_title']=="how to train your dragon: homecoming", 'film_title'] = "how to train your dragon"
data_clean.loc[data_clean['film_title']=="l'auberge espagnole", 'film_title'] = "the spanish apartment"
data_clean.loc[data_clean['film_title']=="dazed&confused", 'film_title'] = "dazed and confused"
data_clean.loc[data_clean['film_title']=="terminator", 'film_title'] = "the terminator"
data_clean.loc[data_clean['film_title']=="the hobbit", 'film_title'] = "the hobbit: an unexpected journey"
data_clean.loc[data_clean['film_title']=="ghost writer", 'film_title'] = "the ghost writer"
data_clean.loc[data_clean['film_title']=="là-haut", 'film_title'] = "up"
data_clean.loc[data_clean['film_title']=="dazed&confused", 'film_title'] = "dazed and confused"
data_clean.loc[data_clean['film_title']=="field of dreams: passing along the pastime", 'film_title'] = "field of dreams"
data_clean.loc[data_clean['film_title']=="victoria gotti: my father's daughter", 'film_title'] = "my father's daughter"
data_clean.loc[data_clean['film_title']=="intouchables", 'film_title'] = "the intouchables"
data_clean.loc[data_clean['film_title']=="the outlander", 'film_title'] = "outlander"
data_clean.loc[data_clean['film_title']=="y tu mamá también", 'film_title'] = "and your mother too"
data_clean.loc[data_clean['film_title']=="big hero 6: baymax returns", 'film_title'] = "big hero 6"
data_clean.loc[data_clean['film_title']=="une journée particulière", 'film_title'] = "Una giornata particolare"
data_clean.loc[data_clean['film_title']=="tree of life", 'film_title'] = "the tree of life"
data_clean.loc[data_clean['film_title']=="the elf", 'film_title'] = "elf"
data_clean.loc[data_clean['film_title']=="les choristes", 'film_title'] = "the chorus"
data_clean.loc[data_clean['film_title']=="il était une fois dans l'ouest", 'film_title'] = "once upon a time in the west"
data_clean.loc[data_clean['film_title']=="le dernier survivant", 'film_title'] = "the quiet earth"
data_clean.loc[data_clean['film_title']=="les sept péchés capitaux", 'film_title'] = "the seven deadly sins"
data_clean.loc[data_clean['film_title']=="young guns 2", 'film_title'] = "young guns ii"
data_clean.loc[data_clean['film_title']=="for our eyes only", 'film_title'] = "for your eyes only"
data_clean.loc[data_clean['film_title']=="les visiteurs", 'film_title'] = "the visitors"
data_clean.loc[data_clean['film_title']=="the good, the bad, & the ugly", 'film_title'] = "the good, the bad and the ugly"
data_clean.loc[data_clean['film_title']=="le comte de monte-cristo", 'film_title'] = "the count of monte cristo"
data_clean.loc[data_clean['film_title']=="le père noël est une ordure", 'film_title'] = "santa claus is a stinker"
data_clean.loc[data_clean['film_title']=="sisi", 'film_title'] = "sissi"
data_clean.loc[data_clean['film_title']=="bad boy", 'film_title'] = "bad boys"
data_clean.loc[data_clean['film_title']=="gone in sixty seconds", 'film_title'] = "gone in 60 seconds"
data_clean.loc[data_clean['film_title']=="megamind: the button of doom", 'film_title'] = "megamind"
data_clean.loc[data_clean['film_title']=="réalité", 'film_title'] = "reality"
data_clean.loc[data_clean['film_title']=="le pacte des loups", 'film_title'] = "brotherhood of the wolf"
data_clean.loc[data_clean['film_title']=="un prophète", 'film_title'] = "a prophet"
data_clean.loc[data_clean['film_title']=="le prénom", 'film_title'] = "what's in a name?"
data_clean.loc[data_clean['film_title']=="mamma mia", 'film_title'] = "mamma mia!"
data_clean.loc[data_clean['film_title']=="the seven deadly sins : cursed by light", 'film_title'] = "the seven deadly sins"
data_clean.loc[data_clean['film_title']=="divergence", 'film_title'] = "divergent"
data_clean.loc[data_clean['film_title']=="poupées russes", 'film_title'] = "russian dolls"
data_clean.loc[data_clean['film_title']=="beyond citizen kane", 'film_title'] = "citizen kane"
data_clean.loc[data_clean['film_title']=="el camino", 'film_title'] = "el camino: a breaking bad movie"
data_clean.loc[data_clean['film_title']=="punisher", 'film_title'] = "the punisher"
data_clean.loc[data_clean['film_title']=="la tour montparnasse infernale", 'film_title'] = "don't die too hard"
data_clean.loc[data_clean['film_title']=="tangled ever after", 'film_title'] = "tangled"
data_clean.loc[data_clean['film_title']=="la bella vita", 'film_title'] = "life is beautiful"
data_clean.loc[data_clean['film_title']=="end game", 'film_title'] = "avengers: endgame"
data_clean.loc[data_clean['film_title']=="the black panther", 'film_title'] = "black panther"
data_clean.loc[data_clean['film_title']=="the guardians of the galaxy holiday special", 'film_title'] = "the guardians of the galaxy"
data_clean.loc[data_clean['film_title']=="war game", 'film_title'] = "wargames"
data_clean.loc[data_clean['film_title']=="daniel craig vs james bond", 'film_title'] = "casino royale"
data_clean.loc[data_clean['film_title']=="unlocking davinci's code", 'film_title'] = "the da vinci code"
data_clean.loc[data_clean['film_title']=="war games", 'film_title'] = "wargames"
data_clean.loc[data_clean['film_title']=="monty python: the meaning of live", 'film_title'] = "the meaning of live"
data_clean.loc[data_clean['film_title']=="the black swan", 'film_title'] = "black swan"
data_clean.loc[data_clean['film_title']=="dodgeball", 'film_title'] = "dodgeball: a true underdog story"
data_clean.loc[data_clean['film_title']=="walli", 'film_title'] = "wall-e"
data_clean.loc[data_clean['film_title']=="deadpool 3", 'film_title'] = "deadpool"
data_clean.loc[data_clean['film_title']=="the watchmen", 'film_title'] = "watchmen"
data_clean.loc[data_clean['film_title']=="blood diamonds", 'film_title'] = "blood diamond"
data_clean.loc[data_clean['film_title']=="les robots", 'film_title'] = "i, robot"
data_clean.loc[data_clean['film_title']=="ninjago: masters of spinjitzu - day of the departed", 'film_title'] = "the departed"
data_clean.loc[data_clean['film_title']=="abyss", 'film_title'] = "the abyss"
data_clean.loc[data_clean['film_title']=="va, vis et deviens", 'film_title'] = "live and become"
data_clean.loc[data_clean['film_title']=="marrying mr. darcy", 'film_title'] = "marrying mr.darcy"
data_clean.loc[data_clean['film_title']=="le hussard sur le toit", 'film_title'] = "the horseman on the roof"
data_clean.loc[data_clean['film_title']=="mr. nobody", 'film_title'] = "mr.nobody"
data_clean.loc[data_clean['film_title']=="new york 1997", 'film_title'] = "escape from new york"
data_clean.loc[data_clean['film_title']=="five feet appart", 'film_title'] = "five feet apart"
data_clean.loc[data_clean['film_title']=="2 days in paris", 'film_title'] = "two days in paris"
data_clean.loc[data_clean['film_title']=="sånger från andra våningen", 'film_title'] = "songs from the second floor"
data_clean.loc[data_clean['film_title']=="le déclin de l'empire américain", 'film_title'] = "the decline of the american empire"
data_clean.loc[data_clean['film_title']=="kill bill: vol. i", 'film_title'] = "kill bill"
data_clean.loc[data_clean['film_title']=="the evil dead 2", 'film_title'] = "evil dead ii"
data_clean.loc[data_clean['film_title']=="being james bond: the daniel craig story", 'film_title'] = "casino royale"
data_clean.loc[data_clean['film_title']=="le transporteur 3", 'film_title'] = "transporter 3"
data_clean.loc[data_clean['film_title']=="portrait de la jeune fille en feu", 'film_title'] = "portrait of a lady on fire"
data_clean.loc[data_clean['film_title']=="le violon rouge", 'film_title'] = "the red violin"
data_clean.loc[data_clean['film_title']=="ivan le terrible", 'film_title'] = "ivan the terrible"
data_clean.loc[data_clean['film_title']=="la passion d'augustine", 'film_title'] = "the passion of augustine"
data_clean.loc[data_clean['film_title']=="il pleuvait des oiseaux", 'film_title'] = "and the birds rained down"
data_clean.loc[data_clean['film_title']=="crna mačka, beli mačor", 'film_title'] = "black cat, white cat"
data_clean.loc[data_clean['film_title']=="star trek: the captains", 'film_title'] = "star trek"
data_clean.loc[data_clean['film_title']=="la grenouille et la baleine", 'film_title'] = "tadpole and the whale"
data_clean.loc[data_clean['film_title']=="le chevalier vert", 'film_title'] = "the green knight"
data_clean.loc[data_clean['film_title']=="la belle verte", 'film_title'] = "the green planet"
data_clean.loc[data_clean['film_title']=="le grand bleu", 'film_title'] = "the big blue"
data_clean.loc[data_clean['film_title']=="ascenseur pour l'échafaud", 'film_title'] = "elevator to the gallows"
data_clean.loc[data_clean['film_title']=="babettes gæstebud", 'film_title'] = "babette's feast"
data_clean.loc[data_clean['film_title']=="les enfants du paradis", 'film_title'] = "children of paradise"
data_clean.loc[data_clean['film_title']=="danser dans le noir", 'film_title'] = "dancer in the dark"
data_clean.loc[data_clean['film_title']=="walkyrie", 'film_title'] = "valkyrie"
data_clean.loc[data_clean['film_title']=="jasmine french", 'film_title'] = "blue jasmine"
data_clean.loc[data_clean['film_title']=="c'est l'apocalypse", 'film_title'] = "apocalypse now"
data_clean.loc[data_clean['film_title']=="wo die grünen ameisen träumen", 'film_title'] = "where the green ants dream"
data_clean.loc[data_clean['film_title']=="benny et joon", 'film_title'] = "benny & joon"
data_clean.loc[data_clean['film_title']=="warriors of the wind", 'film_title'] = "nausicaa"
data_clean.loc[data_clean['film_title']=="césar et rosalie", 'film_title'] = "cesar & rosalie"
data_clean.loc[data_clean['film_title']=="bram stocker's dracula", 'film_title'] = "bram stoker's dracula"
data_clean.loc[data_clean['film_title']=="amicalement vôtre : hollywood au service de sa majesté", 'film_title'] = "on her majesty's secret service"
data_clean.loc[data_clean['film_title']=="south park: post covid special!", 'film_title'] = "south park: bigger, longer & uncut"
data_clean.loc[data_clean['film_title']=="une nuit au roxbury", 'film_title'] = "a night at the roxbury"
data_clean.loc[data_clean['film_title']=="fanny och alexander", 'film_title'] = "fanny and alexander"
data_clean.loc[data_clean['film_title']=="le pouvoir du chien", 'film_title'] = "the power of the dog"
data_clean.loc[data_clean['film_title']=="lettres à juliette", 'film_title'] = "letters to juliet"
data_clean.loc[data_clean['film_title']=="passager 57", 'film_title'] = "passenger 57"
data_clean.loc[data_clean['film_title']=="kill bill: vol. ii", 'film_title'] = "kill bill: vol.ii"
data_clean.loc[data_clean['film_title']=="the ghost writter", 'film_title'] = "the ghost writer"
data_clean.loc[data_clean['film_title']=="la fête du travail", 'film_title'] = "labor day"
data_clean.loc[data_clean['film_title']=="the boy in the striped pyjamas", 'film_title'] = "the boy in the striped pajamas"
data_clean.loc[data_clean['film_title']=="la guerre des mondes", 'film_title'] = "war of the worlds"
data_clean.loc[data_clean['film_title']=="autobiography of oldboy", 'film_title'] = "oldboy"
data_clean.loc[data_clean['film_title']=="el viaje de chihiro", 'film_title'] = "spirited away"
data_clean.loc[data_clean['film_title']=="león", 'film_title'] = "léon"
data_clean.loc[data_clean['film_title']=="planet earth 3", 'film_title'] = "planet earth"
data_clean.loc[data_clean['film_title']=="une histoire de mariage", 'film_title'] = "marriage story"
data_clean.loc[data_clean['film_title']=="ananas express", 'film_title'] = "pineapple express"
data_clean.loc[data_clean['film_title']=="hunger games: la révolte - partie i", 'film_title'] = "the hunger games: mockingjay - part 1"
data_clean.loc[data_clean['film_title']=="a tale before time", 'film_title'] = "the land before time"
data_clean.loc[data_clean['film_title']=="menteur, menteur", 'film_title'] = "liar liar"
data_clean.loc[data_clean['film_title']=="alien vs. prédateur", 'film_title'] = "alien vs. predator"
data_clean.loc[data_clean['film_title']=="bernardo bertolucci, le dernier empereur du cinema", 'film_title'] = "the last emperor"
data_clean.loc[data_clean['film_title']=="l'abysse", 'film_title'] = "the abyss"
data_clean.loc[data_clean['film_title']=="boréal express", 'film_title'] = "the polar express"
data_clean.loc[data_clean['film_title']=="men of honnor", 'film_title'] = "men of honor"
data_clean.loc[data_clean['film_title']=="bar routier", 'film_title'] = "road house"
data_clean.loc[data_clean['film_title']=="arizona junior", 'film_title'] = "raising arizona"
data_clean.loc[data_clean['film_title']=="entre deux plages", 'film_title'] = "beaches"
data_clean.loc[data_clean['film_title']=="no country for old man", 'film_title'] = "no country for old men"
data_clean.loc[data_clean['film_title']=="newa of the world", 'film_title'] = "news of the world"
data_clean.loc[data_clean['film_title']=="crazy rich à asians", 'film_title'] = "crazy rich asians"
data_clean.loc[data_clean['film_title']=="indiana jones and the raiders of the lost Aark", 'film_title'] = "indiana jones and the raiders of the lost ark"
data_clean.loc[data_clean['film_title']=="fourmiz", 'film_title'] = "antz"
data_clean.loc[data_clean['film_title']=="sato no chihiro", 'film_title'] = "spirited away"

In [ ]:
#CETTE PARTIE DU CODE NÉCESSITE UNE CLÉ D'API OMDB VALIDE

#Création de nouvelles features relative au titre du film
data_clean =data_clean.assign(movie_release = np.nan)
data_clean =data_clean.assign(movie_rating = np.nan)
data_clean =data_clean.assign(movie_duration = np.nan)
data_clean =data_clean.assign(movie_director = np.nan)
data_clean =data_clean.assign(movie_writer = np.nan)
data_clean =data_clean.assign(movie_actors = np.nan)
data_clean =data_clean.assign(movie_language = np.nan)
data_clean =data_clean.assign(movie_country = np.nan)
data_clean =data_clean.assign(movie_awards = np.nan)
data_clean =data_clean.assign(movie_metascore = np.nan)
data_clean =data_clean.assign(movie_IMDb_score = np.nan)
data_clean =data_clean.assign(movie_IMDb_votes = np.nan)
data_clean =data_clean.assign(movie_metacritic = np.nan)
data_clean =data_clean.assign(movie_rotten_tomatoes = np.nan)
data_clean =data_clean.assign(movie_box_office = np.nan)
data_clean =data_clean.assign(movie_plot = np.nan)
data_clean =data_clean.assign(movie_tags = np.nan)
data_clean =data_clean.assign(movie_IMDb_id = np.nan)

# première recherche de données 
char_e = ["é", "è", "ê", "ë", "ĕ", "ẽ", "ē", "ė", "æ"]
char_a = ["à", "ä", "â", "ã", "á", "å"]
char_c = ["ç", "ĉ"]
char_i = ["ĭ", "ï", "ì", "ĩ", "ϊ", "î"]
char_o =["ö", "ò", "ö"]
char_spec =["¡", "¿", "$", "%"]
#"½" "œ"

for title in data_clean.film_title.unique():
    movie_title = title.replace(" ", "+")
    for char in char_e:
        movie_title = movie_title.replace(char, "e")
    for char in char_a:
        movie_title = movie_title.replace(char, "a")
    for char in char_c:
        movie_title = movie_title.replace(char, "c")
    for char in char_i:
        movie_title = movie_title.replace(char, "i")
    for char in char_o:
        movie_title = movie_title.replace(char, "o")
    for char in char_spec:
        movie_title = movie_title.replace(char, "")
    movie_title = movie_title.replace("½", "1/2")
    movie_title = movie_title.replace("œ", "oe")
    url = "http://www.omdbapi.com/?t="+movie_title+"&apikey=2405ff46"
    try:
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())
        data_clean.loc[data_clean['film_title'] == title, 'movie_release']= data['Year']
        data_clean.loc[data_clean['film_title'] == title, 'movie_rating']= data['Rated'] 
        data_clean.loc[data_clean['film_title'] == title, 'movie_duration']= data['Runtime']
        data_clean.loc[data_clean['film_title'] == title, 'movie_director']= data['Director']
        data_clean.loc[data_clean['film_title'] == title, 'movie_writer']= data['Writer']
        data_clean.loc[data_clean['film_title'] == title, 'movie_actors']= data['Actors']
        data_clean.loc[data_clean['film_title'] == title, 'movie_language']= data['Language']
        data_clean.loc[data_clean['film_title'] == title, 'movie_country']= data['Country']
        data_clean.loc[data_clean['film_title'] == title, 'movie_metascore']= data['Metascore']
        data_clean.loc[data_clean['film_title'] == title, 'movie_box_office']= data['BoxOffice']
        data_clean.loc[data_clean['film_title'] == title, 'movie_IMDb_score']= data['imdbRating']
        data_clean.loc[data_clean['film_title'] == title, 'movie_IMDb_votes']= data['imdbVotes']
        data_clean.loc[data_clean['film_title'] == title, 'movie_awards']= data['Awards']
        plot = data['Plot']
        plot = plot.replace(";",".")
        data_clean.loc[data_clean['film_title'] == title, 'movie_plot']= plot
        data_clean.loc[data_clean['film_title'] == title, 'movie_tags']= data['Genre']
        data_clean.loc[data_clean['film_title'] == title, 'movie_IMDb_id']= data['imdbID']           
    except:
        pass
    try:
        data_clean.loc[data_clean['film_title'] == title, 'movie_rotten_tomatoes']= data['Ratings'][1]['Value']
        data_clean.loc[data_clean['film_title'] == title, 'movie_metacritic']= data['Ratings'][2]['Value']
    except:
        data_clean.loc[data_clean['film_title'] == title, 'movie_rotten_tomatoes']= "N/A"
        data_clean.loc[data_clean['film_title'] == title, 'movie_metacritic']= "N/A"


In [ ]:
# résolution problèmes au cas par cas:
list_movie_fix =[["bienvenue chez les ch'tis", "tt1064932"], ["la famille bélier", "tt3547740"], ["pour la suite du monde", "tt0057423"], ["le fabuleux destin d'amélie poulain", "tt0211915"],
                ["pride and prejudice", "tt0414387"], ["un long dimanche de fiançailles", "tt0344510"], ["independance day", "tt0116629"], ["fast and furious", "tt1013752"], ["nausicaa","tt0087544"],
                ["la vie d'adèle - chapitres 1 et 2", "tt2278871"], ["l'aventure c'est l'aventure", "tt0066798"], ["viva pedro: the life & times of pedro almodóvar", "tt0287467"],
                ["la guerre des boutons", "tt0054949"], ["la gloire de mon père","tt0099669"], ["nuovo cinema paradiso","tt0095765"], ["les grandes chaleurs", "tt1359423"], ["le goût des autres","tt0216787"],
                ["le gendarme de saint-tropez","tt0058135"], ["les ailes du désir","tt0093191"], ["indagine su un cittadino al di sopra di ogni sospetto","tt0065889"],
                ["les tontons flingueurs","tt0057591"], ["les êtres chers","tt4881276"], ["trois couleurs : bleu","tt0108394"], ["la cité de la peur","tt0109440"], ["répertoire des villes disparues","tt9068100"],
                ["alexandre le bienheureux","tt0061343"],["les vacances de monsieur hulot","tt0046487"], ["la vie et rien d'autre","tt0098596"], ["lola rennt","tt0130827"], ["la bête lumineuse","tt0129807"],
                ["oss 117 : le caire, nid d'espions","tt0464913"], ["oss 117 : bons baisers d'afrique","tt7972012"], ["wall·e","tt0910970"], ["les ordres","tt0071949"], ["m. le maudit","tt0022100"],
                ["la grande bellezza","tt2358891"], ["c'est pas moi, je le jure!","tt1163752"], ["les quatre cents coups","tt0053198"], ["la tigre e la neve","tt0419198"], ["à vos marques... party!","tt0956317"],
                ["le château de ma mère","tt0099266"], ["coco avant chanel","tt1035736"],["le maître de musique","tt0095606"],["horloge biologique","tt0437230"],["le professionnel","tt0082949"],
                ["život je čudo","tt0322420"],["un 32 août sur terre","tt0156248"],["hable con ella","tt0287467"], ["la graine et le mulet","tt0487419"], ["maradona, la mano di dio","tt0465737"],
                ["visages, villages","tt5598102"],["les 12 travaux d'astérix","tt0072901"], ["conte d'automne","tt0137439"],["jésus de montréal","tt0097635"],["mademoiselle de joncquières","tt7530986"],
                ["paris brûle-t-il?","tt0060814"],["astérix et la surprise de césar","tt0088748"],["j'ai tué ma mère","tt1424797"],["anvil! the story of anvil","tt1157605"],["jules et jim","tt0055032"],
                ["astérix et cléopâtre","tt0062687"],["oslo, 31. august","tt1736633"],["la femme de mon frère","tt10155342"], ["les amours imaginaires","tt1600524"],["la véritable histoire du chat botté","tt1239462"],
                ["la maison sous les arbres","tt0067382"], ["kona fer í stríð","tt7279188"],["les aventures de rabbi jacob","tt0069747"],["la chute de l'empire américain","tt7231342"],
                ["nueve reinas","tt0247586"],["det sjunde inseglet","tt0050976"],["37°2 le matin","tt0090563"],["fratello sole, sorella luna","tt0069824"],["kaamelott - premier volet","tt9844322"],
                ["mononoke hime","tt0119698"],["clair de femme","tt0078978"],["de vrais mensonges","tt1529569"],["les petits mouchoirs","tt1440232"],["l'aile ou la cuisse","tt0074103"],["les doigts croches","tt1305796"],
                ["tu dors nicole","tt3483194"],["alice et le maire","tt8903840"],["les glaneurs et la glaneuse","tt0247380"],["cléo de 5 à 7","tt0055852"],["astérix : le domaine des dieux","tt3759416"],
                ["étreintes brisées","tt0913425"],["la meglio gioventù","tt0346336"],["léon","tt0110413"],["le juge et l'assassin","tt0073219"],["nói albínói","tt0351461"],["marius et jeannette","tt0119620"],
                ["kill bill: vol.ii","tt0378194"],["8 femmes","tt0283832"],["harold et maude","tt0067185"],["au revoir là-haut","tt5258850"],["le vieux fusil","tt0073864"],["sale prof","tt1284575"],
                ["borat: cultural learnings of america for make benefit glorious nation of kazakhstan","tt0443453"],["terapia do medo","tt2094955"],["un dimange à la campagne","tt0088318"],
                ["kirikou et la sorcière","tt0181627"],["blinkende lygter","tt0236027"],["les enfants loups, ame et yuki","tt2140203"],["lazzaro felice","tt6752992"], ["arielle dombasle au crasy horse","tt2043814"],
                ["la saga twilight: révélation - partie 2","tt1673434"],["le transporteur","tt0293662"],["jfk unsolved: the real conspiracies","tt0102138"],["la biographie inventée","tt7471026"],
                ["fantastic mr. fox","tt0432283"],["tirez sur le pianiste","tt0054389"],["une sirène à paris", "tt9209312"],["l'année dernière à marienbad","tt0054632"],["xavier dolan: à l'impossible je suis tenu","tt3612616"],
                ["gruppo di famiglia in un interno","tt0071585"],["mitt liv som hund","tt0089606"],["batman: le film","tt4116284"],["le charme discret de la bourgeoisie","tt0068361"],
                ["les enfants du marais","tt0167926"],["the lion king 1½","tt0318403"],["the host (2013)","tt1517260"],["seven samourai","tt0047478"],["en terrains connus","tt1671457"],
                ["la stanza del figlio","tt0208990"],["15 février 1839","tt0237994"],["la terre vue du cœur","tt8297660"],["queen: live aid","tt1421377"], ["um cinema quase paradiso","tt0095765"],
                ["le chant du loup","tt7458762"],["todo sobre mi madre","tt0185125"],["oss 117 : rio ne répond plus","tt1167660"],["effroyables jardins","tt0285052"],["mrs. doubtfire","tt0107614"],
                ["les parapluies de cherbourg","tt0058450"],["vals im bashir","tt1185616"],["lance et compte - téléfilms","tt1466572"],["un pays sans bon sens!","tt0066503"],["quand l'amour se creuse un trou","tt7298796"],
                ["la maison de papier : le phénomène","tt6468322"],["mr. holland's opus","tt0113862"],["elles étaient cinq","tt0400400"],["l'assassinat du père noël","tt0033357"],["la cité des enfants perdus","tt0112682"],
                ["les couloirs du temps: les visiteurs 2","tt0120882"],["la vie scolaire","tt8783532"],["un monde sans pitié","tt0098552"],["jonas qui aura 25 ans en l'an 2000","tt0074718"],["saul fia","tt3808342"],
                ["dernier domicile connu","tt0064225"], ["monsieur ibrahim et les fleurs du coran","tt0329388"],["il était une fois les boys","tt2747786"],["les sous-doués passent le bac","tt0081541"],
                ["13 tzameti","tt0475169"],["le pro","tt0082949"],["astérix chez les bretons","tt0090667"],["elvis gratton 2: miracle à memphis","tt0180668"],["dédé, à travers les brumes","tt1242845"],
                ["la vengeance de monte cristo","tt0245844"],["angélique, marquise des anges","tt0057846"],["peaky blinders: the true story","tt2442560"],["adú","tt9616700"],["au royaume des cieux","tt0041137"],
                ["la mala educación","tt0275491"],["ce que le jour doit à la nuit","tt1934205"],["christiane f. - wir kinder vom bahnhof zoo","tt0082176"],["le déclin de l'empire masculin","tt0090985"],
                ["babamın kemanı","tt14369276"],["qu'est-ce qu'on a fait au bon dieu ?","tt2800240"],["modus anomali","tt2182019"],["la double vie de véronique","tt0101765"],["tom à la ferme","tt2427892"],
                ["félix et meira","tt3685218"],["pokémon 4ever - celebi, la voix de la forêt","tt0287635"], ["jesús de nazareth","tt0233975"],["au revoir les enfants","tt0092593"],["diario segreto di amarcord","tt2038256"],
                ["la mélodie du boucher","tt0059742"],["le 8ème jour","tt0116581"],["demain tout commence","tt5078204"],["diarios de motocicleta","tt0318462"],["le bonheur est dans le pré","tt0112556"],
                ["fúsi","tt2611652"],["giù la testa","tt0067140"],["180° south","tt1407927"],["l'apôtre","tt3493128"],["the note book","tt0332280"],["le jeune karl marx","tt1699518"],["i... comme icare","tt0079322"],
                ["crazy rich asians","tt3104988"],["habitación en roma","tt1263750"],["demon slayer : le train de l'infini","tt11032374"],["azur et asmar","tt0439123"],["the talented mr. ripley","tt0134119"],
                ["l'important c'est d'aimer","tt0073155"],["les rois mongols","tt6320684"],["verdens verste menneske","tt10370710"],["les aventures du baron munchausen","tt0096764"],["avis de mistral","tt3013588"],
                ["roma città aperta","tt0038890"],["il y a des jours... et des lunes","tt0099827"],["le chat dans le sac","tt0057937"],["das leben der anderen","tt0405094"],["l'enfant du soleil","tt11192758"],
                ["l'âge des ténèbres","tt0819953"],["je vais bien, ne t'en fais pas","tt0485241"],["la disparition des lucioles","tt8449092"],["la déesse des mouches à feu","tt4131756"],
                ["dünyayı kurtaran adam","tt0182060"],["la montaña sagrada","tt0071615"],["comme une vague","tt13653682"],["io sono l'amore","tt1226236"],["professor marston and the wonder women","tt6133130"],
                ["fyre : le meilleur festival qui n'a jamais eu lieu","tt9412098"],["relatos salvajes","tt3011894"],["cornouailles","tt0377084"],["les demoiselles de rochefort","tt0062873"],
                ["béliveau: le documentaire","tt6290542"],["planes, trains and automobiles remake","tt0093748"],["elle s'en va","tt2289538"],["la dea fortuna","tt10256570"],["anne roumanoff : aimons-nous les uns les autres","tt8386708"],
                ["hellboy ii: l'armée d'or","tt0411477"],["9/11: the day that changed america","tt14941342"],["la soupe aux choux","tt0083109"],["peau d'âne","tt0066207"],["è stata la mano di dio","tt12680684"],
                ["la fille du puisatier","tt1473063"],["the grease story","tt0077631"],["les navets blancs empêchent de dormir","tt2011099"],["la rafle du vel d'hiv : l'histoire révélée","tt21354830"],
                ["séraphin: un homme et son péché","tt0221013"],["jfk: the assassination","tt0102138"],["anastasia: once upon a time","tt7014356"],["l'attaque des titans","tt2560140"],
                ["le grand blond avec une chaussure noire","tt0068655"],["les bronzés font du ski","tt0078907"],["le parfum : histoire d'un meurtrier","tt0396171"],["la fille au ventre rond","tt4292432"],
                ["ma nuit chez maud","tt0064612"],["è un bracchetto flashdance, charlie brown","tt0085549"],["les triplettes de belleville","tt0286244"],["jay and bob strike back","tt0261392"],
                ["mayrig","tt0102421"],["peppa pig: the fire engine","tt6771444"],["le mari de la coiffeuse","tt0100112"],["giulietta degli spiriti","tt0059229"],["despicable me presents: minion madness","tt2129997"],
                ["mean girls - the musical","tt0377092"],["mùa hè chiều thẳng đứng","tt0224578"],["vale abraão","tt0108471"],["twelve angry men","tt0050083"],["werk ohne autor","tt5311542"],
                ["la collectionneuse","tt0061495"],["en attendant bojangles","tt11310608"],["pokémon: the power of us","tt8108230"],["les valeurs de la famille addams","tt0106220"],
                ["rocco e i suoi fratelli","tt0054248"],["astérix et les vikings","tt0371552"],["only the braves","tt3829920"],["un genio, due compari, un pollo","tt0073036"],["le roman d'un tricheur","tt0028201"],
                ["mr. magorium's wonder emporium","tt0457419"],["debbie ocean 8","tt5164214"],["brüno","tt0889583"],["spiral: from the book of saw","tt10342730"],["christopher doyle in conversation: days of being wild","tt0101258"],
                ["live die repeat and repeat","tt7520794"],["here comes mr. jordan","tt0033712"],["una mujer fantástica","tt5639354"],["bo burnham: inside","tt14544192"],["the girl with the dragon tattoo (2011)","tt1568346"],
                ["guibord s'en va-t-en guerre","tt4116116"],["transformers 3: dark of the moon","tt1399103"],["palmeras en la nieve","tt3202202"],["les oiseaux ivres","tt11051898"],
                ["il casanova di federico fellini","tt0074291"],["l'amour dure trois ans","tt1638328"],["l'histoire de jean-christophe","tt4575576"],["ivresse au combat","tt0111512"],
                ["high school musical: senior year","tt0962726"],["aguirre, der zorn gottes","tt0068182"],["güeros","tt3499048"],["la nuit de varennes","tt0084423"],["laurentie","tt1972694"],
                ["danny ocean 13","tt0496806"],["pelle erobreren","tt0093713"],["à vos marques... party! 2","tt1322410"],["unleashing mr. darcy","tt0414387"],["le scaphandre et le papillon","tt0401383"],
                ["la vie heureuse de léopold z","tt0059873"],["14 jours, 12 nuits","tt6976172"],["on the road by jack kerouac","tt0337692"],["le corniaud","tt0057967"],["morte a venezia","tt0067445"],
                ["como agua para chocolate","tt0103994"],["glee: keep on believin'","tt2347379"],["birds of prey and the fantabulous emancipation of one harley quinn","tt7713068"],
                ["la marche de l'empereur","tt0428803"],["les choses de la vie","tt0064165"],["alphée des étoiles","tt2639422"],["la leggenda del pianista sull'oceano","tt0120731"],
                ["johnny cash - i walk the line","tt0358273"],["el lado oscuro del corazón","tt0104662"],["abattoir 5","tt0069280"],["l'homme de rio","tt0058203"],["pokémon: la destiné de deoxys","tt0435286"],
                ["house arrest starring chris colombo: a docu-comedy based on reality","tt0495124"],["summer of fossil leaves","tt3902612"],["cóctel de camarones. en el día de la secretaria","tt0087108"],
                ["de rouille et d'os","tt2053425"],["zizanie dans le bronx","tt0113326"],["vénus beauté (institut)","tt0174330"],["lol: the movie","tt1194616"],["la vie est un long fleuve tranquille","tt0096386"],
                ["xi gan dao","tt2098878"],["cours apres moi que je t'attrape","tt0150211"],["mr. right","tt2091935"],["la marieuse","tt0209475"],["un amour infini","tt0186894"],["miami vice: brother's keeper","tt0430357"],
                ["málmhaus","tt2374902"],["comment je me suis disputé... (ma vie sexuelle)","tt0115928"],["1917 : il était une fois, la révolution","tt8579674"],["bac nord","tt10404944"],
                ["l'arnacoeur","tt1465487"],["à la croisée des mondes : la boussole d'or","tt0385752"],["le club vinland","tt11433420"],["mauvais sang","tt0091497"],["le dernier métro","tt0080610"],
                ["au revoir le bonheur","tt16235682"],["continental, un film sans fusil","tt1093815"],["8½","tt0056801"],["kangourou jack","tt0257568"],["les aventures de tintin : le secret de la licorne","tt0983193"],
                ["le guide de la famille parfaite","tt11991786"],["la planète au trésor","tt0133240"],["les 120 jours de sodome","tt0073650"],["dr. jekyll and mr. hyde","tt0022835"],["mr. ove","tt4080728"],
                ["nosferatu - phantom der nacht","tt0079641"],["run lola run the jewels run","tt0130827"],["mr. kaplan","tt2642524"],["jason lives: friday the 13th part vi","tt0091080"],
                ["le parapluie de cherbourg","tt0058450"],["les choses qu'on dit, les choses qu'on fait","tt12443930"],["la chute de sparte","tt6632664"],["go! saitama","tt4508902"],
                ["a nightmare on elm street part 2: freddy's revenge","tt0089686"],["the naked gun 2½: the smell of fear","tt0102510"],["astérix & obélix : mission cléopâtre","tt0250223"],
                ["la guerre des tuques","tt0087373"],["twin peaks","tt0098936"],["Les uns et les autres","tt0083260"],["l'impossible","tt1649419"],["the silent voice","tt5323662"],["chien de garde","tt5323662"],
                ["le hérisson","tt1442519"], ["sato no chihiro","tt0245429"],["la fracture","tt13321730"], ["downton abbey","tt1606375"], ["les uns et les autres","tt0083260"], ["outlander", "tt3006802"], ["jagten", "tt2106476"], ["prison 101", "tt2561572"], ["the seven deadly sins", "tt3909224"], 
                ["the handmaid's tale","tt5834204"], ["the outsider","tt8550800"], ["arrived","tt9448676"], ["sex and the city","tt0159206"], ["così è la vita", "tt0166556"], ["over the garden wall","tt3718778"],
                ["game of thrones", "tt0944947"], ["maurice richard","tt0460505"], ["the walking dead", "tt1520211"], ["the office", "tt0386676"], ["the wire", "tt0306414"], ["the crown", "tt4786824"], 
                ["planet earth", "tt0795176"], ["the witcher", "tt5180504"], ["district 13", "tt5090994"], ["illusions perdues", "tt10505316"], ["oblivious", "tt22061554"], ["made in abyss","tt7222086"],
                ["homeland", "tt1796960"], ["outsiders", "tt4816626"], ["westworld", "tt0475784"], ["the boys","tt1190634"], ["bleach", "tt0434665"], ["star wars: the clone wars","tt0458290"],
                ["arcane","tt11126994"], ["stranger things", "tt4574334"], ["young blood", "tt0024795"], ["the stand", "tt1831804"], ["guerre d'algerie - la déchirure", "tt2286447"], ["my hero academia", "tt5626028"],
                ["the blacklist", "tt2741602"], ["les poupées russes", "tt0409184"], ["narcos", "tt2707408"], ["panique au village", "tt1433540"], ["cartel","tt15134202"], ["trailer park boys", "tt0290988"],
                ["the punisher", "tt5675620"], ["yellowstone", "tt4236770"], ["un cœur en hiver", "tt0105682"], ["the thorn birds","tt0085101"], ["grimm", "tt1830617"], ["hodejegerne", "tt1614989"],
                ["lucifer", "tt4052886"], ["good times", "tt0070991"], ["shameless", "tt1586680"], ["maniac", "tt5580146"], ["medal of honor", "tt7440274"], ["braindead", "tt0103873"], ["berserk", "tt0318871"],
                ["belle et sébastien", "tt3146360"], ["sous la peau", "tt1441395"], ["la baby-sitter", "tt0073205"], ["this is us", "tt5555260"], ["black fish", "tt9547482"], ["euphoria", "tt8772296"],
                ["the arrangement", "tt4941288"], ["humans", "tt4122068"], ["crown", "tt4786824"], ["kimi no nawa","tt0161157"],["suits", "tt1632701"], ["attack on titan", "tt2560140"], ["anne of green gables", "tt0088727"],
                ["un dimanche à la campagne","tt0088318"], ["prison break", "tt0455275"], ["new girl", "tt1826940"],["requiem", "tt6712390"], ["teen wolf", "tt1567432"], ["percy jackson & the olympians: the lightning thief", "tt0814255"],
                ["friends", "tt0108778"], ["heartstopper", "tt10638036"], ["sword art online", "tt2250192"], ["band of brothers", "tt0185906"], ["l'entre-deux", "tt12223436"], ["our planet", "tt9253866"], 
                ["the flash", "tt3107288"], ["rise of the teenage mutant ninja turtles", "tt6601082"], ["zed","tt0460387"], ["la grande école", "tt0340137"], ["black mirror", "tt2085059"], ["chernobyl", "tt7366338"], 
                ["attila", "tt0259127"], ["snowdrop", "tt13304700"], ["cidade de deus", "tt0317248"]
                ]

In [ ]:
#On ajoute les données manquantes

for element in list_movie_fix:
    title = element[0]
    id = element[1]
    #print(title)
    #print(id)
    url = "http://www.omdbapi.com/?i="+id+"&apikey=2405ff46"
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    #print(data)
    data_clean.loc[data_clean['film_title'] == title, 'movie_release']= data['Year']
    data_clean.loc[data_clean['film_title'] == title, 'movie_rating']= data['Rated'] 
    data_clean.loc[data_clean['film_title'] == title, 'movie_duration']= data['Runtime']
    data_clean.loc[data_clean['film_title'] == title, 'movie_director']= data['Director']
    data_clean.loc[data_clean['film_title'] == title, 'movie_writer']= data['Writer']
    data_clean.loc[data_clean['film_title'] == title, 'movie_actors']= data['Actors']
    data_clean.loc[data_clean['film_title'] == title, 'movie_language']= data['Language']
    data_clean.loc[data_clean['film_title'] == title, 'movie_country']= data['Country']
    data_clean.loc[data_clean['film_title'] == title, 'movie_metascore']= data['Metascore']
    data_clean.loc[data_clean['film_title'] == title, 'movie_IMDb_score']= data['imdbRating']
    data_clean.loc[data_clean['film_title'] == title, 'movie_IMDb_votes']= data['imdbVotes']
    data_clean.loc[data_clean['film_title'] == title, 'movie_awards']= data['Awards']
    data_clean.loc[data_clean['film_title'] == title, 'movie_tags']= data['Genre']
    plot = data['Plot']
    plot = plot.replace(";",".")
    data_clean.loc[data_clean['film_title'] == title, 'movie_plot']= plot 
    data_clean.loc[data_clean['film_title'] == title, 'movie_IMDb_id']= data['imdbID']
    try:
        data_clean.loc[data_clean['film_title'] == title, 'movie_box_office']= data['BoxOffice']
    except:
        data_clean.loc[data_clean['film_title'] == title, 'movie_box_office']= "N/A"
    try:
        data_clean.loc[data_clean['film_title'] == title, 'movie_rotten_tomatoes']= data['Ratings'][1]['Value']
        data_clean.loc[data_clean['film_title'] == title, 'movie_metacritic']= data['Ratings'][2]['Value']
    except:
        data_clean.loc[data_clean['film_title'] == title, 'movie_rotten_tomatoes']= "N/A"
        data_clean.loc[data_clean['film_title'] == title, 'movie_metacritic']= "N/A"
data_clean.loc[data_clean.film_title == "la vie d'adèle - chapitres 1 et 2"]

In [ ]:
#remplacement des données vides par N/A
data_clean.loc[data_clean.movie_IMDb_score.isna(),'movie_IMDb_score'] = "N/A"
data_clean.loc[data_clean.movie_IMDb_votes.isna(),'movie_IMDb_votes'] = "N/A"
data_clean.loc[data_clean.movie_box_office.isna(),'movie_box_office'] = "N/A"
data_clean.loc[data_clean.movie_awards.isna(),'movie_awards'] = "N/A"

data_final = data_clean.dropna(axis=0)

In [ ]:
#ajout d'une feature donnant la différence entre la date de naissance du sondé et la date de sortie de son film favori

for date in data_final.movie_release:
    if len(date)>4:
        data_final.loc[data_final.movie_release == date, 'movie_release'] = date[0]+date[1]+date[2]+date[3]
data_final =data_final.assign(movie_age_link = np.nan)
data_final.reset_index
for i in data_final.index:
    date_film = data_final.loc[data_final.index == i, 'movie_release']
    age_sonde = data_final.loc[data_final.index == i, 'age']
    ratio_age_film = int(int(date_film.iloc[0])+ age_sonde.iloc[0] -2022)
    data_final.loc[data_final.index == i, 'movie_age_link'] = ratio_age_film

#correction feature movie_box_office
for i in data_final.index:
    box_office = data_final.loc[data_final.index == i, 'movie_box_office'].iloc[0]
    if type(box_office) != int:
        if box_office == "N/A":
            s=0
        else:
            s = box_office.replace("$","")
            s = int(s.replace(",",""))
        data_final.loc[data_final.index == i, 'movie_box_office'] = s

#correction de la feature movie_rotten_tomatoes

for i in data_final.index:
    rt_score = data_final.loc[data_final.index == i, 'movie_rotten_tomatoes'].iloc[0]
    if type(rt_score) != int:
        if rt_score == "N/A":
            rt_score=0
        else:
            rt_score = int(rt_score.replace("%",""))
        data_final.loc[data_final.index == i, 'movie_rotten_tomatoes'] = rt_score

#correction de la feature movie_metacritic

for i in data_final.index:
    mc_score = data_final.loc[data_final.index == i, 'movie_metacritic'].iloc[0]
    if type(mc_score) != int:
        if mc_score == "N/A":
            mc_score = 0
        else:
            mc_score = int(int(mc_score.replace("/",""))/1000 -0.1)
        data_final.loc[data_final.index == i, 'movie_metacritic'] = mc_score

#correction de la feature movie_IMDb_votes

for i in data_final.index:
    imdb_votes = data_final.loc[data_final.index == i, 'movie_IMDb_votes'].iloc[0]
    if type(imdb_votes) != int:
        if imdb_votes == "N/A":
            imdb_votes = 0
        else:
            imdb_votes = int(imdb_votes.replace(",",""))
        data_final.loc[data_final.index == i, 'movie_IMDb_votes'] = imdb_votes

#correction de la feature movie_IMDb_score

for i in data_final.index:
    imdb_score = data_final.loc[data_final.index == i, 'movie_IMDb_score'].iloc[0]
    if type(imdb_score) != int:
        if imdb_score == "N/A":
            imdb_score = 0
        else:
            imdb_score = int(float(imdb_score)*10)
        data_final.loc[data_final.index == i, 'movie_IMDb_score'] = imdb_score

#correction de la feature movie_metascore

for i in data_final.index:
    m_score = data_final.loc[data_final.index == i, 'movie_metascore'].iloc[0]
    if type(m_score) != int:
        if m_score == "N/A":
            m_score = 0
        else:
            imdb_score = int(m_score)
        data_final.loc[data_final.index == i, 'movie_metascore'] = m_score

#correction de la feature movie_duration:

for i in data_final.index:
    duration = data_final.loc[data_final.index == i, 'movie_duration'].iloc[0]
    duration = duration[:-4]
    try:
        duration = int(duration)
    except:
        duration = 0
    data_final.loc[data_final.index == i, 'movie_duration'] = duration

#ajout de features pour les récompenses de films

data_final =data_final.assign(movie_oscar_wins = np.nan)
data_final =data_final.assign(movie_bafta_wins = np.nan)
data_final =data_final.assign(movie_oscar_nominations = np.nan)
data_final =data_final.assign(movie_bafta_nominations = np.nan)
data_final =data_final.assign(movie_wins = np.nan)
data_final =data_final.assign(movie_nominations = np.nan)
data_final =data_final.assign(movie_prizes_total = np.nan)

for i in data_final.index:
    awards = data_final.loc[data_final.index == i, 'movie_awards'].iloc[0]
    oscars = 0
    oscar_nom = 0
    bafta = 0
    bafta_nom = 0
    wins = 0
    nominations = 0
    if awards != "N/A":
        if awards[0] =="W":
            awards = awards[4:]
            prizes = awards[0]
            awards = awards[1:]
            if awards[0] != " ":
                prizes = prizes + awards[0]
                awards = awards[1:]
            if awards[1] == "O":
                oscars = prizes
                while awards[0] not in ["0","1","2","3","4","5","6","7","8","9"]:
                    awards = awards[1:]
            elif awards[1] == "B":
                bafta = prizes
                while awards[0] not in ["0","1","2","3","4","5","6","7","8","9"]:
                    awards = awards[1:]
        elif awards[0] == "N":
            while awards[0] not in ["0","1","2","3","4","5","6","7","8","9"]:
                awards = awards[1:]
            prizes = awards[0]
            awards = awards[1:]
            if awards[0] != " ":
                prizes = prizes + awards[0]
                awards = awards[1:]
            if awards[1] == "O":
                oscar_nom = prizes
                while awards[0] not in ["0","1","2","3","4","5","6","7","8","9"]:
                    awards = awards[1:]
            elif awards[1] == "B":
                bafta_nom = prizes
                while awards[0] not in ["0","1","2","3","4","5","6","7","8","9"]:
                    awards = awards[1:]
        totals = "0"
        while awards[0] != " ":
            totals = totals + awards[0]
            awards = awards[1:]
        if awards[1]=="w":
            wins = totals
            awards = awards[5:]
            if len(awards)>1:
                while awards[0] not in ["0","1","2","3","4","5","6","7","8","9"]:
                    awards = awards[1:]
                totals = "0"
                while awards[0] != " ":
                    totals += awards[0]
                    awards = awards[1:]
                nominations = totals
        else:
            nominations = totals
    data_final.loc[data_final.index == i, 'movie_oscar_wins'] = int(oscars)
    data_final.loc[data_final.index == i, 'movie_bafta_wins'] = int(bafta)
    data_final.loc[data_final.index == i, 'movie_oscar_nominations'] = int(oscar_nom)
    data_final.loc[data_final.index == i, 'movie_bafta_nominations'] = int(bafta_nom)
    data_final.loc[data_final.index == i, 'movie_wins'] = int(wins)
    data_final.loc[data_final.index == i, 'movie_nominations'] = int(nominations)
    data_final.loc[data_final.index == i, 'movie_prizes_total'] = int(int(wins) + int(nominations))

#ajout de features pour le pays d'origine du film:
data_final =data_final.assign(movie_country_USA = np.nan)
data_final =data_final.assign(movie_country_CA = np.nan)
data_final =data_final.assign(movie_country_UK = np.nan)
data_final =data_final.assign(movie_country_FR = np.nan)
data_final =data_final.assign(movie_country_GER = np.nan)
data_final =data_final.assign(movie_country_IT = np.nan)
for i in data_final.index:
    origin = data_final.loc[data_final.index == i, 'movie_country'].iloc[0]
    usa = 0
    ca = 0
    uk = 0
    fr = 0
    ger = 0
    it = 0
    mots = origin.split()
    if "States," in mots:
        usa = 1
        mots.remove("United")
        mots.remove("States,")
    if "States" in mots:
        usa = 1
        mots.remove("United")
        mots.remove("States")
    if "USA," in mots:
        usa = 1
        mots.remove("USA,")
    if "USA" in mots:
        usa = 1
        mots.remove("USA")
    if "Kingdom," in mots:
        uk = 1
        mots.remove("United")
        mots.remove("Kingdom,")
    if "Kingdom" in mots:
        uk = 1
        mots.remove("United")
        mots.remove("Kingdom")
    if "UK," in mots:
        uk = 1
        mots.remove("UK,")
    if "UK" in mots:
        uk = 1
        mots.remove("UK")
    if "Canada," in mots:
        ca =1
        mots.remove("Canada,")
    if "Canada" in mots:
        ca =1
        mots.remove("Canada")
    if "France," in mots:
        fr = 1
        mots.remove("France,")
    if "France" in mots:
        fr = 1
        mots.remove("France")
    if "Germany," in mots:
        ger = 1
        mots.remove("Germany,")
    if "Germany" in mots:
        ger = 1
        mots.remove("Germany")  
    if "Italy," in mots:
        it = 1
        mots.remove("Italy,")          
    if "Italy" in mots:
        it = 1
        mots.remove("Italy")  
    if len(mots)>0:
        other = 1
    data_final.loc[data_final.index == i, 'movie_country_USA'] = usa
    data_final.loc[data_final.index == i, 'movie_country_UK'] = uk
    data_final.loc[data_final.index == i, 'movie_country_CA'] = ca
    data_final.loc[data_final.index == i, 'movie_country_FR'] = fr
    data_final.loc[data_final.index == i, 'movie_country_GER'] = ger
    data_final.loc[data_final.index == i, 'movie_country_IT'] = it
      
    
#Ajout features pour les regions du monde ne faisant pas partie des 6 pays les plus cités:
data_final =data_final.assign(movie_country_other_west_europe = 0)
data_final =data_final.assign(movie_country_other_east_europe = 0)
data_final =data_final.assign(movie_country_other_africa = 0)
data_final =data_final.assign(movie_country_other_south_america = 0)
data_final =data_final.assign(movie_country_other_asia = 0)
data_final =data_final.assign(movie_country_other_middle_east = 0)
data_final =data_final.assign(movie_country_other_oceania = 0)
regex_w_eur = 'Spain|Belgium|Sweden|Denmark|Switzerland|Netherlands|Ireland|Norway|Austria|Luxembourg|Iceland|Finland|Portugal|Monaco'
regex_e_eur = 'Hungary|Czech|Poland|Bulgaria|Malta|Greece|Russia|Romania|Yugoslavia|Soviet|Serbia|Croatia|Slovenia|Czechoslovakia|Bosnia|Latvia|Lithuania|Ukraine'
regex_afr = 'Africa|Morocco|Algeria|Tunisia|Lebanon|Niger|Mali|Malawi|Kenya|Ghana|Egypt|Senegal|Congo|Cameroon|Faso'
regex_sth_ame = 'Mexico|Argentina|Brazil|Chile|Peru|Colombia|Uruguay|Venezuela|Bahamas|Paraguay|Cuba|Bolivia'
regex_asia = 'Japan|China|Hong|India|Korea|Thailand|Taiwan|Singapore|Philippines|Indonesia|Vietnam|Cambodia'
regex_mdl_east = 'Arab|Israel|Turkey|Quatar|Jordan|Georgia|Saudi|Syria|Cyprus|Iran|Armenia|Kazakhstan'
regex_ocea = 'Australia|Zealand'
    
for i in data_final.index:
    w_eur=[]
    e_eur=[]
    afr=[]
    sth_ame=[]
    asia=[]
    mdl_east=[]
    ocea=[]
    countries = data_final.loc[data_final.index == i, 'movie_country'].iloc[0]
    w_eur = re.findall(regex_w_eur, countries)
    e_eur = re.findall(regex_e_eur, countries)
    afr = re.findall(regex_afr, countries)
    sth_ame = re.findall(regex_sth_ame, countries)
    asia = re.findall(regex_asia, countries)
    mdl_east = re.findall(regex_mdl_east, countries)
    ocea = re.findall(regex_ocea, countries)
    #print(w_eur, "\n", e_eur, "\n", afr, "\n", sth_ame, "\n", asia, "\n", mdl_east, "\n", ocea)
    
    if len(w_eur)>0:
        data_final.loc[data_final.index == i, 'movie_country_other_west_europe'] = 1
    if len(e_eur)>0:
        data_final.loc[data_final.index == i, 'movie_country_other_east_europe'] = 1
    if len(e_eur)>0:
        data_final.loc[data_final.index == i, 'movie_country_other_east_europe'] = 1
    if len(afr)>0:
        data_final.loc[data_final.index == i, 'movie_country_other_africa'] = 1
    if len(sth_ame)>0:
        data_final.loc[data_final.index == i, 'movie_country_other_south_america'] = 1
    if len(asia)>0:
        data_final.loc[data_final.index == i, 'movie_country_other_asia'] = 1
    if len(mdl_east)>0:
        data_final.loc[data_final.index == i, 'movie_country_other_middle_east'] = 1
    if len(ocea)>0:
        data_final.loc[data_final.index == i, 'movie_country_other_oceania'] = 1

#ajout de features pour le language du film

data_final =data_final.assign(movie_language_eng = np.nan)
data_final =data_final.assign(movie_language_fr = np.nan)
data_final =data_final.assign(movie_language_other = np.nan)

for i in data_final.index:
    language = data_final.loc[data_final.index == i, 'movie_language'].iloc[0]
    eng = 0
    fr = 0
    other = 0
    mots = language.split()
    if "English," in mots:
        eng = 1
        mots.remove("English,")
    if "English" in mots:
        eng = 1
        mots.remove("English")
    if "French," in mots:
        fr = 1
        mots.remove("French,")
    if "French" in mots:
        fr = 1
        mots.remove("French")
    if len(mots)>0:
        other = 1
    data_final.loc[data_final.index == i, 'movie_language_eng'] = eng
    data_final.loc[data_final.index == i, 'movie_language_fr'] = fr
    data_final.loc[data_final.index == i, 'movie_language_other'] = other

    #ajout d'une feature mesurant la violence ou la présence de sexe dans un film selon le modèle de classification américain:

data_final =data_final.assign(movie_subversivness = np.nan)
for i in data_final.index:
    rating = data_final.loc[data_final.index == i, 'movie_rating'].iloc[0]
    if(rating == "G" or rating == "TV-G" or rating == "TV-Y" or rating == "Approved"):
        subversivness = -2
    elif(rating == "PG" or rating == "TV-PG" or rating == "TV-Y7" or rating == "TV-Y7-FV" or rating == "GP"):
        subversivness = -1
    elif(rating == "N/A" or rating == "Not Rated" or rating == "Unrated" or rating == "Passed" or rating == "M/PG"):
        subversivness = 0
    elif(rating == "PG-13" or rating == "TV-14" or rating == "M" or rating == "TV-MA"):
        subversivness = 1
    elif(rating == "R"):
        subversivness = 2
    elif(rating == "MA-17" or rating == "X" or rating == "16+" or "NC-17"):
        subversivness = 3
    data_final.loc[data_final.index == i, 'movie_subversivness'] = subversivness

#ajout d'une feature donnant la moyenne des scores disponibles par films

data_final =data_final.assign(movie_mean_score = np.nan)
for i in data_final.index:
    metascore = int(data_final.loc[data_final.index == i,'movie_metascore'].iloc[0])
    imdb_score = int(data_final.loc[data_final.index == i, 'movie_IMDb_score'].iloc[0])
    metacritic = int(data_final.loc[data_final.index == i, 'movie_metacritic'].iloc[0])
    rotten_tomatoes = int(data_final.loc[data_final.index == i, 'movie_rotten_tomatoes'].iloc[0])
    denom = 0
    score = 0
    if metascore != 0:
        score += metascore
        denom += 1
    if imdb_score != 0:
        score += imdb_score
        denom += 1
    if metacritic != 0:
        score += metacritic
        denom += 1
    if rotten_tomatoes != 0:
        score += rotten_tomatoes
        denom += 1
    if denom > 0:
        score = score/denom
    data_final.loc[data_final.index == i, 'movie_mean_score'] = score  
    
#ajout de features pour clasification plus précise des genre de film:
data_final =data_final.assign(movie_genre_drama = 0)
data_final =data_final.assign(movie_genre_mystery = 0)
data_final =data_final.assign(movie_genre_scifi = 0)
data_final =data_final.assign(movie_genre_comedy = 0)
data_final =data_final.assign(movie_genre_romance = 0)
data_final =data_final.assign(movie_genre_action = 0)
data_final =data_final.assign(movie_genre_adventure = 0)
data_final =data_final.assign(movie_genre_fantasy = 0)
data_final =data_final.assign(movie_genre_war = 0)
data_final =data_final.assign(movie_genre_crime = 0)
data_final =data_final.assign(movie_genre_animation = 0)
data_final =data_final.assign(movie_genre_family = 0)
data_final =data_final.assign(movie_genre_horror = 0)
data_final =data_final.assign(movie_genre_biography = 0)
data_final =data_final.assign(movie_genre_history = 0)
data_final =data_final.assign(movie_genre_music = 0)
data_final =data_final.assign(movie_genre_thriller = 0)
data_final =data_final.assign(movie_genre_musical = 0)
data_final =data_final.assign(movie_genre_documentary = 0)
data_final =data_final.assign(movie_genre_western = 0)
data_final =data_final.assign(movie_genre_short = 0)
data_final =data_final.assign(movie_genre_film_noir = 0)
data_final =data_final.assign(movie_genre_reality_tv = 0)
data_final =data_final.assign(movie_genre_adult = 0)
data_final =data_final.assign(movie_genre_news = 0)

for i in data_final.index:
    liste_genres = []
    tags = data_final.loc[data_final.index == i, 'movie_tags'].iloc[0]
    liste_genres = tags.split()
    for tag in liste_genres:
        genre = tag.replace(",", "")
        if genre == "Drama":
            data_final.loc[data_final.index == i, 'movie_genre_drama'] = 1
        elif genre == "Mystery":
            data_final.loc[data_final.index == i, 'movie_genre_mystery'] = 1
        elif genre == "Sci-Fi":
            data_final.loc[data_final.index == i, 'movie_genre_scifi'] = 1
        elif genre == "Comedy":
            data_final.loc[data_final.index == i, 'movie_genre_comedy'] = 1
        elif genre == "Romance":
            data_final.loc[data_final.index == i, 'movie_genre_romance'] = 1
        elif genre == "Action":
            data_final.loc[data_final.index == i, 'movie_genre_action'] = 1
        elif genre == "Adventure":
            data_final.loc[data_final.index == i, 'movie_genre_adventure'] = 1
        elif genre == "Fantasy":
            data_final.loc[data_final.index == i, 'movie_genre_fantasy'] = 1
        elif genre == "War":
            data_final.loc[data_final.index == i, 'movie_genre_war'] = 1
        elif genre == "Crime":
            data_final.loc[data_final.index == i, 'movie_genre_crime'] = 1
        elif genre == "Animation":
            data_final.loc[data_final.index == i, 'movie_genre_animation'] = 1
        elif genre == "Family":
            data_final.loc[data_final.index == i, 'movie_genre_family'] = 1
        elif genre == "Horror":
            data_final.loc[data_final.index == i, 'movie_genre_horror'] = 1
        elif genre == "Biography":
            data_final.loc[data_final.index == i, 'movie_genre_biography'] = 1
        elif genre == "History":
            data_final.loc[data_final.index == i, 'movie_genre_history'] = 1
        elif genre == "Music":
            data_final.loc[data_final.index == i, 'movie_genre_music'] = 1
        elif genre == "Thriller":
            data_final.loc[data_final.index == i, 'movie_genre_thriller'] = 1
        elif genre == "Musical":
            data_final.loc[data_final.index == i, 'movie_genre_musical'] = 1
        elif genre == "Documentary":
            data_final.loc[data_final.index == i, 'movie_genre_documentary'] = 1
        elif genre == "Western":
            data_final.loc[data_final.index == i, 'movie_genre_western'] = 1
        elif genre == "Short":
            data_final.loc[data_final.index == i, 'movie_genre_short'] = 1
        elif genre == "Film-Noir":
            data_final.loc[data_final.index == i, 'movie_genre_film_noir'] = 1
        elif genre == "Reality-TV":
            data_final.loc[data_final.index == i, 'movie_genre_reality_tv'] = 1
        elif genre == "Adult":
            data_final.loc[data_final.index == i, 'movie_genre_adult'] = 1
        elif genre == "News":
            data_final.loc[data_final.index == i, 'movie_genre_news'] = 1
            
data_final.head()
data_final.to_csv("data_FINAL_movie.csv")

In [ ]:
# mpst_full_data.csv est un fichier csv disponbible sur kaggle

#Si un film de l'étude existe dans le fichier csv mpst, on utilise son synopsis, son plot et ses tags

data_mpts = pd.read_csv("/home/maver/code/datagochi_données/mpst_full_data.csv")
data_gochi = pd.read_csv("/home/maver/code/datagochi_données/data_FINAL_movie.csv")
data_gochi.drop('movie_country_other', axis=1,inplace=True)
id_mpts = data_mpts.imdb_id
id_mpts_list = []
for i in range(len(id_mpts)):
    id_mpts_list.append(id_mpts[i])
    
for id in data_gochi.movie_IMDb_id:
    if id in id_mpts_list:
        plot = data_mpts.loc[data_mpts.imdb_id == id, 'plot_synopsis'].iloc[0]
        plot = plot.replace(";",".")
        tags = data_mpts.loc[data_mpts.imdb_id == id, 'tags'].iloc[0]
        data_gochi.loc[data_gochi.movie_IMDb_id == id, 'movie_plot'] = plot
        data_gochi.loc[data_gochi.movie_IMDb_id == id, 'movie_tags'] = tags 
        
for i in data_gochi.index:
    if type(data_gochi.loc[data_gochi.index == i, 'movie_director'].iloc[0]) != str:
        data_gochi.loc[data_gochi.index == i, 'movie_director'] = "various directors"
    if type(data_gochi.loc[data_gochi.index == i, 'movie_writer'].iloc[0]) != str:
        data_gochi.loc[data_gochi.index == i, 'movie_writer'] = "various writers"
    if type(data_gochi.loc[data_gochi.index == i, 'movie_actors'].iloc[0]) != str:
        data_gochi.loc[data_gochi.index == i, 'movie_actors'] = "various actors"
data_gochi.fillna(0, inplace=True)

#On crée une colonne contenant un paragraphe type à propos d'une film, énumérant ses producteurs, réalisateurs, scénaristes, acteurs, synopsis et tags
data_gochi =data_gochi.assign(movie_vector = np.nan)
for i in data_gochi.index:
    vector = "A movie from " + str(data_gochi.loc[data_gochi.index == i, 'movie_director'].iloc[0]) + ". Written by " + str(data_gochi.loc[data_gochi.index == i, 'movie_writer'].iloc[0]) + ". Starring " + str(data_gochi.loc[data_gochi.index == i, 'movie_actors'].iloc[0]) + ". The movie explores the themes of "+str(data_gochi.loc[data_gochi.index == i, 'movie_tags'].iloc[0])  + " ."+str(data_gochi.loc[data_gochi.index == i, 'movie_plot'].iloc[0])
    data_gochi.loc[data_gochi.index == i, 'movie_vector'] = vector
 
#On crée un dataframe contenant les 768 vecteurs associées à chaque paragraphe de film à partir du sentence transformer de hugging face    
movie_id = data_gochi.movie_IMDb_id.unique()
movie_to_vec = pd.DataFrame(columns=range(768), index= movie_id)
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
for id in movie_id:
    sentences = data_gochi.loc[data_gochi.movie_IMDb_id == id, 'movie_vector'].iloc[0]
    embeddings = model.encode(sentences)
    for i in range(768):
        movie_to_vec[i][id] = embeddings[i]
        
movie_to_vec = movie_to_vec.set_index('Unnamed: 0')
movie_to_vec.index.name = "id"

movie_id = data_gochi.movie_IMDb_id
movie_vec_data = pd.DataFrame(columns=range(768), index= movie_id)

for column in movie_vec_data:
    for i in range(768):
        if column == str(i):
            name = "movie_vect_"+column
            movie_vec_data.rename(columns = {column : name}, inplace=True)
            
movie_id = data_gochi.movie_IMDb_id
movie_vec_data = pd.DataFrame(columns=range(768), index= movie_id)

for id in movie_vec_data.index.unique():
    vec_list = []
    for vec in movie_to_vec.loc[movie_to_vec.index == id].iloc[0]:
        vec_list.append(vec)
    movie_vec_data.loc[movie_vec_data.index == id] = vec_list
    
for column in movie_vec_data:
    for i in range(768):
        if column == i:
            name = "movie_vect_"+str(column)
            movie_vec_data.rename(columns = {column : name}, inplace=True)
            
data_gochi.set_index('movie_IMDb_id', inplace = True)
data_gochi_wth_vec = pd.concat([data_gochi, movie_vec_data], axis=1)

data_gochi_wth_vec.to_csv("data_gochi_wth_vec.csv")